# Digital Identity Onboarding Benchmark — README

## Overview

This module benchmarks a **minimal digital-identity onboarding pipeline**. It simulates issuing user attributes, credentials, and persistent identifiers (PIDs), and measures **per-user onboarding latency** and **total batch time** across different cohort sizes. Results are emitted as two CSV files: a **raw per-user log** and a **summary with descriptive statistics and bootstrap CIs**.

> This is a toy identity stack for performance exploration only. 

---

## What it Measures

For each batch size (e.g., 10, 50, 100, 500, 1000 users) and for multiple runs:

* **Per-user onboarding time** (seconds): time to issue document → credential → PID.
* **Total batch time** (seconds): wall-clock time to onboard the whole cohort.
* **Descriptive statistics** on per-user times: median, IQR, mean, std, p95.
* **95% bootstrap CI** for the median (10,000 resamples by default).

---

## Identity Pipeline (Toy)

* **`AttributeProvider`**: creates a signed “document” from `(user_id, attributes, timestamp)`.
* **`IdentityProvider`**: verifies the document and issues a “credential” with a simple signature.
* **`PIDProvider`**: derives a deterministic **PID** from the credential.
* **`DigitalIdentityManager`**: orchestrates onboarding by calling the three providers and returning `{user_id, credential, pid}`.

All signatures/hashes are **simple SHA-256 over JSON payloads**—sufficient for timing experiments, not for security.

---

## Benchmark Flow

1. For a given `n_users`, loop over users:

   * Build attributes (name, DOB, nationality).
   * Time `mgr.onboard_user(user_id, attrs)`.
2. Collect all per-user times and total time.
3. Repeat for each run and each cohort size.
4. Write:

   * **Raw CSV**: per-user samples.
   * **Summary CSV**: stats + 95% CI via bootstrap.

Warm-up runs are executed (and discarded) to stabilize JIT/caches.

---

## Statistical Methods

* **Descriptive stats**: `median`, `IQR (P75−P25)`, `mean`, `std (ddof=1)`, `p95`.
* **Bootstrap CI** (median):

  * Resample indices with replacement (`n_boot=10000`, default).
  * Compute median for each bootstrap sample.
  * CI = `[alpha/2, 1−alpha/2]` quantiles (default `alpha=0.05` → 95% CI).
* Randomness controlled via seeds for reproducibility.

---

## Outputs

### `onboarding_raw.csv`

| Column              | Description                                             |
| ------------------- | ------------------------------------------------------- |
| `n_users`           | Cohort size for the run                                 |
| `run_id`            | Run number (1..runs)                                    |
| `user_index`        | 1..n\_users                                             |
| `onboarding_time_s` | Per-user time (seconds, string-formatted to 6 decimals) |

### `onboarding_summary.csv`

| Column               | Description                      |
| -------------------- | -------------------------------- |
| `n_users`            | Cohort size                      |
| `run_id`             | Run number                       |
| `per_user_median_s`  | Median per-user time (s)         |
| `per_user_iqr_s`     | IQR (s)                          |
| `per_user_mean_s`    | Mean per-user time (s)           |
| `per_user_std_s`     | Sample std dev (s)               |
| `per_user_p95_s`     | 95th percentile (s)              |
| `median_ci95_low_s`  | Bootstrap 95% CI lower bound (s) |
| `median_ci95_high_s` | Bootstrap 95% CI upper bound (s) |
| `total_time_s`       | Total batch time (s)             |

---

## How to Run

### Quick start (defaults used in `__main__`)

```bash
python identity_benchmark.py
```

This executes:

* cohort sizes: `[10, 50, 100, 500, 1000]`
* `runs=30`, `warmup=5`, `seed=42`
* outputs: `onboarding_raw.csv`, `onboarding_summary.csv`

### (Optional) CLI

The script is structured to accept argparse, but the default `__main__` block ignores it for notebook-friendliness. You can adapt it to wire CLI flags for `user_sizes`, `runs`, `warmup`, `seed`, and output paths.

---

## Interpreting Results

* Use **median** for robust central tendency (heavy-tailed samples are common).
* **IQR** quantifies spread resistant to outliers.
* **p95** highlights tail latency.
* **Bootstrap CI** on the median conveys sampling uncertainty across runs.
* Compare `total_time_s` vs `n_users` to assess **scaling behavior**.

---

## Dependencies

* Python 3.9+
* Standard library: `random`, `csv`, `statistics`, `argparse`, `typing`, `time`, `json`, `hashlib`
* Third-party: `numpy` (for bootstrap & stats)

Install:

```bash
pip install numpy
```

---

## Reproducibility

* PRNGs are seeded both in Python (`random.seed`) and NumPy (`np.random.default_rng(seed)`), with seeds offset per run to avoid identical sequences.

---

## Limitations

* **Security**: hashing/signing is illustrative; no real KMS, PKI, or cryptographic verification.
* **State & storage**: all in-memory; no persistence or concurrency modeling.
* **System effects**: results depend on machine load; consider pinning CPU frequency/cores for tighter comparisons.

---

## Extending

* Replace toy hashing with real signature schemes (e.g., Ed25519) to gauge crypto overheads.
* Add **concurrency** (threading/async) and model contention.
* Record **CPU/Wall/Peak RSS** with `resource`/`psutil`.
* Emit **percentiles (p50..p99.9)** and **ECDF** plots for richer latency analysis.
* Parameterize attributes size to study payload effects.

---

## Files

* `identity_benchmark.py` (example name) — this module.
* `onboarding_raw.csv`, `onboarding_summary.csv` — outputs generated per run.


In [ ]:
import time
import json
import hashlib
import random
import argparse
import csv
import statistics as stats
from typing import List, Tuple
import numpy as np

# -------------------------
# Minimal digital-identity onboarding 
# -------------------------
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode("utf-8")).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode("utf-8")).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode("utf-8")).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode("utf-8")).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        return {"user_id": user_id, "credential": credential, "pid": pid}

# -------------------------
# Stats helpers
# -------------------------
def bootstrap_ci(data: List[float], n_boot: int = 10000, alpha: float = 0.05, seed: int = 123) -> Tuple[float, float]:
    """95% bootstrap CI for the median by default."""
    rng = np.random.default_rng(seed)
    data = np.asarray(data)
    n = len(data)
    idx = rng.integers(0, n, size=(n_boot, n))
    samples = data[idx]
    medians = np.median(samples, axis=1)
    low = np.quantile(medians, alpha/2)
    high = np.quantile(medians, 1 - alpha/2)
    return float(low), float(high)

def describe(sample: List[float]) -> dict:
    median = float(np.median(sample))
    iqr = float(np.percentile(sample, 75) - np.percentile(sample, 25))
    mean = float(np.mean(sample))
    std = float(np.std(sample, ddof=1)) if len(sample) > 1 else 0.0
    p95 = float(np.percentile(sample, 95))
    ci_low, ci_high = bootstrap_ci(sample)
    return {
        "median": median,
        "iqr": iqr,
        "mean": mean,
        "std": std,
        "p95": p95,
        "ci95_low": ci_low,
        "ci95_high": ci_high,
    }

# -------------------------
# Benchmark
# -------------------------
def run_onboarding_once(n_users: int, seed: int = 0) -> Tuple[List[float], float]:
    random.seed(seed)
    mgr = DigitalIdentityManager()
    per_user_times = []
    t0 = time.time()
    for i in range(1, n_users + 1):
        user_id = f"User_{i}"
        attrs = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
        s = time.time()
        _ = mgr.onboard_user(user_id, attrs)
        per_user_times.append(time.time() - s)
    total = time.time() - t0
    return per_user_times, total

def benchmark(user_sizes: List[int], runs: int, warmup: int, seed: int,
              raw_csv: str = "onboarding_raw.csv",
              summary_csv: str = "onboarding_summary.csv"):
    # Prepare CSVs
    with open(raw_csv, "w", newline="") as fraw, open(summary_csv, "w", newline="") as fsum:
        raw_writer = csv.writer(fraw)
        raw_writer.writerow(["n_users", "run_id", "user_index", "onboarding_time_s"])

        sum_writer = csv.writer(fsum)
        sum_writer.writerow([
            "n_users", "run_id",
            "per_user_median_s", "per_user_iqr_s", "per_user_mean_s", "per_user_std_s", "per_user_p95_s",
            "median_ci95_low_s", "median_ci95_high_s",
            "total_time_s"
        ])

        # Warm-up (not recorded)
        for _ in range(warmup):
            run_onboarding_once(user_sizes[0], seed=seed)

        # Real runs
        for n in user_sizes:
            for r in range(1, runs + 1):
                per_user_times, total_time = run_onboarding_once(n, seed=seed + r + n)
                # write raw
                for idx, t in enumerate(per_user_times, start=1):
                    raw_writer.writerow([n, r, idx, f"{t:.6f}"])
                # write summary
                d = describe(per_user_times)
                sum_writer.writerow([
                    n, r,
                    f"{d['median']:.6f}", f"{d['iqr']:.6f}", f"{d['mean']:.6f}", f"{d['std']:.6f}", f"{d['p95']:.6f}",
                    f"{d['ci95_low']:.6f}", f"{d['ci95_high']:.6f}",
                    f"{total_time:.6f}"
                ])

if __name__ == "__main__":
    # Ignore argparse in notebooks
    user_sizes = [10, 50, 100, 500, 1000]
    benchmark(user_sizes=user_sizes, runs=30, warmup=5, seed=42,
              raw_csv="onboarding_raw.csv",
              summary_csv="onboarding_summary.csv")



# CBDC Offline/Online Payment Simulator — README

## Overview

This simulator models a minimal CBDC ecosystem with:

* **Withdrawals** (blind-signature–style placeholder),
* **Offline payments** (Alice → Bob) using simulated **zk-SNARK proofs**, **AES/RSA hybrid encryption**, and a lightweight **pyipv8-like** transport wrapper,
* **Online deposits** (Alice → RIO/PAD) with secure token packaging,
* **Bank deposits** (Bob → Central Bank) with basic anti–double-spending checks,
* **Bob → Carla scenario** (contract, zk proof, encrypted packet via RIO),
* **Decryption request** (PAD-style: threshold check, double encryption/decryption, recovery of secret `R_sf`),
* **Identity onboarding & verification** (toy credentials + metrics),
* **Metrics export** (latency, CPU load, per-phase throughput → CSV).

> All crypto proofs/transport are **simulated** for research and demo purposes—**not** production security.

---

## Main Components

* **`CentralBank`**: issues coins, signs coins, verifies deposits, tracks used nonces & recorded transactions.
* **`OfflineWallet`**: holds keys, coins, and prepares transactions (Alice, Bob, Carla).
* **`ZKSNARK` (simulated)**: generates/verifies a hash-based stand-in proof.
* **`PyIPv8Transport`**: tiny wrapper emulating message encapsulation/logging.
* **`NFCMock`**: JSON send/receive shim.
* **`Environment`**: records transactions + metrics; exports CSV.
* **`OfflinePaymentSimulation`**: orchestrates workflows (withdrawals, offline payments, online PAD deposit, Bob→Carla scenario, PAD decryption request).
* **`PADLedgerInterface`**: smart contract wrapper (Ethereum/Ganache) to post transactions on-chain.
* **Identity Stack**: `AttributeProvider`, `IdentityProvider`, `PIDProvider`, `DigitalIdentityManager` + `IdentityMetrics`.

---

## Key Flows

### Withdrawal (CentralBank → Alice)

* Alice blinds (placeholder), bank signs, coin issued & stored in wallet.

### Offline Payment (Alice → Bob)

* Alice builds transaction + simulated zk proof.
* AES payload, RSA-encrypted AES key, sent via `PyIPv8Transport`.
* Bob decrypts, validates, checks nonce & logs to prevent replays.

### Bank Deposit (Bob → CentralBank)

* Proof verification + signature check + nonce tracking to avoid double spending.

### Online Deposit via PAD (Alice → own account)

* Token `T` + blinded coin packaged; hybrid encryption and logging.

### Bob Pays Carla (Scenario)

* Contract string, proof generation, encrypted packet “to RIO”, timing recorded.

### PAD Decryption Request

If token `T` > threshold:

* RIO publishes a decryption request with φ = G(H(T), K\_pub).
* Bob recovers the secret by double decryption (RSA + AES).
* Final recovered secret `R_sf` is hashed and logged.
* Latency + CPU load recorded in metrics.

### Digital Identity

* Issue “credentials”, compute simple signatures, verify, track onboarding/verification times.

---

## Metrics & Outputs

* **Console logs**: each step’s status + timings.
* **CSV files**:

  * `simulation_metricswithpad.csv` — per-transaction metrics:

    * `transaction_id, period, to_agent, from_agent, settlement_type, amount, description, latency, zk_proof_generation_time, zk_proof_verification_time, transaction_verification_time, load_computing, gas_used, decryption_time, decryption_cpu_time, status, throughput`.
  * `identity_metrics.csv` — per-user onboarding/verification times.

Throughput is computed **per settlement type** and overall.

---

## Installation

```bash
# Recommended Python 3.10+
pip install pycryptodome pandas rsa web3

# Optional (if you later integrate real pyipv8)
# pip install pyipv8
```

> If `pyipv8` is absent, a simulated payload class is used automatically.

---

## Running

Execute the script directly to run an interactive session:

```bash
python cbdc_simulator.py
```

You will be prompted for:

* number/amount of **withdrawals**,
* number/amount of **offline transactions**,
* number of **deposits**,
* number/amount for **Bob→Carla** scenario,
* amount for **Online PAD Deposit**,
* threshold for **Decryption Request**.

At the end, the simulator exports the CSV metrics.

---

## Configuration Tips

* **Randomness**: coin IDs and tokens use Python’s `random`/`get_random_bytes`.
* **Key sizes**: RSA 2048-bit (via `rsa` and PyCryptodome’s RSA where needed).
* **Proofs**: `ZKSNARK` is a placeholder (hash of `(pk, amount, zk_key)`).

---

## Limitations (Important)

* **Security**: All proof systems, blind signatures, and transport are simplified.
  Do **not** use for production security or cryptographic validation.
* **Replay & State**: Nonce tracking exists but is simplified; no persistent storage.
* **Networking**: No real network; `PyIPv8Transport` only logs and returns payloads.
* **Compliance**: Not aligned to any regulatory/certification standard.

---

## File Map (typical)

* `cbdc_simulator.py` — main simulator (the code you shared).
* `identity_metrics.csv` — identity onboarding metrics (generated).
* `simulation_metricswithpad.csv` — transaction metrics (generated).

---

## Extending the Simulator

* **Real proofs**: integrate a genuine proving system (Groth16/Plonk) and circuits.
* **Transport**: replace `PyIPv8Transport` with actual IPv8 or another P2P layer.
* **Double-spending**: persistent logs, richer state machines, conflict resolution.
* **PAD/RIO**: model contract calls, receipts, on-chain gas & status codes.
* **Decryption**: refine multi-party threshold logic, trustee/validator shares.
* **Privacy**: pseudonym rotation, unlinkability tests, selective disclosure.
* **Evaluation**: add workload generators and benchmark suites.

---

## Quick Example (Suggested Flow)

1. Withdraw 2 coins of 50 units each.
2. Do 3 offline payments (random amount).
3. Deposit 1 coin to the bank.
4. Run 1 Bob→Carla payment of 50 units.
5. Execute 1 online PAD deposit (Alice).
6. Trigger a PAD decryption request with threshold check.
7. Inspect console logs and the two CSV outputs.

---

## Citation / Usage Note

If you include this simulator in a paper or artifact review, mention that cryptographic parts are **simulated** and timings reflect **prototype logic** (not optimized crypto).

---

## License

Research/demo code; choose and add a license as appropriate for your repository (e.g., MIT/BSD/Apache-2.0).


In [ ]:
# !pip install pycryptodome
# !pip install pyipv8
# cbdc_simulator.py
import time
import random
import hashlib
import rsa
import pandas as pd
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import json
import base64
import os


# --- Handling of pyipv8 and definition of a concrete class SimplePayload ---
try:
    from ipv8.messaging.payload import Payload
except ImportError:
    print("pyipv8 not found, using a simulated Payload object.")
    class Payload:
        def __init__(self):
            self.data = None
        def __str__(self):
            return str(self.data)
    SimplePayload = Payload
else:
    class SimplePayload(Payload):
        def __init__(self):
            super().__init__()
            self.data = None
        def to_pack_list(self):
            return [self.data]
        @classmethod
        def from_unpack_list(cls, lst):
            instance = cls()
            instance.data = lst[0]
            return instance

def xor_bytes(a, b):
    """Returns the byte-wise XOR of two byte sequences of the same length."""
    return bytes(x ^ y for x, y in zip(a, b))

# --- PyIPv8Transport class using SimplePayload ---
class PyIPv8Transport:
    @staticmethod
    def encapsulate(packet, sender="Alice", receiver="Bob"):
        header = {
            "protocol": "ipv8",
            "timestamp": time.time(),
            "sender": sender,
            "receiver": receiver
        }
        payload = SimplePayload()
        payload.data = {"header": header, "data": packet}
        return payload

    @staticmethod
    def send(packet, sender="Alice", receiver="Bob"):
        payload = PyIPv8Transport.encapsulate(packet, sender, receiver)
        print("PyIPv8Transport: Sending payload:")
        print(payload)
        return payload

    @staticmethod
    def receive(payload):
        print("PyIPv8Transport: Received payload:")
        print(payload)
        data = payload.data.get("data")
        return data

# --- Other unchanged classes (NFCMock, ZKSNARK, CentralBank, OfflineWallet, Environment) ---
class NFCMock:
    @staticmethod
    def send(transaction_data):
        print("NFCMock: Transaction data sent.")
        return json.dumps(transaction_data)

    @staticmethod
    def receive(transaction_json):
        print("NFCMock: Transaction data received.")
        return json.loads(transaction_json)

class ZKSNARK:
    @staticmethod
    def generate_key():
        return "simulated_zk_key"

    @staticmethod
    def generate_proof(private_key, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()  
        return hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()

    @staticmethod
    def verify_proof(proof, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()  
        expected_proof = hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()
        return proof == expected_proof

class CentralBank:
    def __init__(self, name):
        self.name = name
        self.coins_issued = []
        self.key_pair = rsa.newkeys(2048)
        self.deposited_coins = set()
        self.used_nonces = set()
        self.logged_transactions = []
      
    def issue_coin(self, recipient, value):
        coin_id = hashlib.sha256(str(random.getrandbits(256)).encode()).hexdigest()
        signature = self.sign_coin(value, coin_id)
        signature_b64 = base64.b64encode(signature).decode('utf-8')
        coin = {
            "coin_id": coin_id,
            "value": value,
            "signature": signature_b64
        }
        self.coins_issued.append(coin)
        return coin

    def sign_coin(self, value, coin_id):
        message = f"{value}{coin_id}".encode()
        return rsa.sign(message, self.key_pair[1], 'SHA-256')

    def sign_blinded_hash(self, blinded_hash):
        return hashlib.sha256(f"{blinded_hash}{self.name}".encode()).hexdigest()

    def verify_and_deposit_coin(self, coin, sender_public_key, proof, zk_key, nonce):
        coin_hash = hashlib.sha256((str(coin['value']) + coin['coin_id'] + coin['signature']).encode()).hexdigest()
        if coin_hash in self.deposited_coins:
            return "Coin Already Defunded or Verification Failed"
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, self.key_pair[0])
        except rsa.VerificationError:
            return "Coin Already Defunded or Verification Failed"
        if nonce in self.used_nonces:
            return "Nonce already used. Double spending detected."
        self.used_nonces.add(nonce)
        amount = coin['value']
        expected_proof = hashlib.sha256(f"{sender_public_key}{amount}{zk_key}".encode()).hexdigest()
        if proof != expected_proof:
            return "Coin Already Defunded or Verification Failed"
        transaction_record = (sender_public_key.save_pkcs1(), nonce, proof)
        if transaction_record in self.logged_transactions:
           return "Double spending detected: Transaction already recorded."
        self.logged_transactions.append(transaction_record)
        self.deposited_coins.add(coin_hash)
        return "Coin Deposited Successfully"

class OfflineWallet:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.balance = 100
        self.key_pair = self.generate_key_pair()
        self.zk_key = ZKSNARK.generate_key()
        self.nonce = 0
        self.withdrawal_latencies = []
        self.deposit_latencies = []
        self.coins = []
        self.received_logs = []

    def log_transaction(self, sender_key, nonce, proof):
        self.received_logs.append((sender_key.save_pkcs1(), nonce, proof))

    def has_received_transaction(self, sender_key, nonce, proof):
        return (sender_key.save_pkcs1(), nonce, proof) in self.received_logs

    def generate_key_pair(self):
        return rsa.newkeys(2048)

    def store_coin(self, coin):
        self.coins.append(coin)
        print(f"Coin stored: {coin}")

    def prepare_transaction(self, recipient, amount):
        self.nonce += 1
        if not self.coins:
            raise ValueError("No coins available to send.")
        coin = self.coins[0]
        proof = ZKSNARK.generate_proof(self.key_pair[1], recipient.key_pair[0], amount, self.zk_key)
        return {
            "amount": amount,
            "coin": coin.copy(),
            "recipient_key": base64.b64encode(recipient.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "from_agent_key": base64.b64encode(self.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.nonce,
            "timestamp": time.time()
        }

    def blind_coin(self):
        f = hashlib.sha256(self.key_pair[0].save_pkcs1()).hexdigest()
        blinding_factor = random.randint(1, 10**5)
        blinded_hash = (int(f, 16) * blinding_factor) % (2**256)
        return {"hash": f, "blinded_hash": blinded_hash, "blinding_factor": blinding_factor}

class Environment:
    def __init__(self):
        self.measurement = {
            "transaction_id": [],
            "period": [],
            "to_agent": [],
            "from_agent": [],
            "settlement_type": [],
            "amount": [],
            "description": [],
            "latency": [],
            "zk_proof_generation_time": [],
            "zk_proof_verification_time": [],
            "transaction_verification_time": [],
            "load_computing": [],
            "status": []
        }

    def log_transaction(self, transaction_id, period, to_agent, from_agent, settlement_type, amount, description,
                        latency=None, zk_proof_gen_time=None, zk_proof_ver_time=None, transaction_ver_time=None,
                        load_computing=None, status="Pending"):
        self.measurement["transaction_id"].append(transaction_id)
        self.measurement["period"].append(period)
        self.measurement["to_agent"].append(to_agent)
        self.measurement["from_agent"].append(from_agent)
        self.measurement["settlement_type"].append(settlement_type)
        self.measurement["amount"].append(amount)
        self.measurement["description"].append(description)
        self.measurement["latency"].append(latency)
        self.measurement["zk_proof_generation_time"].append(zk_proof_gen_time)
        self.measurement["zk_proof_verification_time"].append(zk_proof_ver_time)
        self.measurement["transaction_verification_time"].append(transaction_ver_time)
        self.measurement["load_computing"].append(load_computing)
        self.measurement["status"].append(status)

    def generate_metrics(self):
        df = pd.DataFrame(self.measurement)
        total_transactions = len(df)
        total_latency = df["latency"].sum()
        overall_throughput = total_transactions / total_latency if total_latency > 0 else 0
      
        phase_throughput = {}
        for phase in df['settlement_type'].unique():
            phase_df = df[df['settlement_type'] == phase]
            phase_count = len(phase_df)
            phase_total_latency = phase_df["latency"].sum()
            phase_throughput[phase] = phase_count / phase_total_latency if phase_total_latency > 0 else 0
        df["throughput"] = df["settlement_type"].apply(lambda phase: phase_throughput.get(phase, 0))
        
        for col in ["zk_proof_generation_time", "zk_proof_verification_time", "load_computing", "throughput"]:
            df[col] = df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) and isinstance(x, (float, int)) else x)
        
        print("\n--- Simulation Metrics ---")
        print(f"Total Transactions: {total_transactions}")
        print(f"Total Latency (seconds): {total_latency:.6f}")
        print(f"Overall Throughput (transactions/sec): {overall_throughput:.6f}")
        
        print("\nThroughput per phase:")
        for phase, tp in phase_throughput.items():
            print(f"  {phase}: {tp:.6f} transactions/sec")
        df.to_csv("simulation_metrics_test_paper2.csv", index=False)

# --- Integration of the scenario "Bob pays Carla" ---
class OfflinePaymentSimulation:
    def __init__(self):
        self.environment = Environment()
        self.alice_wallet = OfflineWallet("Alice")
        self.bob_wallet = OfflineWallet("Bob")
        self.carla_wallet = OfflineWallet("Carla")  # New wallet for Carla
        self.central_bank = CentralBank("Central Bank")
        self.used_nonces = set()

    def validate_coin_signature(self, coin, public_key):
        if not coin or "signature" not in coin:
            print("Invalid coin: Missing or no signature.")
            return False
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, public_key)
            print("Coin signature is valid.")
            return True
        except rsa.VerificationError:
            print("Invalid coin: Signature verification failed.")
            return False

    def verify_sender_legitimacy(self, sender_public_key, known_keys):
        return any(sender_public_key.save_pkcs1() == key.save_pkcs1() for key in known_keys)
     
    def verify_nonce(self, nonce, used_nonces):
        if nonce in used_nonces:
            print(f"Nonce {nonce} has already been used. Double spending detected.")
            return False
        used_nonces.add(nonce)
        return True

    def verify_recipient(self, transaction_data, expected_recipient_key):
        recipient_key = transaction_data.get("recipient_key")
        if not isinstance(recipient_key, rsa.PublicKey):
            print("Failure: recipient_key is not an rsa.PublicKey object.")
            return False
        return recipient_key.save_pkcs1() == expected_recipient_key.save_pkcs1()


    def simulate_deposit(self):
        """Simulates the deposit of a coin to the central bank."""
        if not self.bob_wallet.coins:
            print("Bob has no coin to deposit.")
            return

        coin = self.bob_wallet.coins[0]  # Select the first coin for deposit
        amount = coin['value']

        computing_start = time.process_time()
        start_time = time.time()

        # Generate ZK proof for deposit
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1], self.central_bank.key_pair[0], amount, self.bob_wallet.zk_key)

        # Prepare deposit data
        deposit_data = {
            "coin": coin.copy(),
            "sender_public_key": base64.b64encode(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.bob_wallet.nonce + 1,  # Increment nonce for the deposit
            "timestamp": time.time()
        }

        # Simulate the deposit process
        deposit_result = self.central_bank.verify_and_deposit_coin(
            coin,
            self.bob_wallet.key_pair[0],
            proof,
            self.bob_wallet.zk_key,
            deposit_data["nonce"]  # Use the incremented nonce for deposit
        )

        if deposit_result == "Coin Deposited Successfully":
            self.bob_wallet.coins.pop(0)  # Remove deposited coin from wallet
            self.bob_wallet.balance -= amount  # Update wallet balance
            self.bob_wallet.nonce += 1  # Increment wallet's nonce after successful deposit
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            print(f"Deposit completed successfully. Total time: {latency:.6f}s, "
                  f"Computing load (CPU): {computing_load:.6f}s.")

            self.environment.log_transaction(
                transaction_id=len(self.environment.measurement["transaction_id"]),
                period=time.time(),
                to_agent="Central Bank",
                from_agent=self.bob_wallet.owner_id,
                settlement_type="deposit",
                amount=amount,
                description="Coin Deposit",
                latency=latency,
                zk_proof_gen_time=None,  # Add timing if needed
                zk_proof_ver_time=None,  # Add timing if needed
                transaction_ver_time=None,  # Add timing if needed
                load_computing=computing_load,
                status="Success"
            )
        else:
            print(f"Deposit failed: {deposit_result}")

    # --- Withdrawal method ---
    def simulate_withdrawal(self, amount):
        print(f"Alice withdraws {amount} units...")
        computing_start = time.process_time()
        total_start_time = time.time()
        blinded_coin = self.alice_wallet.blind_coin()
        interaction_start_time = time.process_time()
        blinded_signature = self.central_bank.sign_blinded_hash(blinded_coin["blinded_hash"])
        interaction_cpu_time = time.process_time() - interaction_start_time
        message = f"{amount}{self.alice_wallet.owner_id}".encode()
        unblinded_signature = rsa.sign(message, self.central_bank.key_pair[1], 'SHA-256')
        coin = self.central_bank.issue_coin(self.alice_wallet.owner_id, amount)
        self.alice_wallet.store_coin(coin)
        total_latency = time.time() - total_start_time
        computing_load = time.process_time() - computing_start
        print(f"Withdrawal completed successfully. Total time: {total_latency:.6f}s, "
              f"Interaction time (CPU): {interaction_cpu_time:.6f}s, "
              f"Computing load (CPU): {computing_load:.6f}s.")
        self.environment.log_transaction(
            transaction_id=len(self.environment.measurement["transaction_id"]),
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Central Bank",
            settlement_type="withdrawal",
            amount=amount,
            description="Withdrawal with blinding",
            latency=total_latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=interaction_cpu_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Offline payment method (Alice pays Bob) ---
    def simulate_offline_transaction(self, amount=None):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        if amount is None:
            amount = random.randint(10, 50)
        if not self.alice_wallet.coins:
            print("No coin available for an offline transaction.")
            return
        coin = self.alice_wallet.coins[0]
        if amount > coin["value"]:
            print(f"Requested amount ({amount}) greater than coin value ({coin['value']}). Using the coin's value.")
            amount = coin["value"]
        elif amount < coin["value"]:
            print("Partial transaction not supported. Using the full coin value.")
            amount = coin["value"]
        computing_start = time.process_time()
        start_time = time.time()
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.bob_wallet.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start_time
        transaction_data = self.alice_wallet.prepare_transaction(self.bob_wallet, amount)
        transaction_data.update({
            "transaction_id": transaction_id,
            "from_agent": self.alice_wallet.owner_id,
            "to_agent": self.bob_wallet.owner_id,
            "nonce": transaction_data["nonce"]
        })
        json_data = json.dumps(transaction_data).encode('utf-8')
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_data)
        packet = {
            "encrypted_aes_key": base64.b64encode(
                PKCS1_OAEP.new(
                    RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM'))
                ).encrypt(aes_key)
            ).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Sending the encapsulated transaction via pyipv8...")
        ipv8_payload = PyIPv8Transport.send(packet, sender=self.alice_wallet.owner_id, receiver=self.bob_wallet.owner_id)
        received_packet = PyIPv8Transport.receive(ipv8_payload)
        try:
            enc_aes_key = base64.b64decode(received_packet["encrypted_aes_key"])
            ciphertext = base64.b64decode(received_packet["ciphertext"])
            tag = base64.b64decode(received_packet["tag"])
            nonce = base64.b64decode(received_packet["nonce"])
            bob_private_pem = self.bob_wallet.key_pair[1].save_pkcs1(format='PEM')
            bob_private_key_py = RSA.importKey(bob_private_pem)
            cipher_rsa_bob = PKCS1_OAEP.new(bob_private_key_py)
            decrypted_aes_key = cipher_rsa_bob.decrypt(enc_aes_key)
            cipher_aes_bob = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=nonce)
            decrypted_data = cipher_aes_bob.decrypt_and_verify(ciphertext, tag)
            received_transaction = json.loads(decrypted_data.decode('utf-8'))
            if isinstance(received_transaction["from_agent_key"], str):
                try:
                    sender_public_key = rsa.PublicKey.load_pkcs1(base64.b64decode(received_transaction["from_agent_key"]))
                    received_transaction["from_agent_key"] = sender_public_key
                    print("Sender public key correctly loaded.")
                except Exception as e:
                    print(f"Error while decoding the sender's key: {e}")
                    return
            if isinstance(received_transaction["recipient_key"], str):
                try:
                    received_transaction["recipient_key"] = rsa.PublicKey.load_pkcs1(
                        base64.b64decode(received_transaction["recipient_key"])
                    )
                    print("recipient_key correctly loaded as PublicKey.")
                except Exception as e:
                    print(f"Error while decoding recipient_key: {e}")
                    return
            if not isinstance(received_transaction["recipient_key"], rsa.PublicKey):
                print("Error: recipient_key is not a valid rsa.PublicKey object after processing.")
                return
            print(f"Received recipient_key: {received_transaction['recipient_key']}")
            print(f"Type of recipient_key: {type(received_transaction['recipient_key'])}")
            verification_start_time = time.time()
            if not self.validate_coin_signature(received_transaction.get("coin"), self.central_bank.key_pair[0]):
                print("Failed to verify the coin signature.")
                return
            sender_public_key = received_transaction.get("from_agent_key")
            if not self.verify_sender_legitimacy(sender_public_key, [self.alice_wallet.key_pair[0]]):
                print("Failed to verify the sender’s legitimacy.")
                return
            else:
                print("Sender legitimacy verification successful: The sender is legitimate.")
            proof = received_transaction.get("proof")
            nonce = received_transaction.get("nonce")
            if self.bob_wallet.has_received_transaction(sender_public_key, nonce, proof):
                print("Double spending detected: transaction already received.")
                return
            else:
                self.bob_wallet.log_transaction(sender_public_key, nonce, proof)
                print("Transaction successfully recorded in the logs.")
            if not self.verify_nonce(received_transaction.get("nonce"), self.used_nonces):
                print("Failed to verify the nonce.")
                return
            zk_proof_ver_start_time = time.time()
            proof_valid = ZKSNARK.verify_proof(
                proof,
                self.bob_wallet.key_pair[0],
                amount,
                self.alice_wallet.zk_key
            )
            zk_proof_ver_time = time.time() - zk_proof_ver_start_time
            if not proof_valid:
                print("zk-SNARK proof verification failed.")
                return
            else:
                print("zk-SNARK proof verification successful: The proof is valid.")
            print("Nonce is verified and unique, and the zk-SNARK proof is verified and unique.")
            verification_time = time.time() - verification_start_time
            self.alice_wallet.coins.pop(0)
            received_transaction["coin"]["nonce"] = received_transaction["nonce"]
            received_transaction["coin"]["recipient"] = self.bob_wallet.owner_id
            self.bob_wallet.store_coin(received_transaction["coin"])
            self.alice_wallet.balance -= amount
            self.bob_wallet.balance += amount
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            self.environment.log_transaction(
                transaction_id=transaction_id,
                period=time.time(),
                to_agent=self.bob_wallet.owner_id,
                from_agent=self.alice_wallet.owner_id,
                settlement_type="offline_transaction",
                amount=amount,
                description="Offline Transaction",
                latency=latency,
                zk_proof_gen_time=zk_proof_gen_time,
                zk_proof_ver_time=zk_proof_ver_time,
                transaction_ver_time=verification_time,
                load_computing=computing_load,
                status="Success"
            )
        except Exception as e:
            print(f"Error during the transaction: {e}")

    # --- New method: scenario of Bob paying Carla ---
    def simulate_bob_pays_carla(self, amount):
        # Define the transaction ID as the current number of transactions + 1
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        # Start measuring time (wall time and CPU time)
        computing_start = time.process_time()
        start_time = time.time()
    
        print(f"\n=== Scenario: Bob pays Carla for {amount} units ===")
        # Step 1: Bob and Carla agree on a contract
        contract = f"Contract_Bob_pays_Carla_{amount}"
        print("Bob and Carla agree on the contract:", contract)
        
        # Step 2: Bob selects the RIO attestation (dummy value)
        rio_cert = "RIO_pk_cert_dummy"
        print("Bob selects the RIO attestation:", rio_cert)
        
        # Step 3: Carla provides Bob with one of her pseudonymous identities
        carla_pseudonym = self.carla_wallet.key_pair[0].save_pkcs1().decode('utf-8')
        print("Carla provides Bob with her pseudonymous identifier.")
        
        # Step 4: Bob selects an e-coin corresponding to the desired amount
        if not self.bob_wallet.coins:
            print("Bob has no coin available for the payment.")
            return
        ecoin = self.bob_wallet.coins[0]
        print("Selected e-Coin:", ecoin)
        
        # Step 5: Bob signs the contract with his e-coin
        signature_contract = rsa.sign(contract.encode('utf-8'), self.bob_wallet.key_pair[1], 'SHA-256')
        signature_contract_b64 = base64.b64encode(signature_contract).decode('utf-8')
        print("Bob signs the contract:", signature_contract_b64)
        
        # Step 6: Bob generates the zk-SNARK proof of payment
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1],
                                       self.carla_wallet.key_pair[0],
                                       amount,
                                       self.bob_wallet.zk_key)
        print("Generated proof:", proof)
        # Measure the proof verification time
        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(
                    proof,
                    self.carla_wallet.key_pair[0],
                    amount,
                    self.bob_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start
        if not proof_valid:
            print("zk-SNARK proof verification failed.")
            return
        else:
            print("zk-SNARK proof verification successful: The proof is valid.")

        # Step 7: Bob constructs the transaction packet including the following data
        transaction_payload = {
            "contract": contract,
            "contract_signature": signature_contract_b64,
            "proof": proof,
            "RIO_cert": rio_cert,
            "amount": amount,
            "bob_pseudonym": self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8'),
            "carla_pseudonym": carla_pseudonym,
            "timestamp": time.time()
        }
        json_payload = json.dumps(transaction_payload).encode('utf-8')
        
        # Step 8: Encryption of the transaction packet (RIO simulator)
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_payload)
    
        # Generation of a key pair for RIO
        rio_key_pair = rsa.newkeys(2048)
        rio_public = rio_key_pair[0]
        rio_private = rio_key_pair[1]
    
        # Encryption of the AES key with RIO's public key
        rio_public_pem = rio_public.save_pkcs1(format='PEM')
        cipher_rsa_enc = PKCS1_OAEP.new(RSA.importKey(rio_public_pem))
        encrypted_aes_key = cipher_rsa_enc.encrypt(aes_key)
    
        encrypted_packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Bob-to-Carla transaction (encrypted for RIO):", encrypted_packet)
    
        # --- Simulation of the verification phase by the RIO ---
        # To decrypt, the RIO uses its private key
        rio_private_pem = rio_private.save_pkcs1(format='PEM')
        cipher_rsa_dec = PKCS1_OAEP.new(RSA.importKey(rio_private_pem))
        # Decryption of the AES key
        decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)
    
        # Use the decrypted key to decrypt the payload
        cipher_aes_rio = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=cipher_aes.nonce)
        decrypted_payload = cipher_aes_rio.decrypt_and_verify(ciphertext, tag)
        # Here, the RIO verifies the payload data (contract, signature, proof, etc.)
        # Initialize verification_start_time here within simulate_bob_pays_carla
        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time: {transaction_verification_time:.6f} seconds")

    # ----------------------------
    # TOTAL LATENCY AND CPU LOAD CALCULATION"
    
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
    # ----------------------------   
    # RECORDING METRICS FOR THE SCENARIO "BOB PAYS CARLA"
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.carla_wallet.owner_id,
            from_agent=self.bob_wallet.owner_id,
            settlement_type="offline transaction 2 (bob_to_carla)",  # or "bob_to_carla"
            amount=amount,
            description="Payment from Bob to Carla",
            latency=latency,
            zk_proof_gen_time=None,  # You can add proof generation time if needed
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=computing_load,
            status="Success"
        )
    # ----------------------------
    def simulate_online_deposit_PAD(self, amount):
        """
        Online Deposit/Transfer scenario via PAD.
        This scenario simulates the online deposit (or transfer to one's own account)
        through the PAD system managed by the RIO.
        """
        print(f"\n=== Online Deposit scenario via PAD for {amount} units ===")
        # Define time variables and the transaction identifier
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()
        
        # -- Step 0: Transaction Verification (similar to the verification of offline transactions) --
        # Here, the RIO verifies that the transaction is not a double spending.
        # For the simulation, we use the existing zk-SNARK proof verification.
        # Suppose we use a coin deposited by Alice for the online deposit.
        if not self.alice_wallet.coins:
            print("No coin available for an online deposit.")
            return
        coin = self.alice_wallet.coins[0]
        # Proof generation and verification (similar to the other scenario)
        zk_proof_start = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.central_bank.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start

        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(proof,
                                           self.central_bank.key_pair[0],
                                           amount,
                                           self.alice_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start

        if not proof_valid:
            print("zk-SNARK proof verification failed during the online transaction.")
            return
        print("Successful verification of zk-SNARK proof.")

        # -- Step 1 : Securing the blinded coin F_s --
        # We assume that Alice's e-coin is used as the blinded coin F_s.
        Fs = self.alice_wallet.blind_coin()  # F_s
        # Generate a token T (for the simulation, a random number as a string)
        T = str(random.randint(1000, 9999))
        print("Generated Token T :", T)
        # Generate a symmetric key K_sym
        K_sym = get_random_bytes(32)
        # Encrypt (T, F_s) with K_sym (we concatenate T and part of Fs)
        data_to_encrypt = T.encode('utf-8') + json.dumps(Fs, sort_keys=True).encode('utf-8')
        cipher_pad = AES.new(K_sym, AES.MODE_GCM)
        C, tag = cipher_pad.encrypt_and_digest(data_to_encrypt)
        nonce_pad = cipher_pad.nonce
        # C represents the encrypted content containing the token and F_s
        
        # Now, sign and encrypt the content with Alice's private key and then with the recipient's public key
        # (here, for the online deposit, we assume that the transaction remains on Alice's same account)
        # For the simulation, we will sign C with Alice's private key:
        signature_C = rsa.sign(C, self.alice_wallet.key_pair[1], 'SHA-256')
        signature_C_b64 = base64.b64encode(signature_C).decode('utf-8')
        # We form a packet C' that combines the content C and its signature
        C_prime = {
            "encrypted_content": base64.b64encode(C).decode('utf-8'),
            "signature": signature_C_b64
        }
        # Then, encrypt C' with the recipient's public key.
        # In the case of a deposit on the same account, we can encrypt with Alice's public key.
        # --- Hybrid encryption of C_prime ---
        # Convert C_prime to JSON (data to be encrypted)
        C_prime_json = json.dumps(C_prime).encode('utf-8')

        # 1. Generate a symmetric key K_PAD to encrypt C_prime_json with AES
        K_PAD = get_random_bytes(32)
        cipher_pad2 = AES.new(K_PAD, AES.MODE_GCM)
        C_prime_ciphertext, C_prime_tag = cipher_pad2.encrypt_and_digest(C_prime_json)
        nonce_pad2 = cipher_pad2.nonce

        # 2. Encrypt the symmetric key K_PAD with Alice's public key (in this case, the online deposit is on her own account)
        cipher_rsa_for_PAD = PKCS1_OAEP.new(RSA.importKey(self.alice_wallet.key_pair[0].save_pkcs1(format='PEM')))
        encrypted_K_PAD = cipher_rsa_for_PAD.encrypt(K_PAD)

        # 3. Form the final packet for C_prime by grouping the encrypted key and the ciphertext
        C_prime_encrypted = {
            "encrypted_key": base64.b64encode(encrypted_K_PAD).decode('utf-8'),
            "ciphertext": base64.b64encode(C_prime_ciphertext).decode('utf-8'),
            "tag": base64.b64encode(C_prime_tag).decode('utf-8'),
            "nonce": base64.b64encode(nonce_pad2).decode('utf-8')
        }

        
        # Masking the symmetric key with a mask R_sym
        R_sym = get_random_bytes(32)
        S_m = xor_bytes(K_sym, R_sym)
        # Generate a dummy share for trustees (N_t, for example 3)
        N_t = b"3"
        S_t = hashlib.sha256(S_m + N_t).hexdigest()
        # Generate a dummy share for validators (N_v, for example 2)
        N_v = b"2"
        V = hashlib.sha256(b"R" + self.alice_wallet.key_pair[0].save_pkcs1() + N_v).hexdigest()
        
        # Encrypt dummy shares for trustees and validators (simulated here)
        E_t = "encrypted_trustee_shares_dummy"
        E_v = "encrypted_validator_shares_dummy"
        
        # Final data transmission structure D_ts
        D_ts = {
            "E_t": E_t,
            "E_v": E_v,
            "H_T": hashlib.sha256(T.encode('utf-8')).hexdigest(),
            "C_prime_encrypted": json.dumps(C_prime_encrypted)
        }
        
        print("PAD data generated (D_ts) :", D_ts)
        
        # Suppose the RIO receives these data, verifies the proof (already done),
        # then proceeds with the online deposit.
        # Here, we simulate the reception and approval by the RIO.
        # For the simulation, we consider that the transaction is validated by the RIO.
        # Measure the RIO's transaction verification time
        verification_start_time = time.time()
        # (Real verification operations would be performed here)
        # We use the real time between reception and end of verification
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time (RIO): {transaction_verification_time:.6f} seconds")
        
        # Update balances and remove Alice's coin
        self.alice_wallet.coins.pop(0)
        self.alice_wallet.balance -= amount
        print(f"Online deposit (via PAD) of {amount} units successfully completed for {self.alice_wallet.owner_id}.")
        
        # Calculate the total latency and CPU load
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
        
        # Record metrics for this PAD scenario
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,  # deposit on her own account
            from_agent=self.alice_wallet.owner_id,
            settlement_type="online_deposit_PAD",
            amount=amount,
            description="Online Deposit via PAD",
            latency=latency,
            zk_proof_gen_time=zk_proof_gen_time,
            zk_proof_ver_time=zk_proof_ver_time,
            transaction_ver_time=transaction_verification_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Multiple methods for each type of transaction ---
    def simulate_multiple_offline_transactions(self, nb_transactions, offline_amount=None):
        nb_possible = len(self.alice_wallet.coins)
        nb_to_execute = min(nb_transactions, nb_possible)
        print(f"Number of offline transactions requested: {nb_transactions}.")
        print(f"Number of coins available in the wallet: {nb_possible}.")
        print(f"Executing {nb_to_execute} offline transaction(s).")
        for i in range(nb_to_execute):
            self.simulate_offline_transaction(amount=offline_amount)

    def simulate_multiple_withdrawals(self, nb_withdrawals, withdrawal_amount):
        print(f"Number of withdrawals requested: {nb_withdrawals}.")
        for i in range(nb_withdrawals):
            self.simulate_withdrawal(withdrawal_amount)

    def simulate_multiple_deposits(self, nb_deposits):
        nb_possible = len(self.bob_wallet.coins)
        nb_to_execute = min(nb_deposits, nb_possible)
        print(f"Number of deposits requested: {nb_deposits}.")
        print(f"Number of coins available in Bob's wallet: {nb_possible}.")
        for i in range(nb_to_execute):
            self.simulate_deposit()

# --- Digital identity management ---
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode('utf-8')).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode('utf-8')).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode('utf-8')).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        digital_identity = {
            "user_id": user_id,
            "credential": credential,
            "pid": pid
        }
        return digital_identity

    def verify_identity(self, digital_identity):
        cred = digital_identity["credential"].copy()
        signature = cred.pop("signature", None)
        cred_str = json.dumps(cred, sort_keys=True)
        expected_signature = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return expected_signature == signature

# --- For measuring digital onboarding performance ---
class IdentityMetrics:
    def __init__(self):
        self.onboarding_times = []
        self.verification_times = []

    def record_onboarding_time(self, t):
        self.onboarding_times.append(t)

    def record_verification_time(self, t):
        self.verification_times.append(t)

    def average_onboarding_time(self):
        if not self.onboarding_times:
            return 0
        return sum(self.onboarding_times) / len(self.onboarding_times)
    
    def average_verification_time(self):
        if not self.verification_times:
            return 0
        return sum(self.verification_times) / len(self.verification_times)

# Creating instances for identity management
id_manager = DigitalIdentityManager()
id_metrics = IdentityMetrics()

# --- Multiple digital onboarding phase with performance measurement ---
print("\n=== Digital Onboarding Phase for multiple users ===")
try:
    nb_users = int(input("Entrez le nombre d'utilisateurs à onboarder : "))
except ValueError:
    print("Invalid number, onboarding 1 by default.")
    nb_users = 1

digital_identities = []
for i in range(1, nb_users + 1):
    user_id = f"User_{i}"
    user_attributes = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
    start_onboarding = time.time()
    digital_identity = id_manager.onboard_user(user_id, user_attributes)
    end_onboarding = time.time()
    onboarding_time = end_onboarding - start_onboarding
    id_metrics.record_onboarding_time(onboarding_time)
    
    start_verif = time.time()
    verification_ok = id_manager.verify_identity(digital_identity)
    end_verif = time.time()
    verification_time = end_verif - start_verif
    id_metrics.record_verification_time(verification_time)
    
    digital_identities.append(digital_identity)
    print(f"Digital Identity for {user_id} :", digital_identity)
    print(f"Onboarding Time for {user_id} : {onboarding_time:.6f} seconds")
    if verification_ok:
        print(f"Identity verification successful for {user_id} in {verification_time:.6f} seconds.")
        #print(f"Identity verification successful for {user_id} in {verification_time:.6f} seconds.")

    else:
        print(f"Identity verification failed for {user_id} in {verification_time:.6f} seconds.")

print("\n=== Digital Identity Metrics ===")
print("Average Time of digital onboarding :", id_metrics.average_onboarding_time(), "seconds")
print("Average Time of identity verification :", id_metrics.average_verification_time(), "seconds")

# Export metrics to CSV
import csv
with open("identity_metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["User_ID", "Onboarding_Time", "Verification_Time"])
    for i, identity in enumerate(digital_identities, 1):
        writer.writerow([f"User_{i}", id_metrics.onboarding_times[i-1], id_metrics.verification_times[i-1]])

        



    # (Additional phases, such as deposits, can be handled similarly.)
        
if __name__ == "__main__":
    simulation = OfflinePaymentSimulation()
    
    # --- Number of Withdrawals ---
    try:
        nb_withdrawals = int(input("Enter the number of withdrawals to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 withdrawal.")
        nb_withdrawals = 1
    try:
        withdrawal_amount = int(input("Enter the amount for each withdrawal (in units): "))
    except ValueError:
        print("Invalid amount, defaulting to 50 units.")
        withdrawal_amount = 50

    simulation.simulate_multiple_withdrawals(nb_withdrawals, withdrawal_amount)
    
     #--- Offline Transactions ---
    try:
        nb_offline = int(input("Enter the number of offline transactions to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 offline transaction.")
        nb_offline = 1
    try:
        offline_amount = int(input("Enter the amount for each offline transaction (in units): "))
    except ValueError:
        print("Invalid amount, using a random amount.")
        offline_amount = None

    simulation.simulate_multiple_offline_transactions(nb_offline, offline_amount)
    
    # --- Deposits ---
    try:
        nb_deposits = int(input("Enter the number of deposits to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 deposit.")
        nb_deposits = 1
    simulation.simulate_multiple_deposits(nb_deposits)
    
    # --- Bob Pays Carla Transactions ---
    try:
        nb_bob_to_carla = int(input("Enter the number of transactions between Bob and Carla to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 transaction.")
        nb_bob_to_carla = 1
    try:
        bob_to_carla_amount = int(input("Enter the amount for each Bob-to-Carla transaction (in units): "))
    except ValueError:
        print("Invalid amount, defaulting to 50 units.")
        bob_to_carla_amount = 50

    for i in range(nb_bob_to_carla):
        #simulation.simulate_bob_pays_carla(bob_to_carla_amount)
    
    # Generate and export the performance metrics CSV
       simulation.environment.generate_metrics()

print("\n=== Digital identity metrics ===")
print("Average time of digital onboarding :", id_metrics.average_onboarding_time(), "seconds")
print("Average time of identity verification :", id_metrics.average_verification_time(), "seconds")


In [1]:
# !pip install pycryptodome
# !pip install pyipv8
# cbdc_simulator.py
import time
import random
import hashlib
import rsa
import pandas as pd
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import json
import base64
import os


# --- Handling of pyipv8 and definition of a concrete class SimplePayload ---
try:
    from ipv8.messaging.payload import Payload
except ImportError:
    print("pyipv8 not found, using a simulated Payload object.")
    class Payload:
        def __init__(self):
            self.data = None
        def __str__(self):
            return str(self.data)
    SimplePayload = Payload
else:
    class SimplePayload(Payload):
        def __init__(self):
            super().__init__()
            self.data = None
        def to_pack_list(self):
            return [self.data]
        @classmethod
        def from_unpack_list(cls, lst):
            instance = cls()
            instance.data = lst[0]
            return instance

def xor_bytes(a, b):
    """Returns the byte-wise XOR of two byte sequences of the same length."""
    return bytes(x ^ y for x, y in zip(a, b))

# --- PyIPv8Transport class using SimplePayload ---
class PyIPv8Transport:
    @staticmethod
    def encapsulate(packet, sender="Alice", receiver="Bob"):
        header = {
            "protocol": "ipv8",
            "timestamp": time.time(),
            "sender": sender,
            "receiver": receiver
        }
        payload = SimplePayload()
        payload.data = {"header": header, "data": packet}
        return payload

    @staticmethod
    def send(packet, sender="Alice", receiver="Bob"):
        payload = PyIPv8Transport.encapsulate(packet, sender, receiver)
        print("PyIPv8Transport: Sending payload:")
        print(payload)
        return payload

    @staticmethod
    def receive(payload):
        print("PyIPv8Transport: Received payload:")
        print(payload)
        data = payload.data.get("data")
        return data

# --- Other unchanged classes (NFCMock, ZKSNARK, CentralBank, OfflineWallet, Environment) ---
class NFCMock:
    @staticmethod
    def send(transaction_data):
        print("NFCMock: Transaction data sent.")
        return json.dumps(transaction_data)

    @staticmethod
    def receive(transaction_json):
        print("NFCMock: Transaction data received.")
        return json.loads(transaction_json)

class ZKSNARK:
    @staticmethod
    def generate_key():
        return "simulated_zk_key"

    @staticmethod
    def generate_proof(private_key, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()  
        return hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()

    @staticmethod
    def verify_proof(proof, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()  
        expected_proof = hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()
        return proof == expected_proof

class CentralBank:
    def __init__(self, name):
        self.name = name
        self.coins_issued = []
        self.key_pair = rsa.newkeys(2048)
        self.deposited_coins = set()
        self.used_nonces = set()
        self.logged_transactions = []
      
    def issue_coin(self, recipient, value):
        coin_id = hashlib.sha256(str(random.getrandbits(256)).encode()).hexdigest()
        signature = self.sign_coin(value, coin_id)
        signature_b64 = base64.b64encode(signature).decode('utf-8')
        coin = {
            "coin_id": coin_id,
            "value": value,
            "signature": signature_b64
        }
        self.coins_issued.append(coin)
        return coin

    def sign_coin(self, value, coin_id):
        message = f"{value}{coin_id}".encode()
        return rsa.sign(message, self.key_pair[1], 'SHA-256')

    def sign_blinded_hash(self, blinded_hash):
        return hashlib.sha256(f"{blinded_hash}{self.name}".encode()).hexdigest()

    def verify_and_deposit_coin(self, coin, sender_public_key, proof, zk_key, nonce):
        coin_hash = hashlib.sha256((str(coin['value']) + coin['coin_id'] + coin['signature']).encode()).hexdigest()
        if coin_hash in self.deposited_coins:
            return "Coin Already Defunded or Verification Failed"
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, self.key_pair[0])
        except rsa.VerificationError:
            return "Coin Already Defunded or Verification Failed"
        if nonce in self.used_nonces:
            return "Nonce already used. Double spending detected."
        self.used_nonces.add(nonce)
        amount = coin['value']
        expected_proof = hashlib.sha256(f"{sender_public_key}{amount}{zk_key}".encode()).hexdigest()
        if proof != expected_proof:
            return "Coin Already Defunded or Verification Failed"
        transaction_record = (sender_public_key.save_pkcs1(), nonce, proof)
        if transaction_record in self.logged_transactions:
           return "Double spending detected: Transaction already recorded."
        self.logged_transactions.append(transaction_record)
        self.deposited_coins.add(coin_hash)
        return "Coin Deposited Successfully"

class OfflineWallet:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.balance = 100
        self.key_pair = self.generate_key_pair()
        self.zk_key = ZKSNARK.generate_key()
        self.nonce = 0
        self.withdrawal_latencies = []
        self.deposit_latencies = []
        self.coins = []
        self.received_logs = []

    def log_transaction(self, sender_key, nonce, proof):
        self.received_logs.append((sender_key.save_pkcs1(), nonce, proof))

    def has_received_transaction(self, sender_key, nonce, proof):
        return (sender_key.save_pkcs1(), nonce, proof) in self.received_logs

    def generate_key_pair(self):
        return rsa.newkeys(2048)

    def store_coin(self, coin):
        self.coins.append(coin)
        print(f"Coin stored: {coin}")

    def prepare_transaction(self, recipient, amount):
        self.nonce += 1
        if not self.coins:
            raise ValueError("No coins available to send.")
        coin = self.coins[0]
        proof = ZKSNARK.generate_proof(self.key_pair[1], recipient.key_pair[0], amount, self.zk_key)
        return {
            "amount": amount,
            "coin": coin.copy(),
            "recipient_key": base64.b64encode(recipient.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "from_agent_key": base64.b64encode(self.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.nonce,
            "timestamp": time.time()
        }

    def blind_coin(self):
        f = hashlib.sha256(self.key_pair[0].save_pkcs1()).hexdigest()
        blinding_factor = random.randint(1, 10**5)
        blinded_hash = (int(f, 16) * blinding_factor) % (2**256)
        return {"hash": f, "blinded_hash": blinded_hash, "blinding_factor": blinding_factor}

class Environment:
    def __init__(self):
        self.measurement = {
            "transaction_id": [],
            "period": [],
            "to_agent": [],
            "from_agent": [],
            "settlement_type": [],
            "amount": [],
            "description": [],
            "latency": [],
            "zk_proof_generation_time": [],
            "zk_proof_verification_time": [],
            "transaction_verification_time": [],
            "load_computing": [],
            "status": []
        }

    def log_transaction(self, transaction_id, period, to_agent, from_agent, settlement_type, amount, description,
                        latency=None, zk_proof_gen_time=None, zk_proof_ver_time=None, transaction_ver_time=None,
                        load_computing=None, status="Pending"):
        self.measurement["transaction_id"].append(transaction_id)
        self.measurement["period"].append(period)
        self.measurement["to_agent"].append(to_agent)
        self.measurement["from_agent"].append(from_agent)
        self.measurement["settlement_type"].append(settlement_type)
        self.measurement["amount"].append(amount)
        self.measurement["description"].append(description)
        self.measurement["latency"].append(latency)
        self.measurement["zk_proof_generation_time"].append(zk_proof_gen_time)
        self.measurement["zk_proof_verification_time"].append(zk_proof_ver_time)
        self.measurement["transaction_verification_time"].append(transaction_ver_time)
        self.measurement["load_computing"].append(load_computing)
        self.measurement["status"].append(status)

    def generate_metrics(self):
        df = pd.DataFrame(self.measurement)
        total_transactions = len(df)
        total_latency = df["latency"].sum()
        overall_throughput = total_transactions / total_latency if total_latency > 0 else 0
      
        phase_throughput = {}
        for phase in df['settlement_type'].unique():
            phase_df = df[df['settlement_type'] == phase]
            phase_count = len(phase_df)
            phase_total_latency = phase_df["latency"].sum()
            phase_throughput[phase] = phase_count / phase_total_latency if phase_total_latency > 0 else 0
        df["throughput"] = df["settlement_type"].apply(lambda phase: phase_throughput.get(phase, 0))
        
        for col in ["zk_proof_generation_time", "zk_proof_verification_time", "load_computing", "throughput"]:
            df[col] = df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) and isinstance(x, (float, int)) else x)
        
        print("\n--- Simulation Metrics ---")
        print(f"Total Transactions: {total_transactions}")
        print(f"Total Latency (seconds): {total_latency:.6f}")
        print(f"Overall Throughput (transactions/sec): {overall_throughput:.6f}")
        
        print("\nThroughput per phase:")
        for phase, tp in phase_throughput.items():
            print(f"  {phase}: {tp:.6f} transactions/sec")
        df.to_csv("simulation_metrics_test_paper2.csv", index=False)

# --- Integration of the scenario "Bob pays Carla" ---
class OfflinePaymentSimulation:
    def __init__(self):
        self.environment = Environment()
        self.alice_wallet = OfflineWallet("Alice")
        self.bob_wallet = OfflineWallet("Bob")
        self.carla_wallet = OfflineWallet("Carla")  # New wallet for Carla
        self.central_bank = CentralBank("Central Bank")
        self.used_nonces = set()

    def validate_coin_signature(self, coin, public_key):
        if not coin or "signature" not in coin:
            print("Invalid coin: Missing or no signature.")
            return False
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, public_key)
            print("Coin signature is valid.")
            return True
        except rsa.VerificationError:
            print("Invalid coin: Signature verification failed.")
            return False

    def verify_sender_legitimacy(self, sender_public_key, known_keys):
        return any(sender_public_key.save_pkcs1() == key.save_pkcs1() for key in known_keys)
     
    def verify_nonce(self, nonce, used_nonces):
        if nonce in used_nonces:
            print(f"Nonce {nonce} has already been used. Double spending detected.")
            return False
        used_nonces.add(nonce)
        return True

    def verify_recipient(self, transaction_data, expected_recipient_key):
        recipient_key = transaction_data.get("recipient_key")
        if not isinstance(recipient_key, rsa.PublicKey):
            print("Failure: recipient_key is not an rsa.PublicKey object.")
            return False
        return recipient_key.save_pkcs1() == expected_recipient_key.save_pkcs1()


    def simulate_deposit(self):
        """Simulates the deposit of a coin to the central bank."""
        if not self.bob_wallet.coins:
            print("Bob has no coin to deposit.")
            return

        coin = self.bob_wallet.coins[0]  # Select the first coin for deposit
        amount = coin['value']

        computing_start = time.process_time()
        start_time = time.time()

        # Generate ZK proof for deposit
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1], self.central_bank.key_pair[0], amount, self.bob_wallet.zk_key)

        # Prepare deposit data
        deposit_data = {
            "coin": coin.copy(),
            "sender_public_key": base64.b64encode(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.bob_wallet.nonce + 1,  # Increment nonce for the deposit
            "timestamp": time.time()
        }

        # Simulate the deposit process
        deposit_result = self.central_bank.verify_and_deposit_coin(
            coin,
            self.bob_wallet.key_pair[0],
            proof,
            self.bob_wallet.zk_key,
            deposit_data["nonce"]  # Use the incremented nonce for deposit
        )

        if deposit_result == "Coin Deposited Successfully":
            self.bob_wallet.coins.pop(0)  # Remove deposited coin from wallet
            self.bob_wallet.balance -= amount  # Update wallet balance
            self.bob_wallet.nonce += 1  # Increment wallet's nonce after successful deposit
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            print(f"Deposit completed successfully. Total time: {latency:.6f}s, "
                  f"Computing load (CPU): {computing_load:.6f}s.")

            self.environment.log_transaction(
                transaction_id=len(self.environment.measurement["transaction_id"]),
                period=time.time(),
                to_agent="Central Bank",
                from_agent=self.bob_wallet.owner_id,
                settlement_type="deposit",
                amount=amount,
                description="Coin Deposit",
                latency=latency,
                zk_proof_gen_time=None,  # Add timing if needed
                zk_proof_ver_time=None,  # Add timing if needed
                transaction_ver_time=None,  # Add timing if needed
                load_computing=computing_load,
                status="Success"
            )
        else:
            print(f"Deposit failed: {deposit_result}")

    # --- Withdrawal method ---
    def simulate_withdrawal(self, amount):
        print(f"Alice withdraws {amount} units...")
        computing_start = time.process_time()
        total_start_time = time.time()
        blinded_coin = self.alice_wallet.blind_coin()
        interaction_start_time = time.process_time()
        blinded_signature = self.central_bank.sign_blinded_hash(blinded_coin["blinded_hash"])
        interaction_cpu_time = time.process_time() - interaction_start_time
        message = f"{amount}{self.alice_wallet.owner_id}".encode()
        unblinded_signature = rsa.sign(message, self.central_bank.key_pair[1], 'SHA-256')
        coin = self.central_bank.issue_coin(self.alice_wallet.owner_id, amount)
        self.alice_wallet.store_coin(coin)
        total_latency = time.time() - total_start_time
        computing_load = time.process_time() - computing_start
        print(f"Withdrawal completed successfully. Total time: {total_latency:.6f}s, "
              f"Interaction time (CPU): {interaction_cpu_time:.6f}s, "
              f"Computing load (CPU): {computing_load:.6f}s.")
        self.environment.log_transaction(
            transaction_id=len(self.environment.measurement["transaction_id"]),
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Central Bank",
            settlement_type="withdrawal",
            amount=amount,
            description="Withdrawal with blinding",
            latency=total_latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=interaction_cpu_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Offline payment method (Alice pays Bob) ---
    def simulate_offline_transaction(self, amount=None):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        if amount is None:
            amount = random.randint(10, 50)
        if not self.alice_wallet.coins:
            print("No coin available for an offline transaction.")
            return
        coin = self.alice_wallet.coins[0]
        if amount > coin["value"]:
            print(f"Requested amount ({amount}) greater than coin value ({coin['value']}). Using the coin's value.")
            amount = coin["value"]
        elif amount < coin["value"]:
            print("Partial transaction not supported. Using the full coin value.")
            amount = coin["value"]
        computing_start = time.process_time()
        start_time = time.time()
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.bob_wallet.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start_time
        transaction_data = self.alice_wallet.prepare_transaction(self.bob_wallet, amount)
        transaction_data.update({
            "transaction_id": transaction_id,
            "from_agent": self.alice_wallet.owner_id,
            "to_agent": self.bob_wallet.owner_id,
            "nonce": transaction_data["nonce"]
        })
        json_data = json.dumps(transaction_data).encode('utf-8')
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_data)
        packet = {
            "encrypted_aes_key": base64.b64encode(
                PKCS1_OAEP.new(
                    RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM'))
                ).encrypt(aes_key)
            ).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Sending the encapsulated transaction via pyipv8...")
        ipv8_payload = PyIPv8Transport.send(packet, sender=self.alice_wallet.owner_id, receiver=self.bob_wallet.owner_id)
        received_packet = PyIPv8Transport.receive(ipv8_payload)
        try:
            enc_aes_key = base64.b64decode(received_packet["encrypted_aes_key"])
            ciphertext = base64.b64decode(received_packet["ciphertext"])
            tag = base64.b64decode(received_packet["tag"])
            nonce = base64.b64decode(received_packet["nonce"])
            bob_private_pem = self.bob_wallet.key_pair[1].save_pkcs1(format='PEM')
            bob_private_key_py = RSA.importKey(bob_private_pem)
            cipher_rsa_bob = PKCS1_OAEP.new(bob_private_key_py)
            decrypted_aes_key = cipher_rsa_bob.decrypt(enc_aes_key)
            cipher_aes_bob = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=nonce)
            decrypted_data = cipher_aes_bob.decrypt_and_verify(ciphertext, tag)
            received_transaction = json.loads(decrypted_data.decode('utf-8'))
            if isinstance(received_transaction["from_agent_key"], str):
                try:
                    sender_public_key = rsa.PublicKey.load_pkcs1(base64.b64decode(received_transaction["from_agent_key"]))
                    received_transaction["from_agent_key"] = sender_public_key
                    print("Sender public key correctly loaded.")
                except Exception as e:
                    print(f"Error while decoding the sender's key: {e}")
                    return
            if isinstance(received_transaction["recipient_key"], str):
                try:
                    received_transaction["recipient_key"] = rsa.PublicKey.load_pkcs1(
                        base64.b64decode(received_transaction["recipient_key"])
                    )
                    print("recipient_key correctly loaded as PublicKey.")
                except Exception as e:
                    print(f"Error while decoding recipient_key: {e}")
                    return
            if not isinstance(received_transaction["recipient_key"], rsa.PublicKey):
                print("Error: recipient_key is not a valid rsa.PublicKey object after processing.")
                return
            print(f"Received recipient_key: {received_transaction['recipient_key']}")
            print(f"Type of recipient_key: {type(received_transaction['recipient_key'])}")
            verification_start_time = time.time()
            if not self.validate_coin_signature(received_transaction.get("coin"), self.central_bank.key_pair[0]):
                print("Failed to verify the coin signature.")
                return
            sender_public_key = received_transaction.get("from_agent_key")
            if not self.verify_sender_legitimacy(sender_public_key, [self.alice_wallet.key_pair[0]]):
                print("Failed to verify the sender’s legitimacy.")
                return
            else:
                print("Sender legitimacy verification successful: The sender is legitimate.")
            proof = received_transaction.get("proof")
            nonce = received_transaction.get("nonce")
            if self.bob_wallet.has_received_transaction(sender_public_key, nonce, proof):
                print("Double spending detected: transaction already received.")
                return
            else:
                self.bob_wallet.log_transaction(sender_public_key, nonce, proof)
                print("Transaction successfully recorded in the logs.")
            if not self.verify_nonce(received_transaction.get("nonce"), self.used_nonces):
                print("Failed to verify the nonce.")
                return
            zk_proof_ver_start_time = time.time()
            proof_valid = ZKSNARK.verify_proof(
                proof,
                self.bob_wallet.key_pair[0],
                amount,
                self.alice_wallet.zk_key
            )
            zk_proof_ver_time = time.time() - zk_proof_ver_start_time
            if not proof_valid:
                print("zk-SNARK proof verification failed.")
                return
            else:
                print("zk-SNARK proof verification successful: The proof is valid.")
            print("Nonce is verified and unique, and the zk-SNARK proof is verified and unique.")
            verification_time = time.time() - verification_start_time
            self.alice_wallet.coins.pop(0)
            received_transaction["coin"]["nonce"] = received_transaction["nonce"]
            received_transaction["coin"]["recipient"] = self.bob_wallet.owner_id
            self.bob_wallet.store_coin(received_transaction["coin"])
            self.alice_wallet.balance -= amount
            self.bob_wallet.balance += amount
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            self.environment.log_transaction(
                transaction_id=transaction_id,
                period=time.time(),
                to_agent=self.bob_wallet.owner_id,
                from_agent=self.alice_wallet.owner_id,
                settlement_type="offline_transaction",
                amount=amount,
                description="Offline Transaction",
                latency=latency,
                zk_proof_gen_time=zk_proof_gen_time,
                zk_proof_ver_time=zk_proof_ver_time,
                transaction_ver_time=verification_time,
                load_computing=computing_load,
                status="Success"
            )
        except Exception as e:
            print(f"Error during the transaction: {e}")

    # --- New method: scenario of Bob paying Carla ---
    def simulate_bob_pays_carla(self, amount):
        # Define the transaction ID as the current number of transactions + 1
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        # Start measuring time (wall time and CPU time)
        computing_start = time.process_time()
        start_time = time.time()
    
        print(f"\n=== Scenario: Bob pays Carla for {amount} units ===")
        # Step 1: Bob and Carla agree on a contract
        contract = f"Contract_Bob_pays_Carla_{amount}"
        print("Bob and Carla agree on the contract:", contract)
        
        # Step 2: Bob selects the RIO attestation (dummy value)
        rio_cert = "RIO_pk_cert_dummy"
        print("Bob selects the RIO attestation:", rio_cert)
        
        # Step 3: Carla provides Bob with one of her pseudonymous identities
        carla_pseudonym = self.carla_wallet.key_pair[0].save_pkcs1().decode('utf-8')
        print("Carla provides Bob with her pseudonymous identifier.")
        
        # Step 4: Bob selects an e-coin corresponding to the desired amount
        if not self.bob_wallet.coins:
            print("Bob has no coin available for the payment.")
            return
        ecoin = self.bob_wallet.coins[0]
        print("Selected e-Coin:", ecoin)
        
        # Step 5: Bob signs the contract with his e-coin
        signature_contract = rsa.sign(contract.encode('utf-8'), self.bob_wallet.key_pair[1], 'SHA-256')
        signature_contract_b64 = base64.b64encode(signature_contract).decode('utf-8')
        print("Bob signs the contract:", signature_contract_b64)
        
        # Step 6: Bob generates the zk-SNARK proof of payment
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1],
                                       self.carla_wallet.key_pair[0],
                                       amount,
                                       self.bob_wallet.zk_key)
        print("Generated proof:", proof)
        # Measure the proof verification time
        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(
                    proof,
                    self.carla_wallet.key_pair[0],
                    amount,
                    self.bob_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start
        if not proof_valid:
            print("zk-SNARK proof verification failed.")
            return
        else:
            print("zk-SNARK proof verification successful: The proof is valid.")

        # Step 7: Bob constructs the transaction packet including the following data
        transaction_payload = {
            "contract": contract,
            "contract_signature": signature_contract_b64,
            "proof": proof,
            "RIO_cert": rio_cert,
            "amount": amount,
            "bob_pseudonym": self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8'),
            "carla_pseudonym": carla_pseudonym,
            "timestamp": time.time()
        }
        json_payload = json.dumps(transaction_payload).encode('utf-8')
        
        # Step 8: Encryption of the transaction packet (RIO simulator)
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_payload)
    
        # Generation of a key pair for RIO
        rio_key_pair = rsa.newkeys(2048)
        rio_public = rio_key_pair[0]
        rio_private = rio_key_pair[1]
    
        # Encryption of the AES key with RIO's public key
        rio_public_pem = rio_public.save_pkcs1(format='PEM')
        cipher_rsa_enc = PKCS1_OAEP.new(RSA.importKey(rio_public_pem))
        encrypted_aes_key = cipher_rsa_enc.encrypt(aes_key)
    
        encrypted_packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Bob-to-Carla transaction (encrypted for RIO):", encrypted_packet)
    
        # --- Simulation of the verification phase by the RIO ---
        # To decrypt, the RIO uses its private key
        rio_private_pem = rio_private.save_pkcs1(format='PEM')
        cipher_rsa_dec = PKCS1_OAEP.new(RSA.importKey(rio_private_pem))
        # Decryption of the AES key
        decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)
    
        # Use the decrypted key to decrypt the payload
        cipher_aes_rio = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=cipher_aes.nonce)
        decrypted_payload = cipher_aes_rio.decrypt_and_verify(ciphertext, tag)
        # Here, the RIO verifies the payload data (contract, signature, proof, etc.)
        # Initialize verification_start_time here within simulate_bob_pays_carla
        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time: {transaction_verification_time:.6f} seconds")

    # ----------------------------
    # TOTAL LATENCY AND CPU LOAD CALCULATION"
    
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
    # ----------------------------   
    # RECORDING METRICS FOR THE SCENARIO "BOB PAYS CARLA"
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.carla_wallet.owner_id,
            from_agent=self.bob_wallet.owner_id,
            settlement_type="offline transaction 2 (bob_to_carla)",  # or "bob_to_carla"
            amount=amount,
            description="Payment from Bob to Carla",
            latency=latency,
            zk_proof_gen_time=None,  # You can add proof generation time if needed
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=computing_load,
            status="Success"
        )
    # ----------------------------
    def simulate_online_deposit_PAD(self, amount):
        """
        Online Deposit/Transfer scenario via PAD.
        This scenario simulates the online deposit (or transfer to one's own account)
        through the PAD system managed by the RIO.
        """
        print(f"\n=== Online Deposit scenario via PAD for {amount} units ===")
        # Define time variables and the transaction identifier
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()
        
        # -- Step 0: Transaction Verification (similar to the verification of offline transactions) --
        # Here, the RIO verifies that the transaction is not a double spending.
        # For the simulation, we use the existing zk-SNARK proof verification.
        # Suppose we use a coin deposited by Alice for the online deposit.
        if not self.alice_wallet.coins:
            print("No coin available for an online deposit.")
            return
        coin = self.alice_wallet.coins[0]
        # Proof generation and verification (similar to the other scenario)
        zk_proof_start = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.central_bank.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start

        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(proof,
                                           self.central_bank.key_pair[0],
                                           amount,
                                           self.alice_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start

        if not proof_valid:
            print("zk-SNARK proof verification failed during the online transaction.")
            return
        print("Successful verification of zk-SNARK proof.")

        # -- Step 1 : Securing the blinded coin F_s --
        # We assume that Alice's e-coin is used as the blinded coin F_s.
        Fs = self.alice_wallet.blind_coin()  # F_s
        # Generate a token T (for the simulation, a random number as a string)
        T = str(random.randint(1000, 9999))
        print("Generated Token T :", T)
        # Generate a symmetric key K_sym
        K_sym = get_random_bytes(32)
        # Encrypt (T, F_s) with K_sym (we concatenate T and part of Fs)
        data_to_encrypt = T.encode('utf-8') + json.dumps(Fs, sort_keys=True).encode('utf-8')
        cipher_pad = AES.new(K_sym, AES.MODE_GCM)
        C, tag = cipher_pad.encrypt_and_digest(data_to_encrypt)
        nonce_pad = cipher_pad.nonce
        # C represents the encrypted content containing the token and F_s
        
        # Now, sign and encrypt the content with Alice's private key and then with the recipient's public key
        # (here, for the online deposit, we assume that the transaction remains on Alice's same account)
        # For the simulation, we will sign C with Alice's private key:
        signature_C = rsa.sign(C, self.alice_wallet.key_pair[1], 'SHA-256')
        signature_C_b64 = base64.b64encode(signature_C).decode('utf-8')
        # We form a packet C' that combines the content C and its signature
        C_prime = {
            "encrypted_content": base64.b64encode(C).decode('utf-8'),
            "signature": signature_C_b64
        }
        # Then, encrypt C' with the recipient's public key.
        # In the case of a deposit on the same account, we can encrypt with Alice's public key.
        # --- Hybrid encryption of C_prime ---
        # Convert C_prime to JSON (data to be encrypted)
        C_prime_json = json.dumps(C_prime).encode('utf-8')

        # 1. Generate a symmetric key K_PAD to encrypt C_prime_json with AES
        K_PAD = get_random_bytes(32)
        cipher_pad2 = AES.new(K_PAD, AES.MODE_GCM)
        C_prime_ciphertext, C_prime_tag = cipher_pad2.encrypt_and_digest(C_prime_json)
        nonce_pad2 = cipher_pad2.nonce

        # 2. Encrypt the symmetric key K_PAD with Alice's public key (in this case, the online deposit is on her own account)
        cipher_rsa_for_PAD = PKCS1_OAEP.new(RSA.importKey(self.alice_wallet.key_pair[0].save_pkcs1(format='PEM')))
        encrypted_K_PAD = cipher_rsa_for_PAD.encrypt(K_PAD)

        # 3. Form the final packet for C_prime by grouping the encrypted key and the ciphertext
        C_prime_encrypted = {
            "encrypted_key": base64.b64encode(encrypted_K_PAD).decode('utf-8'),
            "ciphertext": base64.b64encode(C_prime_ciphertext).decode('utf-8'),
            "tag": base64.b64encode(C_prime_tag).decode('utf-8'),
            "nonce": base64.b64encode(nonce_pad2).decode('utf-8')
        }

        
        # Masking the symmetric key with a mask R_sym
        R_sym = get_random_bytes(32)
        S_m = xor_bytes(K_sym, R_sym)
        # Generate a dummy share for trustees (N_t, for example 3)
        N_t = b"3"
        S_t = hashlib.sha256(S_m + N_t).hexdigest()
        # Generate a dummy share for validators (N_v, for example 2)
        N_v = b"2"
        V = hashlib.sha256(b"R" + self.alice_wallet.key_pair[0].save_pkcs1() + N_v).hexdigest()
        
        # Encrypt dummy shares for trustees and validators (simulated here)
        E_t = "encrypted_trustee_shares_dummy"
        E_v = "encrypted_validator_shares_dummy"
        
        # Final data transmission structure D_ts
        D_ts = {
            "E_t": E_t,
            "E_v": E_v,
            "H_T": hashlib.sha256(T.encode('utf-8')).hexdigest(),
            "C_prime_encrypted": json.dumps(C_prime_encrypted)
        }
        
        print("PAD data generated (D_ts) :", D_ts)
        
        # Suppose the RIO receives these data, verifies the proof (already done),
        # then proceeds with the online deposit.
        # Here, we simulate the reception and approval by the RIO.
        # For the simulation, we consider that the transaction is validated by the RIO.
        # Measure the RIO's transaction verification time
        verification_start_time = time.time()
        # (Real verification operations would be performed here)
        # We use the real time between reception and end of verification
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time (RIO): {transaction_verification_time:.6f} seconds")
        
        # Update balances and remove Alice's coin
        self.alice_wallet.coins.pop(0)
        self.alice_wallet.balance -= amount
        print(f"Online deposit (via PAD) of {amount} units successfully completed for {self.alice_wallet.owner_id}.")
        
        # Calculate the total latency and CPU load
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
        
        # Record metrics for this PAD scenario
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,  # deposit on her own account
            from_agent=self.alice_wallet.owner_id,
            settlement_type="online_deposit_PAD",
            amount=amount,
            description="Online Deposit via PAD",
            latency=latency,
            zk_proof_gen_time=zk_proof_gen_time,
            zk_proof_ver_time=zk_proof_ver_time,
            transaction_ver_time=transaction_verification_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Multiple methods for each type of transaction ---
    def simulate_multiple_offline_transactions(self, nb_transactions, offline_amount=None):
        nb_possible = len(self.alice_wallet.coins)
        nb_to_execute = min(nb_transactions, nb_possible)
        print(f"Number of offline transactions requested: {nb_transactions}.")
        print(f"Number of coins available in the wallet: {nb_possible}.")
        print(f"Executing {nb_to_execute} offline transaction(s).")
        for i in range(nb_to_execute):
            self.simulate_offline_transaction(amount=offline_amount)

    def simulate_multiple_withdrawals(self, nb_withdrawals, withdrawal_amount):
        print(f"Number of withdrawals requested: {nb_withdrawals}.")
        for i in range(nb_withdrawals):
            self.simulate_withdrawal(withdrawal_amount)

    def simulate_multiple_deposits(self, nb_deposits):
        nb_possible = len(self.bob_wallet.coins)
        nb_to_execute = min(nb_deposits, nb_possible)
        print(f"Number of deposits requested: {nb_deposits}.")
        print(f"Number of coins available in Bob's wallet: {nb_possible}.")
        for i in range(nb_to_execute):
            self.simulate_deposit()

# --- Digital identity management ---
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode('utf-8')).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode('utf-8')).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode('utf-8')).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        digital_identity = {
            "user_id": user_id,
            "credential": credential,
            "pid": pid
        }
        return digital_identity

    def verify_identity(self, digital_identity):
        cred = digital_identity["credential"].copy()
        signature = cred.pop("signature", None)
        cred_str = json.dumps(cred, sort_keys=True)
        expected_signature = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return expected_signature == signature

# --- For measuring digital onboarding performance ---
class IdentityMetrics:
    def __init__(self):
        self.onboarding_times = []
        self.verification_times = []

    def record_onboarding_time(self, t):
        self.onboarding_times.append(t)

    def record_verification_time(self, t):
        self.verification_times.append(t)

    def average_onboarding_time(self):
        if not self.onboarding_times:
            return 0
        return sum(self.onboarding_times) / len(self.onboarding_times)
    
    def average_verification_time(self):
        if not self.verification_times:
            return 0
        return sum(self.verification_times) / len(self.verification_times)

# Creating instances for identity management
id_manager = DigitalIdentityManager()
id_metrics = IdentityMetrics()

# --- Multiple digital onboarding phase with performance measurement ---
print("\n=== Digital Onboarding Phase for multiple users ===")
try:
    nb_users = int(input("Entrez le nombre d'utilisateurs à onboarder : "))
except ValueError:
    print("Invalid number, onboarding 1 by default.")
    nb_users = 1

digital_identities = []
for i in range(1, nb_users + 1):
    user_id = f"User_{i}"
    user_attributes = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
    start_onboarding = time.time()
    digital_identity = id_manager.onboard_user(user_id, user_attributes)
    end_onboarding = time.time()
    onboarding_time = end_onboarding - start_onboarding
    id_metrics.record_onboarding_time(onboarding_time)
    
    start_verif = time.time()
    verification_ok = id_manager.verify_identity(digital_identity)
    end_verif = time.time()
    verification_time = end_verif - start_verif
    id_metrics.record_verification_time(verification_time)
    
    digital_identities.append(digital_identity)
    print(f"Digital Identity for {user_id} :", digital_identity)
    print(f"Onboarding Time for {user_id} : {onboarding_time:.6f} seconds")
    if verification_ok:
        print(f"Identity verification successful for {user_id} in {verification_time:.6f} seconds.")
        #print(f"Identity verification successful for {user_id} in {verification_time:.6f} seconds.")

    else:
        print(f"Identity verification failed for {user_id} in {verification_time:.6f} seconds.")

print("\n=== Digital Identity Metrics ===")
print("Average Time of digital onboarding :", id_metrics.average_onboarding_time(), "seconds")
print("Average Time of identity verification :", id_metrics.average_verification_time(), "seconds")

# Export metrics to CSV
import csv
with open("identity_metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["User_ID", "Onboarding_Time", "Verification_Time"])
    for i, identity in enumerate(digital_identities, 1):
        writer.writerow([f"User_{i}", id_metrics.onboarding_times[i-1], id_metrics.verification_times[i-1]])

        



    # (Additional phases, such as deposits, can be handled similarly.)
        
if __name__ == "__main__":
    simulation = OfflinePaymentSimulation()
    
    # --- Number of Withdrawals ---
    try:
        nb_withdrawals = int(input("Enter the number of withdrawals to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 withdrawal.")
        nb_withdrawals = 1
    try:
        withdrawal_amount = int(input("Enter the amount for each withdrawal (in units): "))
    except ValueError:
        print("Invalid amount, defaulting to 50 units.")
        withdrawal_amount = 50

    simulation.simulate_multiple_withdrawals(nb_withdrawals, withdrawal_amount)
    
     #--- Offline Transactions ---
    try:
        nb_offline = int(input("Enter the number of offline transactions to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 offline transaction.")
        nb_offline = 1
    try:
        offline_amount = int(input("Enter the amount for each offline transaction (in units): "))
    except ValueError:
        print("Invalid amount, using a random amount.")
        offline_amount = None

    simulation.simulate_multiple_offline_transactions(nb_offline, offline_amount)
    
    # --- Deposits ---
    try:
        nb_deposits = int(input("Enter the number of deposits to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 deposit.")
        nb_deposits = 1
    simulation.simulate_multiple_deposits(nb_deposits)
    
    # --- Bob Pays Carla Transactions ---
    try:
        nb_bob_to_carla = int(input("Enter the number of transactions between Bob and Carla to perform: "))
    except ValueError:
        print("Invalid number, defaulting to 1 transaction.")
        nb_bob_to_carla = 1
    try:
        bob_to_carla_amount = int(input("Enter the amount for each Bob-to-Carla transaction (in units): "))
    except ValueError:
        print("Invalid amount, defaulting to 50 units.")
        bob_to_carla_amount = 50

    for i in range(nb_bob_to_carla):
        #simulation.simulate_bob_pays_carla(bob_to_carla_amount)
    
    # Generate and export the performance metrics CSV
       simulation.environment.generate_metrics()

print("\n=== Digital identity metrics ===")
print("Average time of digital onboarding :", id_metrics.average_onboarding_time(), "seconds")
print("Average time of identity verification :", id_metrics.average_verification_time(), "seconds")



=== Digital Onboarding Phase for multiple users ===


Entrez le nombre d'utilisateurs à onboarder :  1


Digital Identity for User_1 : {'user_id': 'User_1', 'credential': {'user_id': 'User_1', 'credential': 'Credential_for_User_1', 'issued_at': 1757869147.401377, 'doc_hash': 'e8b276a22e9de9d759859a4d2d16507945827fbf03018253f83bd84592845a9f', 'signature': '59464f7c01397d2ac210434e0fd6a168f18a9cafe429f9a3d41249ecfc4596fc'}, 'pid': '24c4d330ccd33f1e4516a4ba080e6f58123f49021344046bdcd850a7da9f4fc9'}
Onboarding Time for User_1 : 0.000000 seconds
Identity verification successful for User_1 in 0.000000 seconds.

=== Digital Identity Metrics ===
Average Time of digital onboarding : 0.0 seconds
Average Time of identity verification : 0.0 seconds


Enter the number of withdrawals to perform:  1
Enter the amount for each withdrawal (in units):  1


Number of withdrawals requested: 1.
Alice withdraws 1 units...
Coin stored: {'coin_id': '8039afff1535ca2d19a7dbac1ab2e6255ba198b6de7eb99c8513315f70b34252', 'value': 1, 'signature': 'kn1R6WCFocUtchQs+vlURyzHL/C/CjYREtnx6TWbftrEpl+SMRPxdS0GNf6PtQn9bzbbw27IiOLkhJS6ZBjKU7MVvnLyyKCc/LcvjSLsNW6xNUkwZ+XBnXhSw3GRFX1ZpGpvzS24lIcjCD/F174lsIMi9zoIzGmP4+EhJyRM1I449H2PdtS5ZuvqMomoiC2GF+Y/ge5XFsXRCEe538ZsFLW1kEzw7xz8z2J0UHqrgZfIsM+qLhDku8L4z1qMIdduAR5tWFTP5lEYxn4xlPl30a3vH/GO23pHB0Kxow3zmXr732cgDPLjA36RRSB3K+nfNgNxqZUxhS0scpzQl4EC5Q=='}
Withdrawal completed successfully. Total time: 0.110939s, Interaction time (CPU): 0.000000s, Computing load (CPU): 0.062500s.


...

Number of offline transactions requested: 1.
Number of coins available in the wallet: 1.
Executing 1 offline transaction(s).
Sending the encapsulated transaction via pyipv8...
PyIPv8Transport: Sending payload:
SimplePayload
| data: {'header': {'protocol': 'ipv8', 'timestamp': 1757869181.2627463, 'sender': 'Alice', 'receiver': 'Bob'}, 'data': {'encrypted_aes_key': 'bC/3N1QcRJ2QApfjz2qb5aSEv2YHhLbM9OgYD4NI7hImRPSQq8LzsbCY0j756UGHI4wqVhL1cQhGhujfE/mjOR/iW9wapWb1BIucTz1P5WKpKDT1jN2Kf99JHKU49F4NvWBjDWjZJd+iG3IH6evpH1BqCjaUzyrLIAV+kBTrMTZSG6fYGTYx/hDhNF5IS824QqRTgmvgyJbXHUX0SKS1eX8Tne20D5VxFbMi0Jj4SMymUXyhcurnad1AchM1ZgAeKnu5BuevHHq5VXNvEWgLszZhX9AKOltaFCanIi0ZhPGM/ZWqB2bNuMkR5uRruY9XAtRmHZWNGQbzrrJ/aUMlRQ==', 'ciphertext': 'LYDn06aDxNHz1uV1uQ1z9Q1AiXvfAsH7KXEcHUQrfgpW6/F73nlYL/dCJWhPobR14LoN2vQk9fewL9mvEA9f4XqjKNMFRgtcl7Srote3vwT6SAHDlXACWyjQwRaoGisndRRYVD5Zur7qPsxHWfbcbJST79w/YVerySg2Rd8CuSmvtGljikChiXu0kc+x6qbZcdn8j3sfHz8cW7xybvVoD9jXyl1S/WsAbtdbA+rNz+sDE3zUzGmo6D/EdOvpX66O9uRD4M7J6XFcqmY

Enter the number of deposits to perform:  1


Number of deposits requested: 1.
Number of coins available in Bob's wallet: 1.
Deposit failed: Coin Already Defunded or Verification Failed


Enter the number of transactions between Bob and Carla to perform:  1
Enter the amount for each Bob-to-Carla transaction (in units):  1



--- Simulation Metrics ---
Total Transactions: 2
Total Latency (seconds): 0.210151
Overall Throughput (transactions/sec): 9.516947

Throughput per phase:
  withdrawal: 9.013961 transactions/sec
  offline_transaction: 10.079384 transactions/sec

=== Digital identity metrics ===
Average time of digital onboarding : 0.0 seconds
Average time of identity verification : 0.0 seconds


In [ ]:
# !pip install pycryptodome
# !pip install pyipv8
# cbdc_simulator.py
import time
import random
import hashlib
import rsa
import pandas as pd
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import json
import base64
import os
from web3 import Web3

# --- Handling pyipv8 and defining a concrete class SimplePayload ---
try:
    from ipv8.messaging.payload import Payload
except ImportError:
    print("pyipv8 not found, using a simulated Payload object.")
    class Payload:
        def __init__(self):
            self.data = None
        def __str__(self):
            return str(self.data)
    SimplePayload = Payload
else:
    class SimplePayload(Payload):
        def __init__(self):
            super().__init__()
            self.data = None
        def to_pack_list(self):
            return [self.data]
        @classmethod
        def from_unpack_list(cls, lst):
            instance = cls()
            instance.data = lst[0]
            return instance

def xor_bytes(a, b):
    """Returns the byte-wise XOR of two byte sequences of the same length."""
    return bytes(x ^ y for x, y in zip(a, b))

# --- PyIPv8Transport class using SimplePayload ---
class PyIPv8Transport:
    @staticmethod
    def encapsulate(packet, sender="Alice", receiver="Bob"):
        header = {
            "protocol": "ipv8",
            "timestamp": time.time(),
            "sender": sender,
            "receiver": receiver
        }
        payload = SimplePayload()
        payload.data = {"header": header, "data": packet}
        return payload

    @staticmethod
    def send(packet, sender="Alice", receiver="Bob"):
        payload = PyIPv8Transport.encapsulate(packet, sender, receiver)
        print("PyIPv8Transport: Sending payload:")
        print(payload)
        return payload

    @staticmethod
    def receive(payload):
        print("PyIPv8Transport: Received payload:")
        print(payload)
        data = payload.data.get("data")
        return data

# --- Other unchanged classes (NFCMock, ZKSNARK, CentralBank, OfflineWallet, Environment) ---
class NFCMock:
    @staticmethod
    def send(transaction_data):
        print("NFCMock: Transaction data sent.")
        return json.dumps(transaction_data)

    @staticmethod
    def receive(transaction_json):
        print("NFCMock: Transaction data received.")
        return json.loads(transaction_json)

class ZKSNARK:
    @staticmethod
    def generate_key():
        return "simulated_zk_key"

    @staticmethod
    def generate_proof(private_key, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        return hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()

    @staticmethod
    def verify_proof(proof, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        expected_proof = hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()
        return proof == expected_proof

class CentralBank:
    def __init__(self, name):
        self.name = name
        self.coins_issued = []
        self.key_pair = rsa.newkeys(2048)
        self.deposited_coins = set()
        self.used_nonces = set()
        self.logged_transactions = []

    def issue_coin(self, recipient, value):
        coin_id = hashlib.sha256(str(random.getrandbits(256)).encode()).hexdigest()
        signature = self.sign_coin(value, coin_id)
        signature_b64 = base64.b64encode(signature).decode('utf-8')
        coin = {
            "coin_id": coin_id,
            "value": value,
            "signature": signature_b64
        }
        self.coins_issued.append(coin)
        return coin

    def sign_coin(self, value, coin_id):
        message = f"{value}{coin_id}".encode()
        return rsa.sign(message, self.key_pair[1], 'SHA-256')

    def sign_blinded_hash(self, blinded_hash):
        return hashlib.sha256(f"{blinded_hash}{self.name}".encode()).hexdigest()

    def verify_and_deposit_coin(self, coin, sender_public_key, proof, zk_key, nonce):
        coin_hash = hashlib.sha256((str(coin['value']) + coin['coin_id'] + coin['signature']).encode()).hexdigest()
        if coin_hash in self.deposited_coins:
            return "Coin Already Defunded or Verification Failed"
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, self.key_pair[0])
        except rsa.VerificationError:
            return "Coin Already Defunded or Verification Failed"
        if nonce in self.used_nonces:
            return "Nonce already used. Double spending detected."
        self.used_nonces.add(nonce)
        amount = coin['value']
        expected_proof = hashlib.sha256(f"{sender_public_key}{amount}{zk_key}".encode()).hexdigest()
        if proof != expected_proof:
            return "Coin Already Defunded or Verification Failed"
        transaction_record = (sender_public_key.save_pkcs1(), nonce, proof)
        if transaction_record in self.logged_transactions:
           return "Double spending detected: Transaction already recorded."
        self.logged_transactions.append(transaction_record)
        self.deposited_coins.add(coin_hash)
        return "Coin Deposited Successfully"

class OfflineWallet:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.balance = 100
        self.key_pair = self.generate_key_pair()
        self.zk_key = ZKSNARK.generate_key()
        self.nonce = 0
        self.withdrawal_latencies = []
        self.deposit_latencies = []
        self.coins = []
        self.received_logs = []

    def log_transaction(self, sender_key, nonce, proof):
        self.received_logs.append((sender_key.save_pkcs1(), nonce, proof))

    def has_received_transaction(self, sender_key, nonce, proof):
        return (sender_key.save_pkcs1(), nonce, proof) in self.received_logs

    def generate_key_pair(self):
        return rsa.newkeys(2048)

    def store_coin(self, coin):
        self.coins.append(coin)
        print(f"Coin stored: {coin}")

    def prepare_transaction(self, recipient, amount):
        self.nonce += 1
        if not self.coins:
            raise ValueError("No coins available to send.")
        coin = self.coins[0]
        proof = ZKSNARK.generate_proof(self.key_pair[1], recipient.key_pair[0], amount, self.zk_key)
        return {
            "amount": amount,
            "coin": coin.copy(),
            "recipient_key": base64.b64encode(recipient.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "from_agent_key": base64.b64encode(self.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.nonce,
            "timestamp": time.time()
        }

    def blind_coin(self):
        f = hashlib.sha256(self.key_pair[0].save_pkcs1()).hexdigest()
        blinding_factor = random.randint(1, 10**5)
        blinded_hash = (int(f, 16) * blinding_factor) % (2**256)
        return {"hash": f, "blinded_hash": blinded_hash, "blinding_factor": blinding_factor}

class Environment:
    def __init__(self):
        self.measurement = {
            "transaction_id": [],
            "period": [],
            "to_agent": [],
            "from_agent": [],
            "settlement_type": [],
            "amount": [],
            "description": [],
            "latency": [],
            "zk_proof_generation_time": [],
            "zk_proof_verification_time": [],
            "transaction_verification_time": [],
            "load_computing": [],
            "gas_used": [], 
            "decryption_time": [],         # New field for decryption time
            "decryption_cpu_time": [],     # New field for decryption CPU load
            "status": []
        }

    def log_transaction(self, transaction_id, period, to_agent, from_agent, settlement_type, amount, description,
                        latency=None, zk_proof_gen_time=None, zk_proof_ver_time=None, transaction_ver_time=None,
                        load_computing=None, gas_used=None, decryption_time=None, decryption_cpu_time=None, status="Pending"):
        self.measurement["transaction_id"].append(transaction_id)
        self.measurement["period"].append(period)
        self.measurement["to_agent"].append(to_agent)
        self.measurement["from_agent"].append(from_agent)
        self.measurement["settlement_type"].append(settlement_type)
        self.measurement["amount"].append(amount)
        self.measurement["description"].append(description)
        self.measurement["latency"].append(latency)
        self.measurement["zk_proof_generation_time"].append(zk_proof_gen_time)
        self.measurement["zk_proof_verification_time"].append(zk_proof_ver_time)
        self.measurement["transaction_verification_time"].append(transaction_ver_time)
        self.measurement["load_computing"].append(load_computing)
        self.measurement["gas_used"].append(gas_used)
        self.measurement["decryption_time"].append(decryption_time)         # Record decryption time
        self.measurement["decryption_cpu_time"].append(decryption_cpu_time) # Record decryption CPU load
        self.measurement["status"].append(status)

    def generate_metrics(self):
        df = pd.DataFrame(self.measurement)
        total_transactions = len(df)
        total_latency = df["latency"].sum()
        overall_throughput = total_transactions / total_latency if total_latency > 0 else 0

        phase_throughput = {}
        for phase in df['settlement_type'].unique():
            phase_df = df[df['settlement_type'] == phase]
            phase_count = len(phase_df)
            phase_total_latency = phase_df["latency"].sum()
            phase_throughput[phase] = phase_count / phase_total_latency if phase_total_latency > 0 else 0
        df["throughput"] = df["settlement_type"].apply(lambda phase: phase_throughput.get(phase, 0))

        for col in ["zk_proof_generation_time", "zk_proof_verification_time", "load_computing", "throughput"]:
            df[col] = df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) and isinstance(x, (float, int)) else x)

        print("\n--- Simulation Metrics ---")
        print(f"Total Transactions: {total_transactions}")
        print(f"Total Latency (seconds): {total_latency:.6f}")
        print(f"Overall Throughput (transactions/sec): {overall_throughput:.6f}")

        print("\nThroughput per phase:")
        for phase, tp in phase_throughput.items():
            print(f"  {phase}: {tp:.6f} transactions/sec")
        df.to_csv("simulation_metricswithpad.csv", index=False)

# --- Integration of the scenario "Bob pays Carla" ---
class OfflinePaymentSimulation:
    def __init__(self):
        self.environment = Environment()
        self.alice_wallet = OfflineWallet("Alice")
        self.bob_wallet = OfflineWallet("Bob")
        self.carla_wallet = OfflineWallet("Carla")  # New wallet for Carla
        self.central_bank = CentralBank("Central Bank")
        self.used_nonces = set()
        
    def validate_coin_signature(self, coin, public_key):
        if not coin or "signature" not in coin:
            print("Invalid coin: Missing or no signature.")
            return False
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, public_key)
            print("Coin signature is valid.")
            return True
        except rsa.VerificationError:
            print("Invalid coin: Signature verification failed.")
            return False

    def verify_sender_legitimacy(self, sender_public_key, known_keys):
        return any(sender_public_key.save_pkcs1() == key.save_pkcs1() for key in known_keys)

    def verify_nonce(self, nonce, used_nonces):
        if nonce in used_nonces:
            print(f"Nonce {nonce} has already been used. Double spending detected.")
            return False
        used_nonces.add(nonce)
        return True

    def verify_recipient(self, transaction_data, expected_recipient_key):
        recipient_key = transaction_data.get("recipient_key")
        if not isinstance(recipient_key, rsa.PublicKey):
            print("Failure: recipient_key is not an rsa.PublicKey object.")
            return False
        return recipient_key.save_pkcs1() == expected_recipient_key.save_pkcs1()

    def simulate_deposit(self):
        """Simulates the deposit of a coin to the central bank."""
        if not self.bob_wallet.coins:
            print("Bob has no coin to deposit.")
            return

        coin = self.bob_wallet.coins[0]  # Select the first coin for deposit
        amount = coin['value']

        computing_start = time.process_time()
        start_time = time.time()

        # Generate ZK proof for deposit
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1], self.central_bank.key_pair[0], amount, self.bob_wallet.zk_key)

        # Prepare deposit data
        deposit_data = {
            "coin": coin.copy(),
            "sender_public_key": base64.b64encode(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.bob_wallet.nonce + 1,  # Increment nonce for the deposit
            "timestamp": time.time()
        }

        # Simulate the deposit process
        deposit_result = self.central_bank.verify_and_deposit_coin(
            coin,
            self.bob_wallet.key_pair[0],
            proof,
            self.bob_wallet.zk_key,
            deposit_data["nonce"]  # Use the incremented nonce for deposit
        )

        if deposit_result == "Coin Deposited Successfully":
            self.bob_wallet.coins.pop(0)  # Remove deposited coin from wallet
            self.bob_wallet.balance -= amount  # Update wallet balance
            self.bob_wallet.nonce += 1  # Increment wallet's nonce after successful deposit
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            print(f"Deposit completed successfully. Total time: {latency:.6f}s, "
                  f"Computing load (CPU): {computing_load:.6f}s.")

            self.environment.log_transaction(
                transaction_id=len(self.environment.measurement["transaction_id"]),
                period=time.time(),
                to_agent="Central Bank",
                from_agent=self.bob_wallet.owner_id,
                settlement_type="deposit",
                amount=amount,
                description="Coin Deposit",
                latency=latency,
                zk_proof_gen_time=None,  # Add timing if needed
                zk_proof_ver_time=None,  # Add timing if needed
                transaction_ver_time=None,  # Add timing if needed
                load_computing=computing_load,
                status="Success"
            )
        else:
            print(f"Deposit failed: {deposit_result}")

    # --- Withdrawal method ---
    def simulate_withdrawal(self, amount):
        print(f"Alice withdraws {amount} units...")
        computing_start = time.process_time()
        total_start_time = time.time()
        blinded_coin = self.alice_wallet.blind_coin()
        interaction_start_time = time.process_time()
        blinded_signature = self.central_bank.sign_blinded_hash(blinded_coin["blinded_hash"])
        interaction_cpu_time = time.process_time() - interaction_start_time
        message = f"{amount}{self.alice_wallet.owner_id}".encode()
        unblinded_signature = rsa.sign(message, self.central_bank.key_pair[1], 'SHA-256')
        coin = self.central_bank.issue_coin(self.alice_wallet.owner_id, amount)
        self.alice_wallet.store_coin(coin)
        total_latency = time.time() - total_start_time
        computing_load = time.process_time() - computing_start
        print(f"Withdrawal completed successfully. Total time: {total_latency:.6f}s, "
              f"Interaction time (CPU): {interaction_cpu_time:.6f}s, "
              f"Computing load (CPU): {computing_load:.6f}s.")
        self.environment.log_transaction(
            transaction_id=len(self.environment.measurement["transaction_id"]),
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Central Bank",
            settlement_type="withdrawal",
            amount=amount,
            description="Withdrawal with blinding",
            latency=total_latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=interaction_cpu_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Offline payment method (Alice pays Bob) ---
    def simulate_offline_transaction(self, amount=None):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        if amount is None:
            amount = random.randint(10, 50)
        if not self.alice_wallet.coins:
            print("No coin available for an offline transaction.")
            return
        coin = self.alice_wallet.coins[0]
        if amount > coin["value"]:
            print(f"Requested amount ({amount}) greater than coin value ({coin['value']}). Using the coin's value.")
            amount = coin["value"]
        elif amount < coin["value"]:
            print("Partial transaction not supported. Using the full coin value.")
            amount = coin["value"]
        computing_start = time.process_time()
        start_time = time.time()
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.bob_wallet.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start_time
        transaction_data = self.alice_wallet.prepare_transaction(self.bob_wallet, amount)
        transaction_data.update({
            "transaction_id": transaction_id,
            "from_agent": self.alice_wallet.owner_id,
            "to_agent": self.bob_wallet.owner_id,
            "nonce": transaction_data["nonce"]
        })
        json_data = json.dumps(transaction_data).encode('utf-8')
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_data)
        packet = {
            "encrypted_aes_key": base64.b64encode(
                PKCS1_OAEP.new(
                    RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM'))
                ).encrypt(aes_key)
            ).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Sending the encapsulated transaction via pyipv8...")
        ipv8_payload = PyIPv8Transport.send(packet, sender=self.alice_wallet.owner_id, receiver=self.bob_wallet.owner_id)
        received_packet = PyIPv8Transport.receive(ipv8_payload)
        try:
            enc_aes_key = base64.b64decode(received_packet["encrypted_aes_key"])
            ciphertext = base64.b64decode(received_packet["ciphertext"])
            tag = base64.b64decode(received_packet["tag"])
            nonce = base64.b64decode(received_packet["nonce"])
            bob_private_pem = self.bob_wallet.key_pair[1].save_pkcs1(format='PEM')
            bob_private_key_py = RSA.importKey(bob_private_pem)
            cipher_rsa_bob = PKCS1_OAEP.new(bob_private_key_py)
            decrypted_aes_key = cipher_rsa_bob.decrypt(enc_aes_key)
            cipher_aes_bob = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=nonce)
            decrypted_data = cipher_aes_bob.decrypt_and_verify(ciphertext, tag)
            received_transaction = json.loads(decrypted_data.decode('utf-8'))
            if isinstance(received_transaction["from_agent_key"], str):
                try:
                    sender_public_key = rsa.PublicKey.load_pkcs1(base64.b64decode(received_transaction["from_agent_key"]))
                    received_transaction["from_agent_key"] = sender_public_key
                    print("Issuer's public key correctly loaded.")
                except Exception as e:
                    print(f"Error while decoding the issuer's key: {e}")
                    return
            if isinstance(received_transaction["recipient_key"], str):
                try:
                    received_transaction["recipient_key"] = rsa.PublicKey.load_pkcs1(
                        base64.b64decode(received_transaction["recipient_key"])
                    )
                    print("recipient_key correctly loaded in PublicKey.")
                except Exception as e:
                    print(f"Error while decoding recipient_key: {e}")
                    return
            if not isinstance(received_transaction["recipient_key"], rsa.PublicKey):
                print("Error: recipient_key is not a valid rsa.PublicKey object after processing.")
                return
            print(f"Received recipient_key: {received_transaction['recipient_key']}")
            print(f"Type of recipient_key: {type(received_transaction['recipient_key'])}")
            verification_start_time = time.time()
            if not self.validate_coin_signature(received_transaction.get("coin"), self.central_bank.key_pair[0]):
                print("Failed to verify the coin signature.")
                return
            sender_public_key = received_transaction.get("from_agent_key")
            if not self.verify_sender_legitimacy(sender_public_key, [self.alice_wallet.key_pair[0]]):
                print("Failed to verify the sender’s legitimacy.")
                return
            else:
                print("Sender legitimacy verification successful: The sender is legitimate.")
            proof = received_transaction.get("proof")
            nonce = received_transaction.get("nonce")
            if self.bob_wallet.has_received_transaction(sender_public_key, nonce, proof):
                print("Double spending detected: transaction already received.")
                return
            else:
                self.bob_wallet.log_transaction(sender_public_key, nonce, proof)
                print("Transaction successfully logged.")
            if not self.verify_nonce(received_transaction.get("nonce"), self.used_nonces):
                print("Failed to verify the nonce.")
                return
            zk_proof_ver_start_time = time.time()
            proof_valid = ZKSNARK.verify_proof(
                proof,
                self.bob_wallet.key_pair[0],
                amount,
                self.alice_wallet.zk_key
            )
            zk_proof_ver_time = time.time() - zk_proof_ver_start_time
            if not proof_valid:
                print("zk-SNARK proof verification failed.")
                return
            else:
                print("zk-SNARK proof verification successful: The proof is valid.")
            print("Nonce is verified and unique, and the zk-SNARK proof is verified and unique.")
            verification_time = time.time() - verification_start_time
            self.alice_wallet.coins.pop(0)
            received_transaction["coin"]["nonce"] = received_transaction["nonce"]
            received_transaction["coin"]["recipient"] = self.bob_wallet.owner_id
            self.bob_wallet.store_coin(received_transaction["coin"])
            self.alice_wallet.balance -= amount
            self.bob_wallet.balance += amount
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            self.environment.log_transaction(
                transaction_id=transaction_id,
                period=time.time(),
                to_agent=self.bob_wallet.owner_id,
                from_agent=self.alice_wallet.owner_id,
                settlement_type="offline_transaction",
                amount=amount,
                description="Offline Transaction",
                latency=latency,
                zk_proof_gen_time=zk_proof_gen_time,
                zk_proof_ver_time=zk_proof_ver_time,
                transaction_ver_time=verification_time,
                load_computing=computing_load,
                status="Success"
            )
        except Exception as e:
            print(f"Error during the transaction: {e}")

    # --- New method: scenario of Bob paying Carla ---
    def simulate_bob_pays_carla(self, amount):
        # Define the transaction identifier as the current number of transactions + 1
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        # Start measuring time (wall time and CPU time)
        computing_start = time.process_time()
        start_time = time.time()

        print(f"\n=== Scenario: Bob pays Carla for {amount} units===")
        # Step 1: Bob and Carla agree on a contract
        contract = f"Contract_Bob_pays_Carla_{amount}"
        print("Bob and Carla agree on the contract :", contract)

        # Step 2: Bob selects the RIO attestation (dummy value)
        rio_cert = "RIO_pk_cert_dummy"
        print("Bob selects the RIO certificate :", rio_cert)

        # Step 3: Carla provides Bob with one of her pseudonymous identities
        carla_pseudonym = self.carla_wallet.key_pair[0].save_pkcs1().decode('utf-8')
        print("Carla provides Bob with his pseudonymous identifier.")

        # Step 4: Bob selects an e-coin corresponding to the desired amount
        if not self.bob_wallet.coins:
            print("Bob has no coin available for the payment.")
            return
        ecoin = self.bob_wallet.coins[0]
        print("Selected e-Coin :", ecoin)

        # Step 5: Bob signs the contract with his e-coin
        signature_contract = rsa.sign(contract.encode('utf-8'), self.bob_wallet.key_pair[1], 'SHA-256')
        signature_contract_b64 = base64.b64encode(signature_contract).decode('utf-8')
        print("Bob signs the contract :", signature_contract_b64)

        # Step 6: Bob generates the zk-SNARK proof of payment
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1],
                                       self.carla_wallet.key_pair[0],
                                       amount,
                                       self.bob_wallet.zk_key)
        print("Generated proof :", proof)
        # Measure the proof verification time
        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(
                    proof,
                    self.carla_wallet.key_pair[0],
                    amount,
                    self.bob_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start
        if not proof_valid:
            print("zk-SNARK proof verification failed.")
            return
        else:
            print("zk-SNARK proof verification successful: The proof is valid.")

        # Step 7: Bob constructs the transaction packet including the following data
        transaction_payload = {
            "contract": contract,
            "contract_signature": signature_contract_b64,
            "proof": proof,
            "RIO_cert": rio_cert,
            "amount": amount,
            "bob_pseudonym": self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8'),
            "carla_pseudonym": carla_pseudonym,
            "timestamp": time.time()
        }
        json_payload = json.dumps(transaction_payload).encode('utf-8')

        # Step 8: Encryption of the transaction packet (RIO simulator)
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_payload)

        # Generation of a key pair for RIO
        rio_key_pair = rsa.newkeys(2048)
        rio_public = rio_key_pair[0]
        rio_private = rio_key_pair[1]

        # Encryption of the AES key with RIO's public key
        rio_public_pem = rio_public.save_pkcs1(format='PEM')
        cipher_rsa_enc = PKCS1_OAEP.new(RSA.importKey(rio_public_pem))
        encrypted_aes_key = cipher_rsa_enc.encrypt(aes_key)

        encrypted_packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Bob-to-Carla transaction (encrypted for RIO) :", encrypted_packet)

        # --- Simulation of the verification phase by the RIO ---
        # To decrypt, the RIO uses its private key
        rio_private_pem = rio_private.save_pkcs1(format='PEM')
        cipher_rsa_dec = PKCS1_OAEP.new(RSA.importKey(rio_private_pem))
        # Decrypt the AES key
        decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)

        # Use the decrypted key to decrypt the payload
        cipher_aes_rio = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=cipher_aes.nonce)
        decrypted_payload = cipher_aes_rio.decrypt_and_verify(ciphertext, tag)
        # Here, the RIO verifies the payload data (contract, signature, proof, etc.)
        # Initialize verification_start_time here within simulate_bob_pays_carla
        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time: {transaction_verification_time:.6f} seconds")

        # ----------------------------
        # TOTAL LATENCY AND CPU LOAD CALCULATION
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
        # ----------------------------
        # RECORDING METRICS FOR THE "BOB PAYS CARLA" SCENARIO
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.carla_wallet.owner_id,
            from_agent=self.bob_wallet.owner_id,
            settlement_type="online_transaction",  # or "bob_to_carla"
            amount=amount,
            description="Payment from Bob to Carla",
            latency=latency,
            zk_proof_gen_time=None,  # You can add proof generation time if needed
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=computing_load,
            status="Success"
        )

    def simulate_online_deposit_PAD(self, amount):
        """
        Online Deposit/Transfer scenario via PAD.
        This scenario simulates the online deposit (or transfer to one's own account)
        through the PAD system managed by the RIO.
        """
        print(f"\n=== Online Deposit scenario via PAD for {amount} units ===")
        # Define timing variables and the transaction identifier
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()

        # -- Step 0: Transaction Verification (similar to the verification of offline transactions) --
        # Here, the RIO verifies that the transaction is not a double spending.
        # For the simulation, we use the existing zk-SNARK proof verification.
        # Suppose a coin deposited by Alice is used for the online deposit.
        if not self.alice_wallet.coins:
            print("No coin available for an online deposit.")
            return
        coin = self.alice_wallet.coins[0]
        # Proof generation and verification (similar to the other scenario)
        zk_proof_start = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.central_bank.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start

        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(proof,
                                           self.central_bank.key_pair[0],
                                           amount,
                                           self.alice_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start

        if not proof_valid:
            print("zk-SNARK proof verification failed during the online transaction.")
            return
        print("Successful verification of zk-SNARK proof.")

        # -- Step 1: Securing the blinded coin F_s --
        # We assume that Alice's e-coin is used as the blinded coin F_s.
        Fs = self.alice_wallet.blind_coin()  # F_s
        # Generate a token T (for the simulation, a random number as a string)
        T = str(random.randint(1000, 9999))
        print("Generated Token T :", T)
           
        # Save T in the object so that it is available for the decryption step
        self.last_token_value = T
    
        # Simulate the verification key K_verif (e.g., 16 bytes)
        K_verif = get_random_bytes(16)
        print("Verification key K_verif generated :", base64.b64encode(K_verif).decode())
        self.last_K_verif = K_verif
       
        # Generate a symmetric key K_sym
        K_sym = get_random_bytes(32)
        # Encrypt (T, F_s) with K_sym (we concatenate T and a part of Fs)
        data_to_encrypt = T.encode('utf-8') + json.dumps(Fs, sort_keys=True).encode('utf-8')
        cipher_pad = AES.new(K_sym, AES.MODE_GCM)
        C, tag = cipher_pad.encrypt_and_digest(data_to_encrypt)
        nonce_pad = cipher_pad.nonce
        # C represents the encrypted content containing the token and F_s

        # Now, sign and encrypt the content with Alice's private key and then with the recipient's public key
        # (here, for the online deposit, we assume the transaction remains on Alice's same account)
        # For the simulation, we will sign C with Alice's private key:
        signature_C = rsa.sign(C, self.alice_wallet.key_pair[1], 'SHA-256')
        signature_C_b64 = base64.b64encode(signature_C).decode('utf-8')
        # We form a packet C' that combines the content C and its signature
        C_prime = {
            "encrypted_content": base64.b64encode(C).decode('utf-8'),
            "signature": signature_C_b64
        }
        # Then, encrypt C' with the recipient's public key.
        # In the case of a deposit on the same account, we can encrypt with Alice's public key.
        # --- Hybrid encryption of C_prime ---
        # Convert C_prime to JSON (data to be encrypted)
        C_prime_json = json.dumps(C_prime).encode('utf-8')

        # 1. Generate a symmetric key K_PAD to encrypt C_prime_json with AES
        K_PAD = get_random_bytes(32)
        cipher_pad2 = AES.new(K_PAD, AES.MODE_GCM)
        C_prime_ciphertext, C_prime_tag = cipher_pad2.encrypt_and_digest(C_prime_json)
        nonce_pad2 = cipher_pad2.nonce

        # 2. Encrypt the symmetric key K_PAD with Alice's public key (in this case, the online deposit is on her own account)
        cipher_rsa_for_PAD = PKCS1_OAEP.new(RSA.importKey(self.alice_wallet.key_pair[0].save_pkcs1(format='PEM')))
        encrypted_K_PAD = cipher_rsa_for_PAD.encrypt(K_PAD)

        # 3. Form the final packet for C_prime by grouping the encrypted key and the ciphertext
        C_prime_encrypted = {
            "encrypted_key": base64.b64encode(encrypted_K_PAD).decode('utf-8'),
            "ciphertext": base64.b64encode(C_prime_ciphertext).decode('utf-8'),
            "tag": base64.b64encode(C_prime_tag).decode('utf-8'),
            "nonce": base64.b64encode(nonce_pad2).decode('utf-8')
        }

        # Masking the symmetric key with a mask R_sym
        R_sym = get_random_bytes(32)
        S_m = xor_bytes(K_sym, R_sym)
        # Generate a dummy share for trustees (N_t, for example 3)
        N_t = b"3"
        S_t = hashlib.sha256(S_m + N_t).hexdigest()
        # Generate a dummy share for validators (N_v, for example 2)
        N_v = b"2"
        V = hashlib.sha256(b"R" + self.alice_wallet.key_pair[0].save_pkcs1() + N_v).hexdigest()

        # Encrypt dummy shares for trustees and validators (simulated here)
        E_t = "encrypted_trustee_shares_dummy"
        E_v = "encrypted_validator_shares_dummy"

        # Final data transmission structure D_ts
        D_ts = {
            "E_t": E_t,
            "E_v": E_v,
            "H_T": hashlib.sha256(T.encode('utf-8')).hexdigest(),
            "C_prime_encrypted": json.dumps(C_prime_encrypted)
        }

        print("PAD data generated (D_ts) :", D_ts)

        # Suppose the RIO receives these data, verifies the proof (already done),
        # then proceeds with the online deposit.
        # Here, we simulate the reception and approval by the RIO.
        # For the simulation, we consider that the transaction is validated by the RIO.
        # Measure the RIO transaction verification time
        verification_start_time = time.time()
        # (Real verification operations would be performed here)
        # We use the real time between receipt and the end of verification
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time (RIO): {transaction_verification_time:.6f} seconds")

        # Update balances and remove Alice's coin
        self.alice_wallet.coins.pop(0)
        self.alice_wallet.balance -= amount
        print(f"Online deposit (via PAD) of {amount} units successfully completed for {self.alice_wallet.owner_id}.")

        # In the simulate_online_deposit_PAD method, at the end, for example:
        # ... (all your proof generation, encryption operations, etc.)
        # At the end, after computing transaction_id, latency, etc.

        # Post the transaction on the PAD ledger via the smart contract
        sender_address = "0x503b361ed78fea4d357b94daef9612ac4c11c093"  # User address (hex format)
        recipient_address = sender_address  # For a deposit to one's online account
        tx_type = "online_deposit_PAD"
        details = "Online Deposit via PAD with Smart Contract"

        # The private key corresponding to sender_address (hex format)
        private_key = "0x4b0b6d1c3ced6d446803fc8b5726605bbba9aa95016bc07ec60ede51e117f8d0"

        receipt = pad_interface.post_transaction(
            transaction_id,
            sender_address,
            recipient_address,
            amount,
            tx_type,
            details,
            private_key
        )

        # You can log metrics using receipt["gasUsed"] or latency time, for example.
        # Display and retrieve the gas used
        # Retrieve gas used (or None if not present)
        gas_used = receipt.get("gasUsed", None)
        print("Gas used for the PAD transaction :", gas_used)
        
        # Calculate total latency and CPU load
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start

        # Record metrics for this PAD scenario
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,  # deposit on her own account
            from_agent=self.alice_wallet.owner_id,
            settlement_type="online_deposit_PAD",
            amount=amount,
            description="Online Deposit via PAD",
            latency=latency,
            zk_proof_gen_time=zk_proof_gen_time,
            zk_proof_ver_time=zk_proof_ver_time,
            transaction_ver_time=transaction_verification_time,
            load_computing=computing_load,
            gas_used=gas_used,  # Added gas used
            status="Success"
        )

    # --- Multiple methods for each type of transaction ---
    def simulate_multiple_offline_transactions(self, nb_transactions, offline_amount=None):
        nb_possible = len(self.alice_wallet.coins)
        nb_to_execute = min(nb_transactions, nb_possible)
        print(f"Number of offline transactions requested: {nb_transactions}.")
        print(f"Number of coins available in the wallet: {nb_possible}.")
        print(f"Executing {nb_to_execute} offline transaction(s).")
        for i in range(nb_to_execute):
            self.simulate_offline_transaction(amount=offline_amount)

    def simulate_multiple_withdrawals(self, nb_withdrawals, withdrawal_amount):
        print(f"Number of withdrawals requested: {nb_withdrawals}.")
        for i in range(nb_withdrawals):
            self.simulate_withdrawal(withdrawal_amount)

    def simulate_multiple_deposits(self, nb_deposits):
        nb_possible = len(self.bob_wallet.coins)
        nb_to_execute = min(nb_deposits, nb_possible)
        print(f"Number of deposits requested: {nb_deposits}.")
        print(f"Number of coins available in Bob's wallet: {nb_possible}.")
        for i in range(nb_to_execute):
            self.simulate_deposit()

    # ... (other already defined methods)

    def simulate_decryption_request(self, T, threshold, K_verif):
        """
        Simulates the decryption process in the PAD protocol:
         - If T > theta, the RIO creates a decryption request and computes φ = G(H(T), K_{Bpub})
         - Then Bob retrieves and decrypts the secret:
              R_sf ← G(E^{-1}(E^{-1}(C', K_{Bpriv}), K_{verif}))
        """
        print("\n=== Start of decryption request process (PIP)(PIP) ===")
        # Step 2: Threshold check and creation of the decryption request
        if int(T) > threshold:
            # Compute H(T)
            H_T = hashlib.sha256(T.encode()).hexdigest()
            # Retrieve Bob's public key (as string)
            bob_pub = self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8')
            # Compute φ = G(H(T), K_{Bpub}) (here simulated by hashing the concatenation)
            phi = hashlib.sha256((H_T + bob_pub).encode()).hexdigest()
            print("Decryption request published with φ =", phi)
        else:
            print("Token T (", T, ") below threshold (", threshold, "). No decryption request.")
            return

        # Step 3: Bob retrieves and decrypts the secret
        # For the simulation, we define a simulated secret:
        secret_data = "No decryption request"
        print("Original secret (simulated) :", secret_data)
        
        # --- Simulate double encryption (C') of the secret ---
        # First encryption step: encrypt the secret with the verification key K_verif (AES)
        cipher1 = AES.new(K_verif, AES.MODE_GCM)
        secret_ciphertext, secret_tag = cipher1.encrypt_and_digest(secret_data.encode())
        nonce_secret = cipher1.nonce

        # Second step: encrypt the result (secret_ciphertext) with Bob's public key via RSA
        # Convert Bob's public key
        bob_pub = RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1())
        cipher_rsa_enc = PKCS1_OAEP.new(bob_pub)

        # Note: in practice, RSA only encrypts very short messages. Here, it is a simulation.
        C_prime = cipher_rsa_enc.encrypt(secret_ciphertext)
        print("C_prime (simulated double encryption) =", base64.b64encode(C_prime).decode())
        # Measure the time to decrypt the secret by Bob
        decryption_start_time = time.time()
        cpu_start = time.process_time()
        # Now, Bob will retrieve the secret:
        # First step: decrypt C_prime with his private key (RSA)
        bob_priv = RSA.importKey(self.bob_wallet.key_pair[1].save_pkcs1())
        cipher_rsa_dec = PKCS1_OAEP.new(bob_priv)

        decrypted_ciphertext = cipher_rsa_dec.decrypt(C_prime)
        # Second step: decrypt the result with K_verif (AES)
        cipher2 = AES.new(K_verif, AES.MODE_GCM, nonce=nonce_secret)
        recovered_secret = cipher2.decrypt_and_verify(decrypted_ciphertext, secret_tag).decode()
       
        cpu_end = time.process_time()
        decryption_end_time = time.time()
        decryption_time = decryption_end_time - decryption_start_time
        decryption_cpu_time = cpu_end - cpu_start
    
        # Finally, apply a function G to the recovered secret, here a SHA-256 hash
        R_sf = hashlib.sha256(recovered_secret.encode()).hexdigest()
        print("Bob has recovered the secret R_sf", R_sf)

        print("=== End of decryption request process ===")
        # Record decryption metrics in the CSV (you can adapt how you use them)
        self.environment.log_transaction(
            transaction_id="decryption_request",  # or an appropriate identifier
            period=time.time(),
            to_agent=self.bob_wallet.owner_id,
            from_agent="PAD_System",
            settlement_type="decryption_request",
            amount=0,
            description="PAD secret decryption",
            latency=decryption_time,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=decryption_cpu_time,
            gas_used=None,
            status="Success"
         )

class PADLedgerInterface:
    def __init__(self, provider_url, contract_address, abi_file_path, chain_id=1337, gas=2000000, gas_price_gwei="50"):
        # Connect to Ganache
        self.web3 = Web3(Web3.HTTPProvider(provider_url))
        if not self.web3.is_connected():
            raise Exception("Unable to connect to Ganache.")
        # Convert the address to a checksum address
        self.contract_address = self.web3.to_checksum_address(contract_address)
        # Load the ABI from the JSON file
        with open(abi_file_path, "r") as abi_file:
            self.contract_abi = json.load(abi_file)
        self.contract = self.web3.eth.contract(address=self.contract_address, abi=self.contract_abi)
        self.chain_id = chain_id
        self.gas = gas
        self.gas_price = self.web3.to_wei(gas_price_gwei, "gwei")

    def post_transaction(self, transaction_id, sender, recipient, amount, tx_type, details, private_key):
        # Convert sender and recipient to checksum addresses
        sender = self.web3.to_checksum_address(sender)
        recipient = self.web3.to_checksum_address(recipient)
    
        # Build the transaction
        tx = self.contract.functions.postTransaction(
            transaction_id,  # transactionId
            sender,          # from
            recipient,       # to
            amount,          # amount
            tx_type,         # txType
            details          # details (e.g., "Online Deposit via PAD")
        ).build_transaction({
            "chainId": self.chain_id,
            "gas": self.gas,
            "gasPrice": self.gas_price,
            "nonce": self.web3.eth.get_transaction_count(sender),
        })

        # Sign and send the transaction
        signed_tx = self.web3.eth.account.sign_transaction(tx, private_key=private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_tx.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print("Transaction posted on the PAD ledger :", receipt)
        return receipt

# --- Digital identity management ---
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode('utf-8')).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode('utf-8')).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode('utf-8')).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        digital_identity = {
            "user_id": user_id,
            "credential": credential,
            "pid": pid
        }
        return digital_identity

    def verify_identity(self, digital_identity):
        cred = digital_identity["credential"].copy()
        signature = cred.pop("signature", None)
        cred_str = json.dumps(cred, sort_keys=True)
        expected_signature = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return expected_signature == signature

# --- For measuring digital onboarding performance ---
class IdentityMetrics:
    def __init__(self):
        self.onboarding_times = []
        self.verification_times = []

    def record_onboarding_time(self, t):
        self.onboarding_times.append(t)

    def record_verification_time(self, t):
        self.verification_times.append(t)

    def average_onboarding_time(self):
        if not self.onboarding_times:
            return 0
        return sum(self.onboarding_times) / len(self.onboarding_times)

    def average_verification_time(self):
        if not self.verification_times:
            return 0
        return sum(self.verification_times) / len(self.verification_times)

# Create instances for identity management
id_manager = DigitalIdentityManager()
id_metrics = IdentityMetrics()

# --- Multiple digital onboarding phase with performance measurement ---
print("\n=== Digital Onboarding Phase for multiple users ===")
try:
    nb_users = int(input("Enter the number of users to onboard: "))
except ValueError:
    print("Invalid number, onboarding 1 by default.")
    nb_users = 1

digital_identities = []
for i in range(1, nb_users + 1):
    user_id = f"User_{i}"
    user_attributes = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
    start_onboarding = time.time()
    digital_identity = id_manager.onboard_user(user_id, user_attributes)
    end_onboarding = time.time()
    onboarding_time = end_onboarding - start_onboarding
    id_metrics.record_onboarding_time(onboarding_time)

    start_verif = time.time()
    verification_ok = id_manager.verify_identity(digital_identity)
    end_verif = time.time()
    verification_time = end_verif - start_verif
    id_metrics.record_verification_time(verification_time)

    digital_identities.append(digital_identity)
    print(f"Digital Identity for {user_id} :", digital_identity)
    print(f"Onboarding time for {user_id} : {onboarding_time:.6f} seconds")
    if verification_ok:
        print(f"Identity verification successful for {user_id} in {verification_time:.6f} seconds.")
    else:
        print(f"Identity verification failed for {user_id} in {verification_time:.6f} seconds.")

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time :", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time :", id_metrics.average_verification_time(), "seconds")

# Export metrics to a CSV
import csv
with open("identity_metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["User_ID", "Onboarding_Time", "Verification_Time"])
    for i, identity in enumerate(digital_identities, 1):
        writer.writerow([f"User_{i}", id_metrics.onboarding_times[i-1], id_metrics.verification_times[i-1]])

# --- Transaction simulations ---
simulation = OfflinePaymentSimulation()
simulation.simulate_multiple_withdrawals(4, 50)
try:
    nb_offline = int(input("Enter the number of offline transactions to perform: "))
except ValueError:
    print("Invalid number, defaulting to 1 offline transaction.")
    nb_offline = 1
try:
    offline_amount = int(input("Enter the amount for each offline transaction (in units): "))
except ValueError:
    print("Invalid amount, using a random amount.")
    offline_amount = None
simulation.simulate_multiple_offline_transactions(nb_offline, offline_amount)
try:
    nb_deposits = int(input("Enter the number of deposits to perform: "))
except ValueError:
    print("Invalid number, defaulting to 1 deposit.")
    nb_deposits = 1
simulation.simulate_multiple_deposits(nb_deposits)

# --- New scenario: Bob pays Carla ---
try:
    bob_to_carla_amount = int(input("Enter the amount for the payment from Bob to Carla: "))
except ValueError:
    print("Invalid amount, using 50 units by default.")
    bob_to_carla_amount = 50
simulation.simulate_bob_pays_carla(bob_to_carla_amount)

simulation.environment.generate_metrics()
# Example: configuration for the PAD ledger
provider_url = "http://127.0.0.1:7545"
contract_address = "0xcbec3dab4a3fbd4bd9cf57c98556395e97feec2f"  # Replace with the address obtained in Remix
abi_file_path = "PADLedger_ABI.json"
# Create the interface to communicate with the PAD contract
pad_interface = PADLedgerInterface(provider_url, contract_address, abi_file_path)
# --- New scenario: Online Deposit via PAD
try:
    online_deposit_amount = int(input("Enter the amount for the Online Deposit via PAD (for Alice): "))
except ValueError:
    print("Invalid amount, using 50 units by default.")
    online_deposit_amount = 50
simulation.simulate_online_deposit_PAD(online_deposit_amount)

# Call the new method to simulate the decryption request and secret recovery
# Suppose the threshold theta is 500
theta = 500
# Call the decryption request function using the saved T value and K_verif
simulation.simulate_decryption_request(simulation.last_token_value, theta, simulation.last_K_verif)

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time :", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time :", id_metrics.average_verification_time(), "seconds")



=== Digital Onboarding Phase for multiple users ===


# CBDC Offline/Online Payment Simulator with ShardLedger

This simulator is the same as the previous version of the **CBDC Offline/Online Payment Simulator**, but it additionally integrates a **ShardLedger** interface.

---

## ShardLedger Overview

The **ShardLedger** component represents a **sharded ledger interface** designed to extend the baseline PAD ledger functionalities. While the PAD ledger manages transaction posting (withdrawals, deposits, and online PAD transfers), the **ShardLedger** focuses on:

* **State partitioning (sharding):** splitting account states across multiple shards for scalability and parallel processing.
* **Balance updates:** through the function `update_user_balance()`, the ledger supports homomorphic-style updates (increment/decrement) to encrypted balances.
* **Pseudonym binding:** user identities (pseudonyms) are mapped to shard-specific balances.
* **Efficiency:** reducing overhead by isolating balance updates in shard-specific smart contracts.

In the current prototype, the `ShardLedgerInterface` connects to an Ethereum/Ganache node and exposes a method:

```python
update_user_balance(current_balance_enc, T, op, user_pseudonym)
Where:

current_balance_enc: (placeholder) encrypted balance state,

T: the token or amount involved,

op: operation type ("INC" for increment, "DEC" for decrement),

user_pseudonym: identifier of the user bound to the shard.

This interface can be extended to implement real homomorphic operations or zk-enabled sharded updates.



In [ ]:
# !pip install pycryptodome
# !pip install pyipv8
# cbdc_simulator.py
import time
import random
import hashlib
import rsa
import pandas as pd
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import json
import base64
import os
from web3 import Web3

# --- pyipv8 handling and definition of a concrete SimplePayload class ---
try:
    from ipv8.messaging.payload import Payload
except ImportError:
    print("pyipv8 not found, using a simulated Payload object.")
    class Payload:
        def __init__(self):
            self.data = None
        def __str__(self):
            return str(self.data)
    SimplePayload = Payload
else:
    class SimplePayload(Payload):
        def __init__(self):
            super().__init__()
            self.data = None
        def to_pack_list(self):
            return [self.data]
        @classmethod
        def from_unpack_list(cls, lst):
            instance = cls()
            instance.data = lst[0]
            return instance

def xor_bytes(a, b):
    """Return the byte-wise XOR of two byte sequences of the same length."""
    return bytes(x ^ y for x, y in zip(a, b))

# --- PyIPv8Transport class using SimplePayload ---
class PyIPv8Transport:
    @staticmethod
    def encapsulate(packet, sender="Alice", receiver="Bob"):
        header = {
            "protocol": "ipv8",
            "timestamp": time.time(),
            "sender": sender,
            "receiver": receiver
        }
        payload = SimplePayload()
        payload.data = {"header": header, "data": packet}
        return payload

    @staticmethod
    def send(packet, sender="Alice", receiver="Bob"):
        payload = PyIPv8Transport.encapsulate(packet, sender, receiver)
        print("PyIPv8Transport: Sending payload:")
        print(payload)
        return payload

    @staticmethod
    def receive(payload):
        print("PyIPv8Transport: Received payload:")
        print(payload)
        data = payload.data.get("data")
        return data

# --- Other unchanged classes (NFCMock, ZKSNARK, CentralBank, OfflineWallet, Environment) ---
class NFCMock:
    @staticmethod
    def send(transaction_data):
        print("NFCMock: Transaction data sent.")
        return json.dumps(transaction_data)

    @staticmethod
    def receive(transaction_json):
        print("NFCMock: Transaction data received.")
        return json.loads(transaction_json)

class ZKSNARK:
    @staticmethod
    def generate_key():
        return "simulated_zk_key"

    @staticmethod
    def generate_proof(private_key, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        return hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()

    @staticmethod
    def verify_proof(proof, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        expected_proof = hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()
        return proof == expected_proof

class CentralBank:
    def __init__(self, name):
        self.name = name
        self.coins_issued = []
        self.key_pair = rsa.newkeys(2048)
        self.deposited_coins = set()
        self.used_nonces = set()
        self.logged_transactions = []

    def issue_coin(self, recipient, value):
        coin_id = hashlib.sha256(str(random.getrandbits(256)).encode()).hexdigest()
        signature = self.sign_coin(value, coin_id)
        signature_b64 = base64.b64encode(signature).decode('utf-8')
        coin = {
            "coin_id": coin_id,
            "value": value,
            "signature": signature_b64
        }
        self.coins_issued.append(coin)
        return coin

    def sign_coin(self, value, coin_id):
        message = f"{value}{coin_id}".encode()
        return rsa.sign(message, self.key_pair[1], 'SHA-256')

    def sign_blinded_hash(self, blinded_hash):
        return hashlib.sha256(f"{blinded_hash}{self.name}".encode()).hexdigest()

    def verify_and_deposit_coin(self, coin, sender_public_key, proof, zk_key, nonce):
        coin_hash = hashlib.sha256((str(coin['value']) + coin['coin_id'] + coin['signature']).encode()).hexdigest()
        if coin_hash in self.deposited_coins:
            return "Coin Already Defunded or Verification Failed"
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, self.key_pair[0])
        except rsa.VerificationError:
            return "Coin Already Defunded or Verification Failed"
        if nonce in self.used_nonces:
            return "Nonce already used. Double spending detected."
        self.used_nonces.add(nonce)
        amount = coin['value']
        expected_proof = hashlib.sha256(f"{sender_public_key}{amount}{zk_key}".encode()).hexdigest()
        if proof != expected_proof:
            return "Coin Already Defunded or Verification Failed"
        transaction_record = (sender_public_key.save_pkcs1(), nonce, proof)
        if transaction_record in self.logged_transactions:
           return "Double spending detected: Transaction already recorded."
        self.logged_transactions.append(transaction_record)
        self.deposited_coins.add(coin_hash)
        return "Coin Deposited Successfully"

class OfflineWallet:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.balance = 100
        self.key_pair = self.generate_key_pair()
        self.zk_key = ZKSNARK.generate_key()
        self.nonce = 0
        self.withdrawal_latencies = []
        self.deposit_latencies = []
        self.coins = []
        self.received_logs = []

    def log_transaction(self, sender_key, nonce, proof):
        self.received_logs.append((sender_key.save_pkcs1(), nonce, proof))

    def has_received_transaction(self, sender_key, nonce, proof):
        return (sender_key.save_pkcs1(), nonce, proof) in self.received_logs

    def generate_key_pair(self):
        return rsa.newkeys(2048)

    def store_coin(self, coin):
        self.coins.append(coin)
        print(f"Stored coin: {coin}")

    def prepare_transaction(self, recipient, amount):
        self.nonce += 1
        if not self.coins:
            raise ValueError("No coins available to send.")
        coin = self.coins[0]
        proof = ZKSNARK.generate_proof(self.key_pair[1], recipient.key_pair[0], amount, self.zk_key)
        return {
            "amount": amount,
            "coin": coin.copy(),
            "recipient_key": base64.b64encode(recipient.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "from_agent_key": base64.b64encode(self.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.nonce,
            "timestamp": time.time()
        }

    def blind_coin(self):
        f = hashlib.sha256(self.key_pair[0].save_pkcs1()).hexdigest()
        blinding_factor = random.randint(1, 10**5)
        blinded_hash = (int(f, 16) * blinding_factor) % (2**256)
        return {"hash": f, "blinded_hash": blinded_hash, "blinding_factor": blinding_factor}

class Environment:
    def __init__(self):
        self.measurement = {
            "transaction_id": [],
            "period": [],
            "to_agent": [],
            "from_agent": [],
            "settlement_type": [],
            "amount": [],
            "description": [],
            "latency": [],
            "zk_proof_generation_time": [],
            "zk_proof_verification_time": [],
            "transaction_verification_time": [],
            "load_computing": [],
            "gas_used": [], 
            "decryption_time": [],         # New field for decryption time
            "decryption_cpu_time": [],     # New field for decryption CPU load
            "status": []
        }

    def log_transaction(self, transaction_id, period, to_agent, from_agent, settlement_type, amount, description,
                        latency=None, zk_proof_gen_time=None, zk_proof_ver_time=None, transaction_ver_time=None,
                        load_computing=None, gas_used=None, decryption_time=None, decryption_cpu_time=None, status="Pending"):
        self.measurement["transaction_id"].append(transaction_id)
        self.measurement["period"].append(period)
        self.measurement["to_agent"].append(to_agent)
        self.measurement["from_agent"].append(from_agent)
        self.measurement["settlement_type"].append(settlement_type)
        self.measurement["amount"].append(amount)
        self.measurement["description"].append(description)
        self.measurement["latency"].append(latency)
        self.measurement["zk_proof_generation_time"].append(zk_proof_gen_time)
        self.measurement["zk_proof_verification_time"].append(zk_proof_ver_time)
        self.measurement["transaction_verification_time"].append(transaction_ver_time)
        self.measurement["load_computing"].append(load_computing)
        self.measurement["gas_used"].append(gas_used)
        self.measurement["decryption_time"].append(decryption_time)         # Record decryption time
        self.measurement["decryption_cpu_time"].append(decryption_cpu_time) # Record decryption CPU load
        self.measurement["status"].append(status)

    def generate_metrics(self):
        df = pd.DataFrame(self.measurement)
        total_transactions = len(df)
        total_latency = df["latency"].sum()
        overall_throughput = total_transactions / total_latency if total_latency > 0 else 0

        phase_throughput = {}
        for phase in df['settlement_type'].unique():
            phase_df = df[df['settlement_type'] == phase]
            phase_count = len(phase_df)
            phase_total_latency = phase_df["latency"].sum()
            phase_throughput[phase] = phase_count / phase_total_latency if phase_total_latency > 0 else 0
        df["throughput"] = df["settlement_type"].apply(lambda phase: phase_throughput.get(phase, 0))

        for col in ["zk_proof_generation_time", "zk_proof_verification_time", "load_computing", "throughput"]:
            df[col] = df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) and isinstance(x, (float, int)) else x)

        print("\n--- Simulation Metrics ---")
        print(f"Total Transactions: {total_transactions}")
        print(f"Total Latency (seconds): {total_latency:.6f}")
        print(f"Overall Throughput (transactions/sec): {overall_throughput:.6f}")

        print("\nThroughput by phase:")
        for phase, tp in phase_throughput.items():
            print(f"  {phase}: {tp:.6f} transactions/sec")
        df.to_csv("simulation_metricswithpad.csv", index=False)

# --- Integrating the "Bob pays Carla" scenario ---
class OfflinePaymentSimulation:
    def __init__(self):
        self.environment = Environment()
        self.alice_wallet = OfflineWallet("Alice")
        self.bob_wallet = OfflineWallet("Bob")
        self.carla_wallet = OfflineWallet("Carla")  # New wallet for Carla
        self.central_bank = CentralBank("Central Bank")
        self.used_nonces = set()

    def validate_coin_signature(self, coin, public_key):
        if not coin or "signature" not in coin:
            print("Invalid coin: Missing or no signature.")
            return False
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, public_key)
            print("Coin signature is valid.")
            return True
        except rsa.VerificationError:
            print("Invalid coin: Signature verification failed.")
            return False

    def verify_sender_legitimacy(self, sender_public_key, known_keys):
        return any(sender_public_key.save_pkcs1() == key.save_pkcs1() for key in known_keys)

    def verify_nonce(self, nonce, used_nonces):
        if nonce in used_nonces:
            print(f"Nonce {nonce} has already been used. Double spending detected.")
            return False
        used_nonces.add(nonce)
        return True

    def verify_recipient(self, transaction_data, expected_recipient_key):
        recipient_key = transaction_data.get("recipient_key")
        if not isinstance(recipient_key, rsa.PublicKey):
            print("Failure: recipient_key is not an rsa.PublicKey object.")
            return False
        return recipient_key.save_pkcs1() == expected_recipient_key.save_pkcs1()

    def simulate_deposit(self):
        """Simulates the deposit of a coin to the central bank."""
        if not self.bob_wallet.coins:
            print("Bob has no coin to deposit.")
            return

        coin = self.bob_wallet.coins[0]  # Select the first coin for deposit
        amount = coin['value']

        computing_start = time.process_time()
        start_time = time.time()

        # Generate ZK proof for deposit
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1], self.central_bank.key_pair[0], amount, self.bob_wallet.zk_key)

        # Prepare deposit data
        deposit_data = {
            "coin": coin.copy(),
            "sender_public_key": base64.b64encode(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.bob_wallet.nonce + 1,  # Increment nonce for the deposit
            "timestamp": time.time()
        }

        # Simulate the deposit process
        deposit_result = self.central_bank.verify_and_deposit_coin(
            coin,
            self.bob_wallet.key_pair[0],
            proof,
            self.bob_wallet.zk_key,
            deposit_data["nonce"]  # Use the incremented nonce for deposit
        )

        if deposit_result == "Coin Deposited Successfully":
            self.bob_wallet.coins.pop(0)  # Remove deposited coin from wallet
            self.bob_wallet.balance -= amount  # Update wallet balance
            self.bob_wallet.nonce += 1  # Increment wallet's nonce after successful deposit
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            print(f"Deposit completed successfully. Total time: {latency:.6f}s, "
                  f"CPU load: {computing_load:.6f}s.")

            self.environment.log_transaction(
                transaction_id=len(self.environment.measurement["transaction_id"]),
                period=time.time(),
                to_agent="Central Bank",
                from_agent=self.bob_wallet.owner_id,
                settlement_type="deposit",
                amount=amount,
                description="Coin Deposit",
                latency=latency,
                zk_proof_gen_time=None,  # Add timing if needed
                zk_proof_ver_time=None,  # Add timing if needed
                transaction_ver_time=None,  # Add timing if needed
                load_computing=computing_load,
                status="Success"
            )
        else:
            print(f"Deposit failed: {deposit_result}")

    # --- Withdrawal method ---
    def simulate_withdrawal(self, amount):
        print(f"Alice withdraws {amount} units...")
        computing_start = time.process_time()
        total_start_time = time.time()
        blinded_coin = self.alice_wallet.blind_coin()
        interaction_start_time = time.process_time()
        blinded_signature = self.central_bank.sign_blinded_hash(blinded_coin["blinded_hash"])
        interaction_cpu_time = time.process_time() - interaction_start_time
        message = f"{amount}{self.alice_wallet.owner_id}".encode()
        unblinded_signature = rsa.sign(message, self.central_bank.key_pair[1], 'SHA-256')
        coin = self.central_bank.issue_coin(self.alice_wallet.owner_id, amount)
        self.alice_wallet.store_coin(coin)
        total_latency = time.time() - total_start_time
        computing_load = time.process_time() - computing_start
        print(f"Withdrawal completed successfully. Total time: {total_latency:.6f}s, "
              f"Interaction (CPU) time: {interaction_cpu_time:.6f}s, "
              f"CPU load: {computing_load:.6f}s.")
        self.environment.log_transaction(
            transaction_id=len(self.environment.measurement["transaction_id"]),
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Central Bank",
            settlement_type="withdrawal",
            amount=amount,
            description="Withdrawal with blinding",
            latency=total_latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=interaction_cpu_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Offline payment method (Alice pays Bob) ---
    def simulate_offline_transaction(self, amount=None):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        if amount is None:
            amount = random.randint(10, 50)
        if not self.alice_wallet.coins:
            print("No coin available for an offline transaction.")
            return
        coin = self.alice_wallet.coins[0]
        if amount > coin["value"]:
            print(f"Requested amount ({amount}) greater than coin value ({coin['value']}). Using coin value.")
            amount = coin["value"]
        elif amount < coin["value"]:
            print("Partial transaction not supported. Using the full coin value.")
            amount = coin["value"]
        computing_start = time.process_time()
        start_time = time.time()
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.bob_wallet.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start_time
        transaction_data = self.alice_wallet.prepare_transaction(self.bob_wallet, amount)
        transaction_data.update({
            "transaction_id": transaction_id,
            "from_agent": self.alice_wallet.owner_id,
            "to_agent": self.bob_wallet.owner_id,
            "nonce": transaction_data["nonce"]
        })
        json_data = json.dumps(transaction_data).encode('utf-8')
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_data)
        packet = {
            "encrypted_aes_key": base64.b64encode(
                PKCS1_OAEP.new(
                    RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM'))
                ).encrypt(aes_key)
            ).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Sending the encapsulated transaction via pyipv8...")
        ipv8_payload = PyIPv8Transport.send(packet, sender=self.alice_wallet.owner_id, receiver=self.bob_wallet.owner_id)
        received_packet = PyIPv8Transport.receive(ipv8_payload)
        try:
            enc_aes_key = base64.b64decode(received_packet["encrypted_aes_key"])
            ciphertext = base64.b64decode(received_packet["ciphertext"])
            tag = base64.b64decode(received_packet["tag"])
            nonce = base64.b64decode(received_packet["nonce"])
            bob_private_pem = self.bob_wallet.key_pair[1].save_pkcs1(format='PEM')
            bob_private_key_py = RSA.importKey(bob_private_pem)
            cipher_rsa_bob = PKCS1_OAEP.new(bob_private_key_py)
            decrypted_aes_key = cipher_rsa_bob.decrypt(enc_aes_key)
            cipher_aes_bob = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=nonce)
            decrypted_data = cipher_aes_bob.decrypt_and_verify(ciphertext, tag)
            received_transaction = json.loads(decrypted_data.decode('utf-8'))
            if isinstance(received_transaction["from_agent_key"], str):
                try:
                    sender_public_key = rsa.PublicKey.load_pkcs1(base64.b64decode(received_transaction["from_agent_key"]))
                    received_transaction["from_agent_key"] = sender_public_key
                    print("Issuer's public key correctly loaded.")
                except Exception as e:
                    print(f"Error decoding issuer key: {e}")
                    return
            if isinstance(received_transaction["recipient_key"], str):
                try:
                    received_transaction["recipient_key"] = rsa.PublicKey.load_pkcs1(
                        base64.b64decode(received_transaction["recipient_key"])
                    )
                    print("recipient_key correctly loaded into PublicKey.")
                except Exception as e:
                    print(f"Error decoding recipient_key: {e}")
                    return
            if not isinstance(received_transaction["recipient_key"], rsa.PublicKey):
                print("Error: recipient_key is not a valid rsa.PublicKey after processing.")
                return
            print(f"Received recipient_key: {received_transaction['recipient_key']}")
            print(f"Type of recipient_key: {type(received_transaction['recipient_key'])}")
            verification_start_time = time.time()
            if not self.validate_coin_signature(received_transaction.get("coin"), self.central_bank.key_pair[0]):
                print("Failed to verify coin signature.")
                return
            sender_public_key = received_transaction.get("from_agent_key")
            if not self.verify_sender_legitimacy(sender_public_key, [self.alice_wallet.key_pair[0]]):
                print("Failed to verify sender legitimacy.")
                return
            else:
                print("Sender legitimacy verification successful: The sender is legitimate.")
            proof = received_transaction.get("proof")
            nonce_val = received_transaction.get("nonce")
            if self.bob_wallet.has_received_transaction(sender_public_key, nonce_val, proof):
                print("Double spending detected: transaction already received.")
                return
            else:
                self.bob_wallet.log_transaction(sender_public_key, nonce_val, proof)
                print("Transaction successfully logged.")
            if not self.verify_nonce(received_transaction.get("nonce"), self.used_nonces):
                print("Failed to verify nonce.")
                return
            zk_proof_ver_start_time = time.time()
            proof_valid = ZKSNARK.verify_proof(
                proof,
                self.bob_wallet.key_pair[0],
                amount,
                self.alice_wallet.zk_key
            )
            zk_proof_ver_time = time.time() - zk_proof_ver_start_time
            if not proof_valid:
                print("Failed to verify zk-SNARK proof.")
                return
            else:
                print("zk-SNARK proof verification successful: The proof is valid.")
            print("Nonce is verified and unique, and the zk-SNARK proof is verified and unique.")
            verification_time = time.time() - verification_start_time
            self.alice_wallet.coins.pop(0)
            received_transaction["coin"]["nonce"] = received_transaction["nonce"]
            received_transaction["coin"]["recipient"] = self.bob_wallet.owner_id
            self.bob_wallet.store_coin(received_transaction["coin"])
            self.alice_wallet.balance -= amount
            self.bob_wallet.balance += amount
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            self.environment.log_transaction(
                transaction_id=transaction_id,
                period=time.time(),
                to_agent=self.bob_wallet.owner_id,
                from_agent=self.alice_wallet.owner_id,
                settlement_type="offline_transaction",
                amount=amount,
                description="Offline Transaction",
                latency=latency,
                zk_proof_gen_time=zk_proof_gen_time,
                zk_proof_ver_time=zk_proof_ver_time,
                transaction_ver_time=verification_time,
                load_computing=computing_load,
                status="Success"
            )
        except Exception as e:
            print(f"Error during transaction: {e}")

    # --- New method: Bob pays Carla scenario ---
    def simulate_bob_pays_carla(self, amount):
        # Define the transaction ID as current number of transactions + 1
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        # Start timing (wall time and CPU time)
        computing_start = time.process_time()
        start_time = time.time()

        print(f"\n=== Scenario: Bob pays Carla for {amount} units ===")
        # Step 1: Bob and Carla agree on a contract
        contract = f"Contract_Bob_pays_Carla_{amount}"
        print("Bob and Carla agree on the contract:", contract)

        # Step 2: Bob selects the RIO certificate (dummy)
        rio_cert = "RIO_pk_cert_dummy"
        print("Bob selects the RIO certificate:", rio_cert)

        # Step 3: Carla provides Bob with one of her pseudonymous identities
        carla_pseudonym = self.carla_wallet.key_pair[0].save_pkcs1().decode('utf-8')
        print("Carla provides Bob with her pseudonymous identifier.")

        # Step 4: Bob selects an e-coin matching the desired amount
        if not self.bob_wallet.coins:
            print("Bob has no coin available for payment.")
            return
        ecoin = self.bob_wallet.coins[0]
        print("Selected e-coin:", ecoin)

        # Step 5: Bob signs the contract with his e-coin
        signature_contract = rsa.sign(contract.encode('utf-8'), self.bob_wallet.key_pair[1], 'SHA-256')
        signature_contract_b64 = base64.b64encode(signature_contract).decode('utf-8')
        print("Bob signs the contract:", signature_contract_b64)

        # Step 6: Bob generates the zk-SNARK payment proof
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1],
                                       self.carla_wallet.key_pair[0],
                                       amount,
                                       self.bob_wallet.zk_key)
        print("Generated proof:", proof)
        # Measure proof verification time
        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(
                    proof,
                    self.carla_wallet.key_pair[0],
                    amount,
                    self.bob_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start
        if not proof_valid:
            print("Failed to verify zk-SNARK proof.")
            return
        else:
            print("zk-SNARK proof verification successful: The proof is valid.")

        # Step 7: Bob builds the transaction packet including:
        transaction_payload = {
            "contract": contract,
            "contract_signature": signature_contract_b64,
            "proof": proof,
            "RIO_cert": rio_cert,
            "amount": amount,
            "bob_pseudonym": self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8'),
            "carla_pseudonym": carla_pseudonym,
            "timestamp": time.time()
        }
        json_payload = json.dumps(transaction_payload).encode('utf-8')

        # Step 8: Encrypt the transaction packet (RIO simulator)
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_payload)

        # Generate a key pair for RIO
        rio_key_pair = rsa.newkeys(2048)
        rio_public = rio_key_pair[0]
        rio_private = rio_key_pair[1]

        # Encrypt the AES key with RIO's public key
        rio_public_pem = rio_public.save_pkcs1(format='PEM')
        cipher_rsa_enc = PKCS1_OAEP.new(RSA.importKey(rio_public_pem))
        encrypted_aes_key = cipher_rsa_enc.encrypt(aes_key)

        encrypted_packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Bob→Carla transaction (encrypted for RIO):", encrypted_packet)

        # --- Simulate RIO verification phase ---
        # To decrypt, RIO uses its private key
        rio_private_pem = rio_private.save_pkcs1(format='PEM')
        cipher_rsa_dec = PKCS1_OAEP.new(RSA.importKey(rio_private_pem))
        # Decrypt the AES key
        decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)

        # Use the decrypted key to decrypt the payload
        cipher_aes_rio = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=cipher_aes.nonce)
        decrypted_payload = cipher_aes_rio.decrypt_and_verify(ciphertext, tag)
        # Here, RIO verifies the payload data (contract, signature, proof, etc.)
        # Initialize verification_start_time here within simulate_bob_pays_carla
        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time: {transaction_verification_time:.6f} seconds")

        # ----------------------------
        # TOTAL LATENCY AND CPU LOAD
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start
        # ----------------------------
        # RECORD METRICS FOR "BOB PAYS CARLA"
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.carla_wallet.owner_id,
            from_agent=self.bob_wallet.owner_id,
            settlement_type="online_transaction",  # or "bob_to_carla"
            amount=amount,
            description="Payment from Bob to Carla",
            latency=latency,
            zk_proof_gen_time=None,  # Add proof generation time if needed
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=computing_load,
            status="Success"
        )

    # ----------------------------
    def simulate_online_deposit_PAD(self, amount):
        """
        Online Deposit/Transfer via PAD scenario.
        This simulates an online deposit (or transfer to one's own account)
        via the PAD system managed by the RIO.
        """
        print(f"\n=== Online Deposit via PAD scenario for {amount} units ===")
        # Define timing variables and transaction ID
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()

        # -- Step 0: Transaction Verification (similar to offline verification) --
        # RIO checks that the transaction is not a double spend.
        # For simulation, reuse the zk-SNARK proof verification.
        # Assume Alice uses one of her coins for the online deposit.
        if not self.alice_wallet.coins:
            print("No coin available for an online deposit.")
            return
        coin = self.alice_wallet.coins[0]
        # Generate and verify the proof (similar to the other scenario)
        zk_proof_start = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.central_bank.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start

        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(proof,
                                           self.central_bank.key_pair[0],
                                           amount,
                                           self.alice_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start

        if not proof_valid:
            print("Failed to verify zk-SNARK proof during online transaction.")
            return
        print("zk-SNARK proof verification succeeded.")

        # -- Step 1: Securing the blinded coin F_s --
        # Assume Alice's e-coin is used as the blinded coin F_s.
        Fs = self.alice_wallet.blind_coin()  # F_s
        # Generate a token T (for simulation, a random number as string)
        T = str(random.randint(1000, 9999))
        print("Generated token T:", T)
        # Save T so it's available for the decryption step
        self.last_token_value = T

        # Simulate the verification key K_verif (e.g., 16 bytes)
        K_verif = get_random_bytes(16)
        print("Generated verification key K_verif:", base64.b64encode(K_verif).decode())
        self.last_K_verif = K_verif

        # Generate a symmetric key K_sym
        K_sym = get_random_bytes(32)
        # Encrypt (T, F_s) with K_sym (concatenate T and a JSON of Fs)
        data_to_encrypt = T.encode('utf-8') + json.dumps(Fs, sort_keys=True).encode('utf-8')
        cipher_pad = AES.new(K_sym, AES.MODE_GCM)
        C, tag = cipher_pad.encrypt_and_digest(data_to_encrypt)
        nonce_pad = cipher_pad.nonce
        # C is the encrypted content containing the token and F_s

        # Now, sign and then encrypt the content with Alice's private key, then with the recipient's public key
        # (here, for online deposit we assume it's to Alice's own account)
        # For simulation, we sign C with Alice's private key:
        signature_C = rsa.sign(C, self.alice_wallet.key_pair[1], 'SHA-256')
        signature_C_b64 = base64.b64encode(signature_C).decode('utf-8')
        # Form C' that combines the content C and its signature
        C_prime = {
            "encrypted_content": base64.b64encode(C).decode('utf-8'),
            "signature": signature_C_b64
        }
        # Then encrypt C' with the recipient's public key (Alice's public key here).
        # --- Hybrid encryption of C_prime ---
        # Convert C_prime to JSON (data to encrypt)
        C_prime_json = json.dumps(C_prime).encode('utf-8')

        # 1. Generate a symmetric key K_PAD to encrypt C_prime_json with AES
        K_PAD = get_random_bytes(32)
        cipher_pad2 = AES.new(K_PAD, AES.MODE_GCM)
        C_prime_ciphertext, C_prime_tag = cipher_pad2.encrypt_and_digest(C_prime_json)
        nonce_pad2 = cipher_pad2.nonce

        # 2. Encrypt K_PAD with Alice's public key (online deposit to own account)
        cipher_rsa_for_PAD = PKCS1_OAEP.new(RSA.importKey(self.alice_wallet.key_pair[0].save_pkcs1(format='PEM')))
        encrypted_K_PAD = cipher_rsa_for_PAD.encrypt(K_PAD)

        # 3. Build the final packet for C_prime by grouping the encrypted key and the ciphertext
        C_prime_encrypted = {
            "encrypted_key": base64.b64encode(encrypted_K_PAD).decode('utf-8'),
            "ciphertext": base64.b64encode(C_prime_ciphertext).decode('utf-8'),
            "tag": base64.b64encode(C_prime_tag).decode('utf-8'),
            "nonce": base64.b64encode(nonce_pad2).decode('utf-8')
        }

        # Mask the symmetric key with a mask R_sym
        R_sym = get_random_bytes(32)
        S_m = xor_bytes(K_sym, R_sym)
        # Dummy share for trustees (N_t, e.g., 3)
        N_t = b"3"
        S_t = hashlib.sha256(S_m + N_t).hexdigest()
        # Dummy share for validators (N_v, e.g., 2)
        N_v = b"2"
        V = hashlib.sha256(b"R" + self.alice_wallet.key_pair[0].save_pkcs1() + N_v).hexdigest()

        # Encrypted dummy shares for trustees and validators (simulated)
        E_t = "encrypted_trustee_shares_dummy"
        E_v = "encrypted_validator_shares_dummy"

        # Final data transmission structure D_ts
        D_ts = {
            "E_t": E_t,
            "E_v": E_v,
            "H_T": hashlib.sha256(T.encode('utf-8')).hexdigest(),
            "C_prime_encrypted": json.dumps(C_prime_encrypted)
        }

        print("PAD data generated (D_ts):", D_ts)

        # Assume RIO receives this data, verifies the proof (done already),
        # then proceeds with the online deposit.
        # Here we simulate the reception and approval by RIO.
        verification_start_time = time.time()
        # (Real verification operations would be performed here)
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time (RIO): {transaction_verification_time:.6f} seconds")

        # Update balances and remove Alice's coin
        self.alice_wallet.coins.pop(0)
        self.alice_wallet.balance -= amount
        print(f"Online deposit (via PAD) of {amount} units completed successfully for {self.alice_wallet.owner_id}.")

        # Post the transaction on the PAD ledger via the smart contract
        sender_address = "0x4da60efbbf6a9aa9846e749c0c7b52656a484636"  # User address (hex)
        recipient_address = sender_address  # Deposit to own online account
        tx_type = "online_deposit_PAD"
        details = "Online Deposit via PAD with Smart Contract"

        # Private key corresponding to sender_address (hex)
        private_key = "0x584beec8f497a30c0430679175fd36f49a5d97bfa8fd42242d423e5696397d29"

        receipt = pad_interface.post_transaction(
            transaction_id,
            sender_address,
            recipient_address,
            amount,
            tx_type,
            details,
            private_key
        )

        # Record metrics using receipt["gasUsed"] or latency, etc.
        gas_used = receipt.get("gasUsed", None)
        print("Gas used for PAD transaction:", gas_used)
        
        # Compute total latency and CPU load
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start

        # Log metrics for this PAD scenario
        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,  # deposit to own account
            from_agent=self.alice_wallet.owner_id,
            settlement_type="online_deposit_PAD",
            amount=amount,
            description="Online Deposit via PAD",
            latency=latency,
            zk_proof_gen_time=zk_proof_gen_time,
            zk_proof_ver_time=zk_proof_ver_time,
            transaction_ver_time=transaction_verification_time,
            load_computing=computing_load,
            gas_used=gas_used,  # record gas used
            status="Success"
        )

    # --- Multiple methods for each transaction type ---
    def simulate_multiple_offline_transactions(self, nb_transactions, offline_amount=None):
        nb_possible = len(self.alice_wallet.coins)
        nb_to_execute = min(nb_transactions, nb_possible)
        print(f"Requested number of offline transactions: {nb_transactions}.")
        print(f"Number of coins available in wallet: {nb_possible}.")
        print(f"Executing {nb_to_execute} offline transaction(s).")
        for i in range(nb_to_execute):
            self.simulate_offline_transaction(amount=offline_amount)

    def simulate_multiple_withdrawals(self, nb_withdrawals, withdrawal_amount):
        print(f"Requested number of withdrawals: {nb_withdrawals}.")
        for i in range(nb_withdrawals):
            self.simulate_withdrawal(withdrawal_amount)

    def simulate_multiple_deposits(self, nb_deposits):
        nb_possible = len(self.bob_wallet.coins)
        nb_to_execute = min(nb_deposits, nb_possible)
        print(f"Requested number of deposits: {nb_deposits}.")
        print(f"Number of coins available in Bob's wallet: {nb_possible}.")
        for i in range(nb_to_execute):
            self.simulate_deposit()

    # ... (other previously defined methods)

    def simulate_decryption_request(self, T, threshold, K_verif):
        """
        Simulate the decryption process in the PAD protocol:
         - If T > theta, RIO creates a decryption request and computes φ = G(H(T), K_{Bpub})
         - Then Bob retrieves and decrypts the secret:
              R_sf ← G(E^{-1}(E^{-1}(C', K_{Bpriv}), K_{verif}))
        """
        print("\n=== Start of decryption request process (PIP) ===")
        # Step 2: Threshold check and creation of the decryption request
        if int(T) > threshold:
            # Compute H(T)
            H_T = hashlib.sha256(T.encode()).hexdigest()
            # Retrieve Bob's public key (as string)
            bob_pub = self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8')
            # Compute φ = G(H(T), K_{Bpub}) (simulated by hashing the concatenation)
            phi = hashlib.sha256((H_T + bob_pub).encode()).hexdigest()
            print("Decryption request published with φ =", phi)
        else:
            print("Token T (", T, ") below threshold (", threshold, "). No decryption request.")
            return

        # Step 3: Bob retrieves and decrypts the secret
        # For simulation, define a simulated secret:
        secret_data = "my_very_confidential_secret"
        print("Original secret (simulated):", secret_data)
        
        # --- Simulate double encryption (C') of the secret ---
        # First encryption: encrypt secret with verification key K_verif (AES)
        cipher1 = AES.new(K_verif, AES.MODE_GCM)
        secret_ciphertext, secret_tag = cipher1.encrypt_and_digest(secret_data.encode())
        nonce_secret = cipher1.nonce

        # Second encryption: encrypt the result (secret_ciphertext) with Bob's public key via RSA
        bob_pub_key = RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1())
        cipher_rsa_enc = PKCS1_OAEP.new(bob_pub_key)

        # Note: in practice, RSA only encrypts very small messages. Here, it's a simulation.
        C_prime = cipher_rsa_enc.encrypt(secret_ciphertext)
        print("C_prime (simulated double encryption) =", base64.b64encode(C_prime).decode())
        # Measure Bob's decryption time
        decryption_start_time = time.time()
        cpu_start = time.process_time()
        # Bob retrieves the secret:
        # First step: decrypt C_prime with his private key (RSA)
        bob_priv = RSA.importKey(self.bob_wallet.key_pair[1].save_pkcs1())
        cipher_rsa_dec = PKCS1_OAEP.new(bob_priv)

        decrypted_ciphertext = cipher_rsa_dec.decrypt(C_prime)
        # Second step: decrypt the result with K_verif (AES)
        cipher2 = AES.new(K_verif, AES.MODE_GCM, nonce=nonce_secret)
        recovered_secret = cipher2.decrypt_and_verify(decrypted_ciphertext, secret_tag).decode()
       
        cpu_end = time.process_time()
        decryption_end_time = time.time()
        decryption_time = decryption_end_time - decryption_start_time
        decryption_cpu_time = cpu_end - cpu_start
    
        # Finally, apply a function G to the recovered secret, here a SHA-256 hash
        R_sf = hashlib.sha256(recovered_secret.encode()).hexdigest()
        print("Bob has recovered the secret R_sf =", R_sf)

        print("=== End of decryption request process ===")
        # Log decryption metrics to CSV (adapt how you use them as needed)
        self.environment.log_transaction(
            transaction_id="decryption_request",  # or another appropriate identifier
            period=time.time(),
            to_agent=self.bob_wallet.owner_id,
            from_agent="PAD_System",
            settlement_type="decryption_request",
            amount=0,
            description="PAD secret decryption",
            latency=decryption_time,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=decryption_cpu_time,
            gas_used=None,
            status="Success"
         )

class PADLedgerInterface:
    def __init__(self, provider_url, contract_address, abi_file_path, chain_id=1337, gas=2000000, gas_price_gwei="50"):
        # Connect to Ganache
        self.web3 = Web3(Web3.HTTPProvider(provider_url))
        if not self.web3.is_connected():
            raise Exception("Unable to connect to Ganache.")
        # Convert address to checksum address
        self.contract_address = self.web3.to_checksum_address(contract_address)
        # Load ABI from JSON file
        with open(abi_file_path, "r") as abi_file:
            self.contract_abi = json.load(abi_file)
        self.contract = self.web3.eth.contract(address=self.contract_address, abi=self.contract_abi)
        self.chain_id = chain_id
        self.gas = gas
        self.gas_price = self.web3.to_wei(gas_price_gwei, "gwei")

    def post_transaction(self, transaction_id, sender, recipient, amount, tx_type, details, private_key):
        # Convert sender and recipient to checksum addresses
        sender = self.web3.to_checksum_address(sender)
        recipient = self.web3.to_checksum_address(recipient)
    
        # Build the transaction
        tx = self.contract.functions.postTransaction(
            transaction_id,  # transactionId
            sender,          # from
            recipient,       # to
            amount,          # amount
            tx_type,         # txType
            details          # details (e.g., "Online Deposit via PAD")
        ).build_transaction({
            "chainId": self.chain_id,
            "gas": self.gas,
            "gasPrice": self.gas_price,
            "nonce": self.web3.eth.get_transaction_count(sender),
        })

        # Sign and send the transaction
        signed_tx = self.web3.eth.account.sign_transaction(tx, private_key=private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_tx.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print("Transaction posted on PAD ledger:", receipt)
        
        return receipt

class ShardLedgerInterface:
    def __init__(self, provider_url="http://127.0.0.1:8545", contract_address="0x6ac672b37c0e0fa3d211bf94565e33f2eb391298", abi_file_path="shardledger_ABI.json", chain_id=1337, gas=2000000, gas_price_gwei="50"):
        # Connect to Ganache (Shard Ledger)
        self.web3 = Web3(Web3.HTTPProvider(provider_url))
        if not self.web3.is_connected():
            raise Exception("Unable to connect to Ganache (Shard Ledger).")
        self.contract_address = self.web3.to_checksum_address(contract_address)
        with open(abi_file_path, "r") as abi_file:
            self.contract_abi = json.load(abi_file)
        self.contract = self.web3.eth.contract(address=self.contract_address, abi=self.contract_abi)
        self.chain_id = chain_id
        self.gas = gas
        self.gas_price = self.web3.to_wei(gas_price_gwei, "gwei")

    def update_user_balance(self, current_balance_enc, T, op, user_pseudonym):
        # Build input data according to your algorithm
        # For example, compute c_T = Enc(T, R_pk) and perform homomorphic operation.
        tx = self.contract.functions.updateBalance(
            user_pseudonym,  # or the address associated with the pseudonym
            T,               # Amount or token
            op               # "INC" or "DEC"
        ).build_transaction({
            "chainId": self.chain_id,
            "gas": self.gas,
            "gasPrice": self.gas_price,
            "nonce": self.web3.eth.get_transaction_count(user_pseudonym),
        })
        # Sign and send the transaction (private key must be provided)
        # ...
        # Return the tx dict and/or the receipt after sending
        return tx  # (or the receipt after submission)

# --- Digital Identity management ---
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode('utf-8')).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode('utf-8')).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode('utf-8')).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        digital_identity = {
            "user_id": user_id,
            "credential": credential,
            "pid": pid
        }
        return digital_identity

    def verify_identity(self, digital_identity):
        cred = digital_identity["credential"].copy()
        signature = cred.pop("signature", None)
        cred_str = json.dumps(cred, sort_keys=True)
        expected_signature = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return expected_signature == signature

# --- Performance measurement for digital onboarding ---
class IdentityMetrics:
    def __init__(self):
        self.onboarding_times = []
        self.verification_times = []

    def record_onboarding_time(self, t):
        self.onboarding_times.append(t)

    def record_verification_time(self, t):
        self.verification_times.append(t)

    def average_onboarding_time(self):
        if not self.onboarding_times:
            return 0
        return sum(self.onboarding_times) / len(self.onboarding_times)

    def average_verification_time(self):
        if not self.verification_times:
            return 0
        return sum(self.verification_times) / len(self.verification_times)

# Create instances for identity management
id_manager = DigitalIdentityManager()
id_metrics = IdentityMetrics()

# --- Digital onboarding phase for multiple users with performance measurement ---
print("\n=== Digital Onboarding Phase for Multiple Users ===")
try:
    nb_users = int(input("Enter the number of users to onboard: "))
except ValueError:
    print("Invalid number, onboarding 1 by default.")
    nb_users = 1

digital_identities = []
for i in range(1, nb_users + 1):
    user_id = f"User_{i}"
    user_attributes = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
    start_onboarding = time.time()
    digital_identity = id_manager.onboard_user(user_id, user_attributes)
    end_onboarding = time.time()
    onboarding_time = end_onboarding - start_onboarding
    id_metrics.record_onboarding_time(onboarding_time)

    start_verif = time.time()
    verification_ok = id_manager.verify_identity(digital_identity)
    end_verif = time.time()
    verification_time = end_verif - start_verif
    id_metrics.record_verification_time(verification_time)

    digital_identities.append(digital_identity)
    print(f"Digital Identity for {user_id}:", digital_identity)
    print(f"Onboarding time for {user_id}: {onboarding_time:.6f} seconds")
    if verification_ok:
        print(f"Identity verification succeeded for {user_id} in {verification_time:.6f} seconds.")
    else:
        print(f"Identity verification failed for {user_id} in {verification_time:.6f} seconds.")

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time:", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time:", id_metrics.average_verification_time(), "seconds")

# Export metrics to CSV
import csv
with open("identity_metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["User_ID", "Onboarding_Time", "Verification_Time"])
    for i, identity in enumerate(digital_identities, 1):
        writer.writerow([f"User_{i}", id_metrics.onboarding_times[i-1], id_metrics.verification_times[i-1]])

# --- Transaction simulations ---
simulation = OfflinePaymentSimulation()
simulation.simulate_multiple_withdrawals(4, 50)
try:
    nb_offline = int(input("Enter the number of offline transactions to execute: "))
except ValueError:
    print("Invalid number, executing 1 offline transaction by default.")
    nb_offline = 1
try:
    offline_amount = int(input("Enter the amount for each offline transaction (units): "))
except ValueError:
    print("Invalid amount, using a random amount.")
    offline_amount = None
simulation.simulate_multiple_offline_transactions(nb_offline, offline_amount)
try:
    nb_deposits = int(input("Enter the number of deposits to execute: "))
except ValueError:
    print("Invalid number, executing 1 deposit by default.")
    nb_deposits = 1
simulation.simulate_multiple_deposits(nb_deposits)

# --- New scenario: Bob pays Carla ---
try:
    bob_to_carla_amount = int(input("Enter the amount for Bob→Carla payment: "))
except ValueError:
    print("Invalid amount, using 50 units by default.")
    bob_to_carla_amount = 50
simulation.simulate_bob_pays_carla(bob_to_carla_amount)

simulation.environment.generate_metrics()

# Example: PAD ledger configuration
provider_url = "http://127.0.0.1:8545"
contract_address = "0x6ac672b37c0e0fa3d211bf94565e33f2eb391298"  # Replace with the address from Remix
abi_file_path = "PADLedger_ABI.json"
# Create the interface to communicate with the PAD contract
pad_interface = PADLedgerInterface(provider_url, contract_address, abi_file_path)

# --- New scenario: Online Deposit via PAD ---
try:
    online_deposit_amount = int(input("Enter the amount for Online Deposit via PAD (for Alice): "))
except ValueError:
    print("Invalid amount, using 50 units by default.")
    online_deposit_amount = 50
simulation.simulate_online_deposit_PAD(online_deposit_amount)

# Call the new method to simulate the decryption request and secret recovery
# Assume the threshold theta is 500
theta = 500
# Call the decryption request using the saved T and K_verif values
simulation.simulate_decryption_request(simulation.last_token_value, theta, simulation.last_K_verif)

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time:", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time:", id_metrics.average_verification_time(), "seconds")


# CBDC Offline/Online Payment Simulator — README

## Overview

This simulator models a minimal CBDC ecosystem with:

- **Withdrawals** (blind-signature–style placeholder),
- **Offline payments** (Alice → Bob) using simulated zk-SNARK proofs, AES/RSA hybrid encryption, and a lightweight pyipv8-like transport wrapper,
- **Online deposits** (Alice → RIO/PAD) with secure token packaging,
- **Bank deposits** (Bob → Central Bank) with basic anti–double-spending checks,
- **Bob → Carla scenario** (contract, zk proof, encrypted packet via RIO),
- **Decryption request** (PAD-style: threshold check, double encryption/decryption, recovery of secret `R_sf`),
- **Funding/Defunding operations** (fiat-to-CBDC balance updates on the Shard Ledger via smart contracts),
- **Identity onboarding & verification** (toy credentials + metrics),
- **Metrics export** (latency, CPU load, per-phase throughput → CSV).

> All crypto proofs/transport are **simulated** for research and demo purposes—**not** production security.

---

## Main Components

- **`CentralBank`**: issues coins, signs coins, verifies deposits, tracks used nonces & recorded transactions.
- **`OfflineWallet`**: holds keys, coins, and prepares transactions (Alice, Bob, Carla).
- **`ZKSNARK` (simulated)**: generates/verifies a hash-based stand-in proof.
- **`PyIPv8Transport`**: tiny wrapper emulating message encapsulation/logging.
- **`NFCMock`**: JSON send/receive shim.
- **`Environment`**: records transactions + metrics; exports CSV.
- **`OfflinePaymentSimulation`**: orchestrates workflows (withdrawals, offline payments, online PAD deposit, Bob→Carla scenario, PAD decryption request, funding/defunding).
- **`PADLedgerInterface`**: smart contract wrapper (Ethereum/Ganache) to post transactions on-chain.
- **`ShardLedgerInterface`**: smart contract wrapper for **Funding/Defunding** operations (updates encrypted user balances with `INC`/`DEC`).
- **Identity Stack**: `AttributeProvider`, `IdentityProvider`, `PIDProvider`, `DigitalIdentityManager` + `IdentityMetrics`.

---

## Key Flows

1. **Withdrawal (CentralBank → Alice)**  
   Alice blinds (placeholder), bank signs, coin issued & stored in wallet.

2. **Offline Payment (Alice → Bob)**  
   Alice builds transaction + simulated zk proof.  
   AES payload, RSA-encrypted AES key, sent via `PyIPv8Transport`.  
   Bob decrypts, validates, checks nonce & logs to prevent replays.

3. **Bank Deposit (Bob → CentralBank)**  
   Proof verification + signature check + nonce tracking to avoid double spending.

4. **Online Deposit via PAD (Alice → own account)**  
   Token `T` + blinded coin packaged; hybrid encryption and logging.

5. **Bob Pays Carla (Scenario)**  
   Contract string, proof generation, encrypted packet “to RIO”, timing recorded.

6. **PAD Decryption Request**  
   If token `T` > threshold, RIO publishes a decryption request with φ = G(H(T), K_pub).  
   Bob recovers the secret by double decryption (RSA + AES).  
   Final recovered secret `R_sf` is hashed and logged.  

7. **Funding/Defunding (New)**  
   - **Funding**: Simulates transfer from a **fiat account → CBDC wallet** by creating a claim check (λ_A), re-randomizing it (λ'_A), and updating balance on the **Shard Ledger** smart contract with an `INC` operation.  
   - **Defunding**: Reverse process — CBDC balance is decreased on the Shard Ledger with a `DEC` operation, simulating conversion back to fiat.  
   Both record **latency, CPU load, and gas usage** in metrics.

8. **Digital Identity**  
   Issue “credentials”, compute simple signatures, verify, track onboarding/verification times.

---

## Metrics & Outputs

- **Console logs**: each step’s status + timings.
- **CSV files**:
  - `simulation_metricsfunding.csv` — per-transaction metrics including funding/defunding:
    ```
    transaction_id, period, to_agent, from_agent, settlement_type, amount, description, latency, zk_proof_generation_time, zk_proof_verification_time, transaction_verification_time, load_computing, gas_used, decryption_time, decryption_cpu_time, status, throughput
    ```
  - `identity_metrics.csv` — per-user onboarding/verification times.

Throughput is computed per settlement type and overall.

---



In [ ]:

#!pip install pycryptodome 
#!pip install pyipv8
# cbdc_simulator.py
import time
import random
import hashlib
import rsa
import pandas as pd
from Crypto.Cipher import AES, PKCS1_OAEP
from Crypto.PublicKey import RSA
from Crypto.Random import get_random_bytes
import json
import base64
import os
from web3 import Web3

# --- pyipv8 handling and definition of a concrete SimplePayload class ---
try:
    from ipv8.messaging.payload import Payload
except ImportError:
    print("pyipv8 not found, using a simulated Payload object.")
    class Payload:
        def __init__(self):
            self.data = None
        def __str__(self):
            return str(self.data)
    SimplePayload = Payload
else:
    class SimplePayload(Payload):
        def __init__(self):
            super().__init__()
            self.data = None
        def to_pack_list(self):
            return [self.data]
        @classmethod
        def from_unpack_list(cls, lst):
            instance = cls()
            instance.data = lst[0]
            return instance

def xor_bytes(a, b):
    """Returns the byte-wise XOR of two byte sequences of the same length."""
    return bytes(x ^ y for x, y in zip(a, b))

# --- PyIPv8Transport class using SimplePayload ---
class PyIPv8Transport:
    @staticmethod
    def encapsulate(packet, sender="Alice", receiver="Bob"):
        header = {
            "protocol": "ipv8",
            "timestamp": time.time(),
            "sender": sender,
            "receiver": receiver
        }
        payload = SimplePayload()
        payload.data = {"header": header, "data": packet}
        return payload

    @staticmethod
    def send(packet, sender="Alice", receiver="Bob"):
        payload = PyIPv8Transport.encapsulate(packet, sender, receiver)
        print("PyIPv8Transport: Sending payload:")
        print(payload)
        return payload

    @staticmethod
    def receive(payload):
        print("PyIPv8Transport: Received payload:")
        print(payload)
        data = payload.data.get("data")
        return data

# --- Other unchanged classes (NFCMock, ZKSNARK, CentralBank, OfflineWallet, Environment) ---
class NFCMock:
    @staticmethod
    def send(transaction_data):
        print("NFCMock: Transaction data sent.")
        return json.dumps(transaction_data)

    @staticmethod
    def receive(transaction_json):
        print("NFCMock: Transaction data received.")
        return json.loads(transaction_json)

class ZKSNARK:
    @staticmethod
    def generate_key():
        return "simulated_zk_key"

    @staticmethod
    def generate_proof(private_key, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        return hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()

    @staticmethod
    def verify_proof(proof, public_key, amount, zk_key):
        # Convert public key to bytes using save_pkcs1() before hashing
        public_key_bytes = public_key.save_pkcs1()
        expected_proof = hashlib.sha256(f"{public_key_bytes}{amount}{zk_key}".encode()).hexdigest()
        return proof == expected_proof

class CentralBank:
    def __init__(self, name):
        self.name = name
        self.coins_issued = []
        self.key_pair = rsa.newkeys(2048)
        self.deposited_coins = set()
        self.used_nonces = set()
        self.logged_transactions = []

    def issue_coin(self, recipient, value):
        coin_id = hashlib.sha256(str(random.getrandbits(256)).encode()).hexdigest()
        signature = self.sign_coin(value, coin_id)
        signature_b64 = base64.b64encode(signature).decode('utf-8')
        coin = {
            "coin_id": coin_id,
            "value": value,
            "signature": signature_b64
        }
        self.coins_issued.append(coin)
        return coin

    def sign_coin(self, value, coin_id):
        message = f"{value}{coin_id}".encode()
        return rsa.sign(message, self.key_pair[1], 'SHA-256')

    def sign_blinded_hash(self, blinded_hash):
        return hashlib.sha256(f"{blinded_hash}{self.name}".encode()).hexdigest()

    def verify_and_deposit_coin(self, coin, sender_public_key, proof, zk_key, nonce):
        coin_hash = hashlib.sha256((str(coin['value']) + coin['coin_id'] + coin['signature']).encode()).hexdigest()
        if coin_hash in self.deposited_coins:
            return "Coin Already Defunded or Verification Failed"
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, self.key_pair[0])
        except rsa.VerificationError:
            return "Coin Already Defunded or Verification Failed"
        if nonce in self.used_nonces:
            return "Nonce already used. Double spending detected."
        self.used_nonces.add(nonce)
        amount = coin['value']
        expected_proof = hashlib.sha256(f"{sender_public_key}{amount}{zk_key}".encode()).hexdigest()
        if proof != expected_proof:
            return "Coin Already Defunded or Verification Failed"
        transaction_record = (sender_public_key.save_pkcs1(), nonce, proof)
        if transaction_record in self.logged_transactions:
           return "Double spending detected: Transaction already recorded."
        self.logged_transactions.append(transaction_record)
        self.deposited_coins.add(coin_hash)
        return "Coin Deposited Successfully"

class OfflineWallet:
    def __init__(self, owner_id):
        self.owner_id = owner_id
        self.balance = 100
        self.key_pair = self.generate_key_pair()
        self.zk_key = ZKSNARK.generate_key()
        self.nonce = 0
        self.withdrawal_latencies = []
               self.deposit_latencies = []
        self.coins = []
        self.received_logs = []

    def log_transaction(self, sender_key, nonce, proof):
        self.received_logs.append((sender_key.save_pkcs1(), nonce, proof))

    def has_received_transaction(self, sender_key, nonce, proof):
        return (sender_key.save_pkcs1(), nonce, proof) in self.received_logs

    def generate_key_pair(self):
        return rsa.newkeys(2048)

    def store_coin(self, coin):
        self.coins.append(coin)
        print(f"Coin stored: {coin}")

    def prepare_transaction(self, recipient, amount):
        self.nonce += 1
        if not self.coins:
            raise ValueError("No coins available to send.")
        coin = self.coins[0]
        proof = ZKSNARK.generate_proof(self.key_pair[1], recipient.key_pair[0], amount, self.zk_key)
        return {
            "amount": amount,
            "coin": coin.copy(),
            "recipient_key": base64.b64encode(recipient.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "from_agent_key": base64.b64encode(self.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.nonce,
            "timestamp": time.time()
        }

    def blind_coin(self):
        f = hashlib.sha256(self.key_pair[0].save_pkcs1()).hexdigest()
        blinding_factor = random.randint(1, 10**5)
        blinded_hash = (int(f, 16) * blinding_factor) % (2**256)
        return {"hash": f, "blinded_hash": blinded_hash, "blinding_factor": blinding_factor}

class Environment:
    def __init__(self):
        self.measurement = {
            "transaction_id": [],
            "period": [],
            "to_agent": [],
            "from_agent": [],
            "settlement_type": [],
            "amount": [],
            "description": [],
            "latency": [],
            "zk_proof_generation_time": [],
            "zk_proof_verification_time": [],
            "transaction_verification_time": [],
            "load_computing": [],
            "gas_used": [], 
            "decryption_time": [],         # New field for decryption time
            "decryption_cpu_time": [],     # New field for decryption CPU load
            "status": []
        }

    def log_transaction(self, transaction_id, period, to_agent, from_agent, settlement_type, amount, description,
                        latency=None, zk_proof_gen_time=None, zk_proof_ver_time=None, transaction_ver_time=None,
                        load_computing=None, gas_used=None, decryption_time=None, decryption_cpu_time=None, status="Pending"):
        self.measurement["transaction_id"].append(transaction_id)
        self.measurement["period"].append(period)
        self.measurement["to_agent"].append(to_agent)
        self.measurement["from_agent"].append(from_agent)
        self.measurement["settlement_type"].append(settlement_type)
        self.measurement["amount"].append(amount)
        self.measurement["description"].append(description)
        self.measurement["latency"].append(latency)
        self.measurement["zk_proof_generation_time"].append(zk_proof_gen_time)
        self.measurement["zk_proof_verification_time"].append(zk_proof_ver_time)
        self.measurement["transaction_verification_time"].append(transaction_ver_time)
        self.measurement["load_computing"].append(load_computing)
        self.measurement["gas_used"].append(gas_used)
        self.measurement["decryption_time"].append(decryption_time)         # Record decryption time
        self.measurement["decryption_cpu_time"].append(decryption_cpu_time) # Record decryption CPU load
        self.measurement["status"].append(status)

    def generate_metrics(self):
        df = pd.DataFrame(self.measurement)
        total_transactions = len(df)
        total_latency = df["latency"].sum()
        overall_throughput = total_transactions / total_latency if total_latency > 0 else 0

        phase_throughput = {}
        for phase in df['settlement_type'].unique():
            phase_df = df[df['settlement_type'] == phase]
            phase_count = len(phase_df)
            phase_total_latency = phase_df["latency"].sum()
            phase_throughput[phase] = phase_count / phase_total_latency if phase_total_latency > 0 else 0
        df["throughput"] = df["settlement_type"].apply(lambda phase: phase_throughput.get(phase, 0))

        for col in ["zk_proof_generation_time", "zk_proof_verification_time", "load_computing", "throughput"]:
            df[col] = df[col].apply(lambda x: f"{x:.4f}" if pd.notnull(x) and isinstance(x, (float, int)) else x)

        print("\n--- Simulation Metrics ---")
        print(f"Total Transactions: {total_transactions}")
        print(f"Total Latency (seconds): {total_latency:.6f}")
        print(f"Overall Throughput (transactions/sec): {overall_throughput:.6f}")

        print("\nThroughput per phase:")
        for phase, tp in phase_throughput.items():
            print(f"  {phase}: {tp:.6f} transactions/sec")
        df.to_csv("simulation_metricsfunding.csv", index=False)

# Add this class to interact with the Shard Ledger
class ShardLedgerInterface:
    def __init__(self, provider_url, contract_address, abi_file_path, chain_id=1337, gas=2000000, gas_price_gwei="50"):
        # Connect to Ganache (or another network)
        self.web3 = Web3(Web3.HTTPProvider(provider_url))
        if not self.web3.is_connected():
            raise Exception("Connection error to Ganache on Shard Ledger.")
        print("Connected to Shard Ledger with", provider_url)
        self.contract_address = self.web3.to_checksum_address(contract_address)
        with open(abi_file_path, "r") as abi_file:
            self.contract_abi = json.load(abi_file)
        self.contract = self.web3.eth.contract(address=self.contract_address, abi=self.contract_abi)
        self.chain_id = chain_id
        self.gas = gas
        self.gas_price = self.web3.to_wei(gas_price_gwei, "gwei")

    def update_user_balance(self, current_balance, T, op, R_pk, alpha, sender, private_key):
        """
        Calls the updateBalance function of the Shard Ledger smart contract.
        - current_balance: the current encrypted state (c_B) of the user's account.
        - T: the token (amount) to add (or subtract if op == "DEC")
        - op: "INC" to increase, "DEC" to decrease.
        - R_pk: public key (or other parameter) used for encryption.
        - alpha: beneficiary identifier (e.g., user's pseudonym).
        - sender: address that sends the transaction.
        - private_key: private key corresponding to sender.
        """
        tx = self.contract.functions.updateBalance(
            alpha,      # assuming the smart contract takes the user's pseudonym
            T,          # amount to modify
            op          # operation: "INC" or "DEC"
        ).build_transaction({
            "chainId": self.chain_id,
            "gas": self.gas,
            "gasPrice": self.gas_price,
            "nonce": self.web3.eth.get_transaction_count(self.web3.to_checksum_address(sender)),
        })
        signed_tx = self.web3.eth.account.sign_transaction(tx, private_key=private_key)
        tx_hash = self.web3.eth.send_raw_transaction(signed_tx.rawTransaction)
        receipt = self.web3.eth.wait_for_transaction_receipt(tx_hash)
        print("Balance update sent transaction. Receipt:", receipt)
        return receipt

# --- Integration of the "Bob pays Carla" scenario ---
class OfflinePaymentSimulation:
    def __init__(self):
        self.environment = Environment()
        self.alice_wallet = OfflineWallet("Alice")
        self.bob_wallet = OfflineWallet("Bob")
        self.carla_wallet = OfflineWallet("Carla")  # New wallet for Carla
        self.central_bank = CentralBank("Central Bank")
        self.used_nonces = set()
        # Create a RIO wallet to simulate the RIO wallet
        self.rio_wallet = OfflineWallet("RIO")
        # We can save T and the verification key for other operations
        self.last_token_value = None
        self.last_K_verif = None
        
    def validate_coin_signature(self, coin, public_key):
        if not coin or "signature" not in coin:
            print("Invalid coin: Missing or no signature.")
            return False
        message = f"{coin['value']}{coin['coin_id']}".encode()
        try:
            signature_bytes = base64.b64decode(coin["signature"])
            rsa.verify(message, signature_bytes, public_key)
            print("Coin signature is valid.")
            return True
        except rsa.VerificationError:
            print("Invalid coin: Signature verification failed.")
            return False

    def verify_sender_legitimacy(self, sender_public_key, known_keys):
        return any(sender_public_key.save_pkcs1() == key.save_pkcs1() for key in known_keys)

    def verify_nonce(self, nonce, used_nonces):
        if nonce in used_nonces:
            print(f"Nonce {nonce} has already been used. Double spending detected.")
            return False
        used_nonces.add(nonce)
        return True

    def verify_recipient(self, transaction_data, expected_recipient_key):
        recipient_key = transaction_data.get("recipient_key")
        if not isinstance(recipient_key, rsa.PublicKey):
            print("Failure: recipient_key is not an rsa.PublicKey object.")
            return False
        return recipient_key.save_pkcs1() == expected_recipient_key.save_pkcs1()


    def simulate_deposit(self):
        """Simulates the deposit of a coin to the central bank."""
        if not self.bob_wallet.coins:
            print("Bob has no coin to deposit.")
            return

        coin = self.bob_wallet.coins[0]  # Select the first coin for deposit
        amount = coin['value']

        computing_start = time.process_time()
        start_time = time.time()

        # Generate ZK proof for deposit
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1], self.central_bank.key_pair[0], amount, self.bob_wallet.zk_key)

        # Prepare deposit data
        deposit_data = {
            "coin": coin.copy(),
            "sender_public_key": base64.b64encode(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM')).decode('utf-8'),
            "proof": proof,
            "nonce": self.bob_wallet.nonce + 1,  # Increment nonce for the deposit
            "timestamp": time.time()
        }

        # Simulate the deposit process
        deposit_result = self.central_bank.verify_and_deposit_coin(
            coin,
            self.bob_wallet.key_pair[0],
            proof,
            self.bob_wallet.zk_key,
            deposit_data["nonce"]  # Use the incremented nonce for deposit
        )

        if deposit_result == "Coin Deposited Successfully":
            self.bob_wallet.coins.pop(0)  # Remove deposited coin from wallet
            self.bob_wallet.balance -= amount  # Update wallet balance
            self.bob_wallet.nonce += 1  # Increment wallet's nonce after successful deposit
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            print(f"Deposit completed successfully. Total time: {latency:.6f}s, "
                  f"CPU compute load: {computing_load:.6f}s.")

            self.environment.log_transaction(
                transaction_id=len(self.environment.measurement["transaction_id"]),
                period=time.time(),
                to_agent="Central Bank",
                from_agent=self.bob_wallet.owner_id,
                settlement_type="deposit",
                amount=amount,
                description="Coin Deposit",
                latency=latency,
                zk_proof_gen_time=None,  # Add timing if needed
                zk_proof_ver_time=None,  # Add timing if needed
                transaction_ver_time=None,  # Add timing if needed
                load_computing=computing_load,
                status="Success"
            )
        else:
            print(f"Deposit failed: {deposit_result}")

    # --- Withdrawal method ---
    def simulate_withdrawal(self, amount):
        print(f"Alice withdraws {amount} units...")
        computing_start = time.process_time()
        total_start_time = time.time()
        blinded_coin = self.alice_wallet.blind_coin()
        interaction_start_time = time.process_time()
        blinded_signature = self.central_bank.sign_blinded_hash(blinded_coin["blinded_hash"])
        interaction_cpu_time = time.process_time() - interaction_start_time
        message = f"{amount}{self.alice_wallet.owner_id}".encode()
        unblinded_signature = rsa.sign(message, self.central_bank.key_pair[1], 'SHA-256')
        coin = self.central_bank.issue_coin(self.alice_wallet.owner_id, amount)
        self.alice_wallet.store_coin(coin)
        total_latency = time.time() - total_start_time
        computing_load = time.process_time() - computing_start
        print(f"Withdrawal completed successfully. Total time: {total_latency:.6f}s, "
              f"Interaction time (CPU): {interaction_cpu_time:.6f}s, "
              f"CPU compute load: {computing_load:.6f}s.")
        self.environment.log_transaction(
            transaction_id=len(self.environment.measurement["transaction_id"]),
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Central Bank",
            settlement_type="withdrawal",
            amount=amount,
            description="Withdrawal with blinding",
            latency=total_latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=interaction_cpu_time,
            load_computing=computing_load,
            status="Success"
        )

    # --- Offline payment method (Alice pays Bob) ---
    def simulate_offline_transaction(self, amount=None):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        if amount is None:
            amount = random.randint(10, 50)
        if not self.alice_wallet.coins:
            print("No coin available for an offline transaction.")
            return
        coin = self.alice_wallet.coins[0]
        if amount > coin["value"]:
            print(f"Requested amount ({amount}) greater than coin value ({coin['value']}). Using the coin's value.")
            amount = coin["value"]
        elif amount < coin["value"]:
            print("Partial transaction not supported. Using the coin's full value.")
            amount = coin["value"]
        computing_start = time.process_time()
        start_time = time.time()
        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.bob_wallet.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start_time
        transaction_data = self.alice_wallet.prepare_transaction(self.bob_wallet, amount)
        transaction_data.update({
            "transaction_id": transaction_id,
            "from_agent": self.alice_wallet.owner_id,
            "to_agent": self.bob_wallet.owner_id,
            "nonce": transaction_data["nonce"]
        })
        json_data = json.dumps(transaction_data).encode('utf-8')
        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_data)
        packet = {
            "encrypted_aes_key": base64.b64encode(
                PKCS1_OAEP.new(
                    RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1(format='PEM'))
                ).encrypt(aes_key)
            ).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Sending transaction encapsulated via pyipv8...")
        ipv8_payload = PyIPv8Transport.send(packet, sender=self.alice_wallet.owner_id, receiver=self.bob_wallet.owner_id)
        received_packet = PyIPv8Transport.receive(ipv8_payload)
        try:
            enc_aes_key = base64.b64decode(received_packet["encrypted_aes_key"])
            ciphertext = base64.b64decode(received_packet["ciphertext"])
            tag = base64.b64decode(received_packet["tag"])
            nonce = base64.b64decode(received_packet["nonce"])
            bob_private_pem = self.bob_wallet.key_pair[1].save_pkcs1(format='PEM')
            bob_private_key_py = RSA.importKey(bob_private_pem)
            cipher_rsa_bob = PKCS1_OAEP.new(bob_private_key_py)
            decrypted_aes_key = cipher_rsa_bob.decrypt(enc_aes_key)
            cipher_aes_bob = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=nonce)
            decrypted_data = cipher_aes_bob.decrypt_and_verify(ciphertext, tag)
            received_transaction = json.loads(decrypted_data.decode('utf-8'))
            if isinstance(received_transaction["from_agent_key"], str):
                try:
                    sender_public_key = rsa.PublicKey.load_pkcs1(base64.b64decode(received_transaction["from_agent_key"]))
                    received_transaction["from_agent_key"] = sender_public_key
                    print("Issuer's public key correctly loaded.")
                except Exception as e:
                    print(f"Error while decoding issuer's key: {e}")
                    return
            if isinstance(received_transaction["recipient_key"], str):
                try:
                    received_transaction["recipient_key"] = rsa.PublicKey.load_pkcs1(
                        base64.b64decode(received_transaction["recipient_key"])
                    )
                    print("recipient_key correctly loaded into PublicKey.")
                except Exception as e:
                    print(f"Error while decoding recipient_key: {e}")
                    return
            if not isinstance(received_transaction["recipient_key"], rsa.PublicKey):
                print("Error: recipient_key is not a valid rsa.PublicKey object after processing.")
                return
            print(f"Received recipient_key: {received_transaction['recipient_key']}")
            print(f"Type of recipient_key: {type(received_transaction['recipient_key'])}")
            verification_start_time = time.time()
            if not self.validate_coin_signature(received_transaction.get("coin"), self.central_bank.key_pair[0]):
                print("Coin signature verification failed.")
                return
            sender_public_key = received_transaction.get("from_agent_key")
            if not self.verify_sender_legitimacy(sender_public_key, [self.alice_wallet.key_pair[0]]):
                print("Sender legitimacy verification failed.")
                return
            else:
                print("Sender legitimacy verification successful: The sender is legitimate.")
            proof = received_transaction.get("proof")
            nonce = received_transaction.get("nonce")
            if self.bob_wallet.has_received_transaction(sender_public_key, nonce, proof):
                print("Double spending detected: transaction already received.")
                return
            else:
                self.bob_wallet.log_transaction(sender_public_key, nonce, proof)
                print("Transaction successfully logged.")
            if not self.verify_nonce(received_transaction.get("nonce"), self.used_nonces):
                print("Nonce verification failed.")
                return
            zk_proof_ver_start_time = time.time()
            proof_valid = ZKSNARK.verify_proof(
                proof,
                self.bob_wallet.key_pair[0],
                amount,
                self.alice_wallet.zk_key
            )
            zk_proof_ver_time = time.time() - zk_proof_ver_start_time
            if not proof_valid:
                print("zk-SNARK proof verification failed.")
                return
            else:
                print("zk-SNARK proof verification successful: The proof is valid.")
            print("Nonce is verified and unique, and the zk-SNARK proof is verified and unique.")
            verification_time = time.time() - verification_start_time
            self.alice_wallet.coins.pop(0)
            received_transaction["coin"]["nonce"] = received_transaction["nonce"]
            received_transaction["coin"]["recipient"] = self.bob_wallet.owner_id
            self.bob_wallet.store_coin(received_transaction["coin"])
            self.alice_wallet.balance -= amount
            self.bob_wallet.balance += amount
            latency = time.time() - start_time
            computing_load = time.process_time() - computing_start
            self.environment.log_transaction(
                transaction_id=transaction_id,
                period=time.time(),
                to_agent=self.bob_wallet.owner_id,
                from_agent=self.alice_wallet.owner_id,
                settlement_type="offline_transaction",
                amount=amount,
                description="Offline Transaction",
                latency=latency,
                zk_proof_gen_time=zk_proof_gen_time,
                zk_proof_ver_time=zk_proof_ver_time,
                transaction_ver_time=verification_time,
                load_computing=computing_load,
                status="Success"
            )
        except Exception as e:
            print(f"Error during transaction: {e}")

    # --- New method: Bob pays Carla scenario ---
    def simulate_bob_pays_carla(self, amount):
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()

        print(f"\n=== Scenario: Bob pays Carla for {amount} units ===")
        contract = f"Contract_Bob_pays_Carla_{amount}"
        print("Bob and Carla agree on the contract:", contract)

        rio_cert = "RIO_pk_cert_dummy"
        print("Bob selects the RIO certificate:", rio_cert)

        carla_pseudonym = self.carla_wallet.key_pair[0].save_pkcs1().decode('utf-8')
        print("Carla provides Bob her pseudonymous identifier.")

        if not self.bob_wallet.coins:
            print("Bob has no available coin for payment.")
            return
        ecoin = self.bob_wallet.coins[0]
        print("Selected e-coin:", ecoin)

        signature_contract = rsa.sign(contract.encode('utf-8'), self.bob_wallet.key_pair[1], 'SHA-256')
        signature_contract_b64 = base64.b64encode(signature_contract).decode('utf-8')
        print("Bob signs the contract:", signature_contract_b64)

        zk_proof_start_time = time.time()
        proof = ZKSNARK.generate_proof(self.bob_wallet.key_pair[1],
                                       self.carla_wallet.key_pair[0],
                                       amount,
                                       self.bob_wallet.zk_key)
        print("Generated proof:", proof)
        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(
                    proof,
                    self.carla_wallet.key_pair[0],
                    amount,
                    self.bob_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start
        if not proof_valid:
            print("zk-SNARK proof verification failed.")
            return
        else:
            print("zk-SNARK proof verification successful: The proof is valid.")

        transaction_payload = {
            "contract": contract,
            "contract_signature": signature_contract_b64,
            "proof": proof,
            "RIO_cert": rio_cert,
            "amount": amount,
            "bob_pseudonym": self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8'),
            "carla_pseudonym": carla_pseudonym,
            "timestamp": time.time()
        }
        json_payload = json.dumps(transaction_payload).encode('utf-8')

        aes_key = get_random_bytes(32)
        cipher_aes = AES.new(aes_key, AES.MODE_GCM)
        ciphertext, tag = cipher_aes.encrypt_and_digest(json_payload)

        rio_key_pair = rsa.newkeys(2048)
        rio_public = rio_key_pair[0]
        rio_private = rio_key_pair[1]

        rio_public_pem = rio_public.save_pkcs1(format='PEM')
        cipher_rsa_enc = PKCS1_OAEP.new(RSA.importKey(rio_public_pem))
        encrypted_aes_key = cipher_rsa_enc.encrypt(aes_key)

        encrypted_packet = {
            "encrypted_aes_key": base64.b64encode(encrypted_aes_key).decode('utf-8'),
            "ciphertext": base64.b64encode(ciphertext).decode('utf-8'),
            "tag": base64.b64encode(tag).decode('utf-8'),
            "nonce": base64.b64encode(cipher_aes.nonce).decode('utf-8')
        }
        print("Bob to Carla transaction (encrypted for RIO):", encrypted_packet)

        rio_private_pem = rio_private.save_pkcs1(format='PEM')
        cipher_rsa_dec = PKCS1_OAEP.new(RSA.importKey(rio_private_pem))
        decrypted_aes_key = cipher_rsa_dec.decrypt(encrypted_aes_key)

        cipher_aes_rio = AES.new(decrypted_aes_key, AES.MODE_GCM, nonce=cipher_aes.nonce)
        decrypted_payload = cipher_aes_rio.decrypt_and_verify(ciphertext, tag)
        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time: {transaction_verification_time:.6f} seconds")

        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start

        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.carla_wallet.owner_id,
            from_agent=self.bob_wallet.owner_id,
            settlement_type="online_transaction",  # or "bob_to_carla"
            amount=amount,
            description="Payment from Bob to Carla",
            latency=latency,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=computing_load,
            status="Success"
        )

    def simulate_online_deposit_PAD(self, amount):
        """
        Online Deposit/Transfer via PAD scenario.
        Simulates an online deposit (or transfer to own account)
        through the PAD system managed by the RIO.
        """
        print(f"\n=== Online Deposit via PAD scenario for {amount} units ===")
        transaction_id = len(self.environment.measurement["transaction_id"]) + 1
        computing_start = time.process_time()
        start_time = time.time()

        if not self.alice_wallet.coins:
            print("No coin available for an online deposit.")
            return
        coin = self.alice_wallet.coins[0]
        zk_proof_start = time.time()
        proof = ZKSNARK.generate_proof(self.alice_wallet.key_pair[1],
                                       self.central_bank.key_pair[0],
                                       amount,
                                       self.alice_wallet.zk_key)
        zk_proof_gen_time = time.time() - zk_proof_start

        zk_ver_start = time.time()
        proof_valid = ZKSNARK.verify_proof(proof,
                                           self.central_bank.key_pair[0],
                                           amount,
                                           self.alice_wallet.zk_key)
        zk_proof_ver_time = time.time() - zk_ver_start

        if not proof_valid:
            print("zk-SNARK proof verification failed during online transaction.")
            return
        print("zk-SNARK proof verification succeeded.")

        Fs = self.alice_wallet.blind_coin()
        T = str(random.randint(1000, 9999))
        print("Generated token T:", T)
        self.last_token_value = T
    
        K_verif = get_random_bytes(16)
        print("Generated verification key K_verif:", base64.b64encode(K_verif).decode())
        self.last_K_verif = K_verif
       
        K_sym = get_random_bytes(32)
        data_to_encrypt = T.encode('utf-8') + json.dumps(Fs, sort_keys=True).encode('utf-8')
        cipher_pad = AES.new(K_sym, AES.MODE_GCM)
        C, tag = cipher_pad.encrypt_and_digest(data_to_encrypt)
        nonce_pad = cipher_pad.nonce

        signature_C = rsa.sign(C, self.alice_wallet.key_pair[1], 'SHA-256')
        signature_C_b64 = base64.b64encode(signature_C).decode('utf-8')
        C_prime = {
            "encrypted_content": base64.b64encode(C).decode('utf-8'),
            "signature": signature_C_b64
        }
        C_prime_json = json.dumps(C_prime).encode('utf-8')

        K_PAD = get_random_bytes(32)
        cipher_pad2 = AES.new(K_PAD, AES.MODE_GCM)
        C_prime_ciphertext, C_prime_tag = cipher_pad2.encrypt_and_digest(C_prime_json)
        nonce_pad2 = cipher_pad2.nonce

        cipher_rsa_for_PAD = PKCS1_OAEP.new(RSA.importKey(self.alice_wallet.key_pair[0].save_pkcs1(format='PEM')))
        encrypted_K_PAD = cipher_rsa_for_PAD.encrypt(K_PAD)

        C_prime_encrypted = {
            "encrypted_key": base64.b64encode(encrypted_K_PAD).decode('utf-8'),
            "ciphertext": base64.b64encode(C_prime_ciphertext).decode('utf-8'),
            "tag": base64.b64encode(C_prime_tag).decode('utf-8'),
            "nonce": base64.b64encode(nonce_pad2).decode('utf-8')
        }

        R_sym = get_random_bytes(32)
        S_m = xor_bytes(K_sym, R_sym)
        N_t = b"3"
        S_t = hashlib.sha256(S_m + N_t).hexdigest()
        N_v = b"2"
        V = hashlib.sha256(b"R" + self.alice_wallet.key_pair[0].save_pkcs1() + N_v).hexdigest()

        E_t = "encrypted_trustee_shares_dummy"
        E_v = "encrypted_validator_shares_dummy"

        D_ts = {
            "E_t": E_t,
            "E_v": E_v,
            "H_T": hashlib.sha256(T.encode('utf-8')).hexdigest(),
            "C_prime_encrypted": json.dumps(C_prime_encrypted)
        }

        print("Generated PAD data (D_ts):", D_ts)

        verification_start_time = time.time()
        verification_end_time = time.time()
        transaction_verification_time = verification_end_time - verification_start_time
        print(f"Transaction verification time (RIO): {transaction_verification_time:.6f} seconds")

        self.alice_wallet.coins.pop(0)
        self.alice_wallet.balance -= amount
        print(f"Online deposit (via PAD) of {amount} units completed successfully for {self.alice_wallet.owner_id}.")

        sender_address = "0x4da60efbbf6a9aa9846e749c0c7b52656a484636"
        recipient_address = sender_address
        tx_type = "online_deposit_PAD"
        details = "Online Deposit via PAD with Smart Contract"

        private_key = "0x584beec8f497a30c0430679175fd36f49a5d97bfa8fd42242d423e5696397d29"

        receipt = pad_interface.post_transaction(
            transaction_id,
            sender_address,
            recipient_address,
            amount,
            tx_type,
            details,
            private_key
        )

        gas_used = receipt.get("gasUsed", None)
        print("Gas used for PAD transaction:", gas_used)
        
        latency = time.time() - start_time
        computing_load = time.process_time() - computing_start

        self.environment.log_transaction(
            transaction_id=transaction_id,
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent=self.alice_wallet.owner_id,
            settlement_type="online_deposit_PAD",
            amount=amount,
            description="Online Deposit via PAD",
            latency=latency,
            zk_proof_gen_time=zk_proof_gen_time,
            zk_proof_ver_time=zk_proof_ver_time,
            transaction_ver_time=transaction_verification_time,
            load_computing=computing_load,
            gas_used=gas_used,
            status="Success"
        )

    def simulate_multiple_offline_transactions(self, nb_transactions, offline_amount=None):
        nb_possible = len(self.alice_wallet.coins)
        nb_to_execute = min(nb_transactions, nb_possible)
        print(f"Number of offline transactions requested: {nb_transactions}.")
        print(f"Number of coins available in the wallet: {nb_possible}.")
        print(f"Executing {nb_to_execute} offline transaction(s).")
        for i in range(nb_to_execute):
            self.simulate_offline_transaction(amount=offline_amount)

    def simulate_multiple_withdrawals(self, nb_withdrawals, withdrawal_amount):
        print(f"Number of withdrawals requested: {nb_withdrawals}.")
        for i in range(nb_withdrawals):
            self.simulate_withdrawal(withdrawal_amount)

    def simulate_multiple_deposits(self, nb_deposits):
        nb_possible = len(self.bob_wallet.coins)
        nb_to_execute = min(nb_deposits, nb_possible)
        print(f"Number of deposits requested: {nb_deposits}.")
        print(f"Number of coins available in Bob's wallet: {nb_possible}.")
        for i in range(nb_to_execute):
            self.simulate_deposit()

    def simulate_decryption_request(self, T, threshold, K_verif):
        """
        Simulates the decryption process in the PAD protocol:
         - If T > theta, RIO creates a decryption request and computes φ = G(H(T), K_{Bpub})
         - Then Bob retrieves and decrypts the secret:
              R_sf ← G(E^{-1}(E^{-1}(C', K_{Bpriv}), K_{verif}))
        """
        print("\n=== Start of decryption request process (PIP) ===")
        if int(T) > threshold:
            H_T = hashlib.sha256(T.encode()).hexdigest()
            bob_pub = self.bob_wallet.key_pair[0].save_pkcs1().decode('utf-8')
            phi = hashlib.sha256((H_T + bob_pub).encode()).hexdigest()
            print("Decryption request published with φ =", phi)
        else:
            print("Token T (", T, ") below threshold (", threshold, "). No decryption request.")
            return

        secret_data = "my_very_confidential_secret"
        print("Original secret (simulated):", secret_data)
       
        cipher1 = AES.new(K_verif, AES.MODE_GCM)
        secret_ciphertext, secret_tag = cipher1.encrypt_and_digest(secret_data.encode())
        nonce_secret = cipher1.nonce

        bob_pub = RSA.importKey(self.bob_wallet.key_pair[0].save_pkcs1())
        cipher_rsa_enc = PKCS1_OAEP.new(bob_pub)

        C_prime = cipher_rsa_enc.encrypt(secret_ciphertext)
        print("C_prime (simulated double encryption) =", base64.b64encode(C_prime).decode())

        decryption_start_time = time.time()
        cpu_start = time.process_time()

        bob_priv = RSA.importKey(self.bob_wallet.key_pair[1].save_pkcs1())
        cipher_rsa_dec = PKCS1_OAEP.new(bob_priv)

        decrypted_ciphertext = cipher_rsa_dec.decrypt(C_prime)
        cipher2 = AES.new(K_verif, AES.MODE_GCM, nonce=nonce_secret)
        recovered_secret = cipher2.decrypt_and_verify(decrypted_ciphertext, secret_tag).decode()
       
        cpu_end = time.process_time()
        decryption_end_time = time.time()
        decryption_time = decryption_end_time - decryption_start_time
        decryption_cpu_time = cpu_end - cpu_start
    
        R_sf = hashlib.sha256(recovered_secret.encode()).hexdigest()
        print("Bob has recovered the secret R_sf =", R_sf)

        print("=== End of decryption request process ===")
        self.environment.log_transaction(
            transaction_id="decryption_request",
            period=time.time(),
            to_agent=self.bob_wallet.owner_id,
            from_agent="PAD_System",
            settlement_type="decryption_request",
            amount=0,
            description="Decryption of PAD secret",
            latency=decryption_time,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=decryption_cpu_time,
            gas_used=None,
            status="Success"
         )

    def simulate_funding_operation(self, A_B, amount, shard_interface, sender_address, sender_private_key):
        """
        Simulates the Funding Process.
          - A_B: Alice's bank account identifier (e.g., "Alice_Fiat_Account_001")
          - amount: amount T to deposit
          - shard_interface: instance of ShardLedgerInterface to call update_user_balance
          - sender_address and sender_private_key: information of the sender (e.g., PAD account)
        """
        print("\n=== The beginning of Funding Process ===")
        comp_start = time.process_time()
        start_time = time.time()

        R_pk = self.rio_wallet.key_pair[0].save_pkcs1().decode()
        P_T = hashlib.sha256((A_B + str(amount) + R_pk).encode()).hexdigest()
        print("P_T (result of κ):", P_T)
        
        lambda_A = hashlib.sha256(P_T.encode()).hexdigest()
        print("Claim check λ_A:", lambda_A)
        
        random_r = str(random.getrandbits(128))
        lambda_prime_A = hashlib.sha256((lambda_A + random_r).encode()).hexdigest()
        print("Re-randomized claim check λ'_A:", lambda_prime_A)
        
        self.last_token_value = str(amount)
        self.last_K_verif = get_random_bytes(16)

        funding_time = time.time() - start_time
        funding_cpu = time.process_time() - comp_start

        op = "INC"
        alpha = self.alice_wallet.owner_id
        current_balance = 0
        
        print("Sending balance update request on Shard Ledger...")
        shard_receipt = shard_interface.update_user_balance(current_balance, amount, op, R_pk, alpha, sender_address, sender_private_key)
        
        update_time = time.time() - start_time
        update_cpu = time.process_time() - comp_start

        self.environment.log_transaction(
            transaction_id="funding_request",
            period=time.time(),
            to_agent=self.alice_wallet.owner_id,
            from_agent="Bank",
            settlement_type="funding",
            amount=amount,
            description="Funding Operation: deposit from fiat account to RIO wallet & update balance",
            latency=update_time,
            zk_proof_gen_time=None,
            zk_proof_ver_time=None,
            transaction_ver_time=None,
            load_computing=update_cpu,
            gas_used=shard_receipt.get("gasUsed", None),
            decryption_time=None,
            decryption_cpu_time=None,
            status="Success"
        )
        print("End of Funding process. Total time: {:.6f} sec, CPU load: {:.6f}".format(update_time, update_cpu))
        return {
            "P_T": P_T,
            "lambda_A": lambda_A,
            "lambda_prime_A": lambda_prime_A,
            "funding_time": update_time,
            "funding_cpu": update_cpu,
            "shard_receipt": shard_receipt
        }

# --- Digital identity management ---
class AttributeProvider:
    def __init__(self):
        self.documents = {}

    def issue_document(self, user_id, attributes):
        doc = {
            "user_id": user_id,
            "attributes": attributes,
            "timestamp": time.time()
        }
        doc_str = json.dumps(doc, sort_keys=True)
        signature = hashlib.sha256(doc_str.encode('utf-8')).hexdigest()
        doc["signature"] = signature
        self.documents[user_id] = doc
        return doc

class IdentityProvider:
    def __init__(self, attribute_provider):
        self.attribute_provider = attribute_provider

    def verify_and_issue_credential(self, user_id, attributes):
        doc = self.attribute_provider.issue_document(user_id, attributes)
        credential = {
            "user_id": user_id,
            "credential": f"Credential_for_{user_id}",
            "issued_at": time.time(),
            "doc_hash": hashlib.sha256(json.dumps(doc, sort_keys=True).encode('utf-8')).hexdigest()
        }
        cred_str = json.dumps(credential, sort_keys=True)
        credential["signature"] = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return credential

class PIDProvider:
    def __init__(self):
        self.pids = {}

    def issue_pid(self, credential):
        pid = hashlib.sha256((credential["user_id"] + credential["signature"]).encode('utf-8')).hexdigest()
        self.pids[credential["user_id"]] = pid
        return pid

class DigitalIdentityManager:
    def __init__(self):
        self.attribute_provider = AttributeProvider()
        self.identity_provider = IdentityProvider(self.attribute_provider)
        self.pid_provider = PIDProvider()

    def onboard_user(self, user_id, attributes):
        credential = self.identity_provider.verify_and_issue_credential(user_id, attributes)
        pid = self.pid_provider.issue_pid(credential)
        digital_identity = {
            "user_id": user_id,
            "credential": credential,
            "pid": pid
        }
        return digital_identity

    def verify_identity(self, digital_identity):
        cred = digital_identity["credential"].copy()
        signature = cred.pop("signature", None)
        cred_str = json.dumps(cred, sort_keys=True)
        expected_signature = hashlib.sha256(cred_str.encode('utf-8')).hexdigest()
        return expected_signature == signature

# --- Identity onboarding performance metrics ---
class IdentityMetrics:
    def __init__(self):
        self.onboarding_times = []
        self.verification_times = []

    def record_onboarding_time(self, t):
        self.onboarding_times.append(t)

    def record_verification_time(self, t):
        self.verification_times.append(t)

    def average_onboarding_time(self):
        if not self.onboarding_times:
            return 0
        return sum(self.onboarding_times) / len(self.onboarding_times)

    def average_verification_time(self):
        if not self.verification_times:
            return 0
        return sum(self.verification_times) / len(self.verification_times)

# Create instances for identity management
id_manager = DigitalIdentityManager()
id_metrics = IdentityMetrics()

# --- Digital onboarding phase for multiple users with performance measurement ---
print("\n=== Digital Onboarding Phase for multiple users ===")
try:
    nb_users = int(input("Enter the number of users to onboard: "))
except ValueError:
    print("Invalid number, defaulting to 1 user.")
    nb_users = 1

digital_identities = []
for i in range(1, nb_users + 1):
    user_id = f"User_{i}"
    user_attributes = {"name": f"User {i}", "dob": "1990-01-01", "nationality": "FR"}
    start_onboarding = time.time()
    digital_identity = id_manager.onboard_user(user_id, user_attributes)
    end_onboarding = time.time()
    onboarding_time = end_onboarding - start_onboarding
    id_metrics.record_onboarding_time(onboarding_time)

    start_verif = time.time()
    verification_ok = id_manager.verify_identity(digital_identity)
    end_verif = time.time()
    verification_time = end_verif - start_verif
    id_metrics.record_verification_time(verification_time)

    digital_identities.append(digital_identity)
    print(f"Digital Identity for {user_id}:", digital_identity)
    print(f"Onboarding time for {user_id}: {onboarding_time:.6f} seconds")
    if verification_ok:
        print(f"Identity verification succeeded for {user_id} in {verification_time:.6f} seconds.")
    else:
        print(f"Identity verification FAILED for {user_id} in {verification_time:.6f} seconds.")

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time:", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time:", id_metrics.average_verification_time(), "seconds")

# Export metrics to CSV
import csv
with open("identity_metrics.csv", "w", newline="") as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(["User_ID", "Onboarding_Time", "Verification_Time"])
    for i, identity in enumerate(digital_identities, 1):
        writer.writerow([f"User_{i}", id_metrics.onboarding_times[i-1], id_metrics.verification_times[i-1]])

# --- Transaction simulations ---
simulation = OfflinePaymentSimulation()
simulation.simulate_multiple_withdrawals(4, 50)
try:
    nb_offline = int(input("Enter number of offline transactions to perform: "))
except ValueError:
    print("Invalid number, defaulting to 1 offline transaction.")
    nb_offline = 1
try:
    offline_amount = int(input("Enter the amount for each offline transaction (in units): "))
except ValueError:
    print("Invalid amount, using a random amount.")
    offline_amount = None
simulation.simulate_multiple_offline_transactions(nb_offline, offline_amount)
try:
    nb_deposits = int(input("Enter number of deposits to perform: "))
except ValueError:
    print("Invalid number, defaulting to 1 deposit.")
    nb_deposits = 1
simulation.simulate_multiple_deposits(nb_deposits)

# --- New scenario: Bob pays Carla ---
try:
    bob_to_carla_amount = int(input("Enter the amount for Bob→Carla payment: "))
except ValueError:
    print("Invalid amount, using default 50 units.")
    bob_to_carla_amount = 50
simulation.simulate_bob_pays_carla(bob_to_carla_amount)

simulation.environment.generate_metrics()

# (After your other simulations, you can call the new funding method.)

# Ask the user the amount to deposit in the funding process
try:
    deposit_amount = int(input("Enter the amount for the Funding Operation (for Alice): "))
except ValueError:
    print("Invalid amount, using 1000 by default.")
    deposit_amount = 1000

# Shard Ledger configuration
shard_provider_url = "http://127.0.0.1:8545"  # Change the port if needed
shard_contract_address = "0xd9145CCE52D386f254917e481eB44e9943F39138"  # Replace with the deployed address via Remix for the Shard Ledger
shard_abi_file_path = "ShardLedger_ABI.json"  # Path to your Shard Ledger ABI file

# Create the Shard Ledger interface
shard_interface = ShardLedgerInterface(shard_provider_url, shard_contract_address, shard_abi_file_path)

# Information of the account sending the transaction (e.g., PAD account)
sender_address = "0x5B38Da6a701c568545dCfcB03FcB875f56beddC4"
sender_private_key = "0x4b0b6d1c3ced6d446803fc8b5726605bbba9aa95016bc07ec60ede51e117f8d0"

# Example bank account identifier for Alice (string)
A_B = "Alice_Fiat_Account_001"

funding_result = simulation.simulate_funding_operation(A_B, deposit_amount, shard_interface, sender_address, sender_private_key)

print("\n=== Digital Identity Metrics ===")
print("Average digital onboarding time:", id_metrics.average_onboarding_time(), "seconds")
print("Average identity verification time:", id_metrics.average_verification_time(), "seconds")
"""


